# 01. Setup & Environment (라이브러리 및 GPU 설정)

In [ ]:
"""
============================================================
  project_setup.py
  ML/DL 프로젝트 시작 시 1회 실행하는 최적화 설정 파일
  대상 환경: Windows 11 Pro
             Intel i9-14900K | RTX 4070 12GB
             CUDA 12.4 | cuDNN 9.19.0 | Python 3.12.10
============================================================
사용법:
    import project_setup          # 자동 실행
    또는
    from project_setup import setup
    cfg = setup()
============================================================
"""

import os
import sys
import platform
import warnings
import logging
import time

# ─────────────────────────────────────────────────────────
# 내부 출력 헬퍼
# ─────────────────────────────────────────────────────────
def _log(msg: str, level: str = "ok"):
    icons = {"ok": "✅", "warn": "⚠️ ", "err": "❌", "info": "📌", "title": "🚀"}
    print(f"  {icons.get(level, ' ')} {msg}")


# ─────────────────────────────────────────────────────────
# 1. 경고 & 로깅 설정
# ─────────────────────────────────────────────────────────
def _configure_warnings():
    # 불필요한 경고 억제
    warnings.filterwarnings("ignore", category=UserWarning)
    warnings.filterwarnings("ignore", category=FutureWarning)
    warnings.filterwarnings("ignore", category=DeprecationWarning)

    # 외부 라이브러리 경고 억제
    os.environ["TF_CPP_MIN_LOG_LEVEL"]        = "3"
    os.environ["TRANSFORMERS_VERBOSITY"]       = "error"
    os.environ["TOKENIZERS_PARALLELISM"]       = "false"
    os.environ["DATASETS_VERBOSITY"]           = "error"

    # Python 기본 로거 레벨
    logging.basicConfig(
        level=logging.WARNING,
        format="%(asctime)s [%(levelname)s] %(message)s",
        datefmt="%H:%M:%S",
    )
    # 외부 라이브러리 로거 억제
    for noisy in ["transformers", "datasets", "PIL", "matplotlib",
                  "urllib3", "filelock", "huggingface_hub"]:
        logging.getLogger(noisy).setLevel(logging.ERROR)

    _log("경고 & 로깅 설정 완료")


# ─────────────────────────────────────────────────────────
# 2. Windows 11 멀티프로세싱 설정
# ─────────────────────────────────────────────────────────
def _configure_multiprocessing():
    """
    Windows에서 DataLoader num_workers > 0 사용 시
    반드시 if __name__ == '__main__' 블록이 필요하거나,
    spawn context를 명시해야 함.
    여기서는 환경 변수만 최적화.
    """
    # i9-14900K: P코어 8개(HT=16) + E코어 16개 = 32스레드
    # DataLoader에 적합한 worker 수 (보통 P코어 수 = 8)
    cpu_count = os.cpu_count() or 8
    optimal_workers = min(8, cpu_count // 2)

    os.environ["OMP_NUM_THREADS"]      = str(optimal_workers)
    os.environ["MKL_NUM_THREADS"]      = str(optimal_workers)
    os.environ["OPENBLAS_NUM_THREADS"] = str(optimal_workers)
    os.environ["NUMEXPR_NUM_THREADS"]  = str(optimal_workers)

    _log(f"CPU 멀티스레드 설정 완료 (threads={optimal_workers}, cpu_total={cpu_count})")
    return optimal_workers


# ─────────────────────────────────────────────────────────
# 3. CUDA 활성화 & 컨텍스트 초기화
# ─────────────────────────────────────────────────────────
def activate_cuda(verbose: bool = True) -> dict:
    """
    CUDA 컨텍스트를 명시적으로 초기화하고 활성 상태를 검증합니다.

    일반적으로 첫 번째 CUDA 연산 시 자동 초기화되지만,
    이 함수를 호출하면:
      - 드라이버 레벨 초기화를 선행 완료
      - 첫 번째 실제 연산의 지연(latency) 제거
      - CUDA / cuDNN / 연산 파이프라인 이상 여부 사전 점검

    Returns
    -------
    dict — CUDA 활성화 결과 및 상세 정보
    """
    result = {
        "cuda_available"    : False,
        "context_initialized": False,
        "warmup_passed"     : False,
        "cuda_version"      : None,
        "cudnn_version"     : None,
        "driver_version"    : None,
        "device_name"       : None,
        "compute_capability": None,
        "vram_total_gb"     : None,
        "vram_free_gb"      : None,
    }

    if verbose:
        print()
        print("  ┌─────────────── CUDA 활성화 & 검증 ────────────────┐")

    try:
        import torch

        # ── Step 1: 기본 가용성 체크 ──────────────────────────
        if not torch.cuda.is_available():
            _log("CUDA 사용 불가 — 드라이버/설치 상태를 확인하세요", "err")
            if verbose:
                print("  └────────────────────────────────────────────────────┘")
            return result

        result["cuda_available"] = True
        result["cuda_version"]   = torch.version.cuda
        result["cudnn_version"]  = torch.backends.cudnn.version()

        # ── Step 2: 드라이버 버전 확인 ────────────────────────
        try:
            import subprocess
            smi = subprocess.run(
                ["nvidia-smi", "--query-gpu=driver_version",
                 "--format=csv,noheader"],
                capture_output=True, text=True, timeout=5
            )
            if smi.returncode == 0:
                result["driver_version"] = smi.stdout.strip()
        except Exception:
            result["driver_version"] = "확인 불가"

        # ── Step 3: 컨텍스트 초기화 (핵심) ───────────────────
        #   torch.cuda.init() : CUDA 런타임 컨텍스트 명시적 생성
        #   이후 연산에서 발생하는 ~1~2초 지연을 여기서 선 처리
        torch.cuda.init()
        torch.cuda.set_device(0)               # GPU 0번 명시적 선택
        result["context_initialized"] = True

        # ── Step 4: GPU 상세 정보 수집 ────────────────────────
        props = torch.cuda.get_device_properties(0)
        free_mem, total_mem = torch.cuda.mem_get_info(0)

        result.update({
            "device_name"       : torch.cuda.get_device_name(0),
            "compute_capability": f"{props.major}.{props.minor}",
            "vram_total_gb"     : round(total_mem / 1024**3, 2),
            "vram_free_gb"      : round(free_mem  / 1024**3, 2),
            "sm_count"          : props.multi_processor_count,
        })

        # ── Step 5: Warm-up 커널 실행 ─────────────────────────
        #   실제 CUDA 커널을 실행해 파이프라인 이상 여부 확인
        #   (1) 소형 텐서 할당 / 연산 / 동기화
        #   (2) cuDNN 컨볼루션 커널 실행 (cuDNN 컨텍스트 초기화)
        #   (3) Mixed Precision 연산 확인
        device = torch.device("cuda:0")

        # Warm-up 1: 기본 CUDA 연산
        _a = torch.ones(512, 512, device=device)
        _b = torch.ones(512, 512, device=device)
        _c = torch.mm(_a, _b)
        torch.cuda.synchronize()

        # Warm-up 2: cuDNN 컨볼루션 커널 초기화
        _conv_in  = torch.randn(1, 3, 64, 64, device=device)
        _conv     = torch.nn.Conv2d(3, 16, 3, padding=1).to(device)
        _conv_out = _conv(_conv_in)
        torch.cuda.synchronize()

        # Warm-up 3: Mixed Precision (FP16) 경로 확인
        _a16 = _a.half()
        _b16 = _b.half()
        _c16 = torch.mm(_a16, _b16)
        torch.cuda.synchronize()

        # Warm-up 텐서 정리
        del _a, _b, _c, _a16, _b16, _c16
        del _conv_in, _conv, _conv_out
        torch.cuda.empty_cache()

        result["warmup_passed"] = True

        # ── Step 6: 결과 출력 ─────────────────────────────────
        if verbose:
            ok  = "✅"
            vram_used = result["vram_total_gb"] - result["vram_free_gb"]
            bar_len   = 25
            ratio     = vram_used / result["vram_total_gb"]
            filled    = int(bar_len * ratio)
            bar       = "█" * filled + "░" * (bar_len - filled)

            print(f"  │  {ok} CUDA {result['cuda_version']}  |  "
                  f"cuDNN {result['cudnn_version']}  |  "
                  f"Driver {result['driver_version']}")
            print(f"  │  {ok} GPU   : {result['device_name']}")
            print(f"  │  {ok} CC    : {result['compute_capability']}  |  "
                  f"SM: {result['sm_count']}개")
            print(f"  │  {ok} VRAM  : [{bar}] "
                  f"{vram_used:.2f}/{result['vram_total_gb']:.2f} GB "
                  f"(여유 {result['vram_free_gb']:.2f} GB)")
            print(f"  │  {ok} Context 초기화 완료")
            print(f"  │  {ok} Warm-up 완료 (CUDA / cuDNN / FP16)")
            print("  └────────────────────────────────────────────────────┘")
            print()

    except RuntimeError as e:
        _log(f"CUDA 활성화 실패: {e}", "err")
        if verbose:
            print("  └────────────────────────────────────────────────────┘")

    except ImportError:
        _log("PyTorch 미설치 → CUDA 활성화 불가", "err")
        if verbose:
            print("  └────────────────────────────────────────────────────┘")

    return result


# ─────────────────────────────────────────────────────────
# 3-B. CUDA / PyTorch 최적화
# ─────────────────────────────────────────────────────────
def _configure_pytorch():
    info = {}
    try:
        import torch

        # ── GPU 선택 ─────────────────────────────────────
        os.environ["CUDA_DEVICE_ORDER"]   = "PCI_BUS_ID"
        os.environ["CUDA_VISIBLE_DEVICES"] = "0"          # RTX 4070 단일 GPU

        # ── VRAM 단편화 방지 (12GB에 최적화) ─────────────
        os.environ["PYTORCH_CUDA_ALLOC_CONF"] = (
            "max_split_size_mb:512,"
            "expandable_segments:True,"
            "garbage_collection_threshold:0.8"
        )

        if not torch.cuda.is_available():
            _log("CUDA 사용 불가 → CPU 모드", "warn")
            info["device"] = torch.device("cpu")
            return info

        device = torch.device("cuda:0")

        # ── cuDNN 최적화 ──────────────────────────────────
        torch.backends.cudnn.enabled        = True
        torch.backends.cudnn.benchmark      = True   # 입력 크기 고정 시 최고 속도
        torch.backends.cudnn.deterministic  = False  # benchmark=True와 병행 시 False 권장
        torch.backends.cudnn.allow_tf32     = True   # Ampere 이상 TF32 가속

        # ── TF32 (RTX 4070 Ada 아키텍처 가속) ─────────────
        torch.backends.cuda.matmul.allow_tf32 = True
        torch.backends.cuda.matmul.allow_fp16_reduced_precision_reduction = True

        # ── 메모리 사전 준비 (단편화 방지용 warm-up) ──────
        _dummy = torch.zeros(1, device=device)
        del _dummy
        torch.cuda.empty_cache()

        # ── 정보 수집 ─────────────────────────────────────
        props = torch.cuda.get_device_properties(0)
        total_vram = props.total_memory / 1024**3

        info = {
            "device"       : device,
            "gpu_name"     : torch.cuda.get_device_name(0),
            "cuda_version" : torch.version.cuda,
            "cudnn_version": torch.backends.cudnn.version(),
            "vram_total_gb": round(total_vram, 2),
            "torch_version": torch.__version__,
            "compute_cap"  : f"{props.major}.{props.minor}",
            "sm_count"     : props.multi_processor_count,
        }

        _log(f"PyTorch {torch.__version__} | CUDA {torch.version.cuda} | "
             f"cuDNN {torch.backends.cudnn.version()}")
        _log(f"GPU: {info['gpu_name']} | VRAM: {total_vram:.2f} GB | "
             f"SM: {props.multi_processor_count} | CC: {props.major}.{props.minor}")
        _log("cuDNN benchmark=True | TF32=True | VRAM 단편화 방지 설정 완료")

    except ImportError:
        _log("PyTorch 미설치", "err")

    return info


# ─────────────────────────────────────────────────────────
# 4. NumPy / Pandas 최적화
# ─────────────────────────────────────────────────────────
def _configure_numpy_pandas():
    try:
        import numpy as np
        # MKL/OpenBLAS 사용 확인
        np_config = np.__config__
        _log(f"NumPy {np.__version__} 로드 완료")
    except ImportError:
        _log("NumPy 미설치", "err")

    try:
        import pandas as pd
        # Copy-on-Write (pandas 2.0+) 활성화 → 메모리 효율 향상
        pd.options.mode.copy_on_write            = True
        pd.options.display.max_columns           = 50
        pd.options.display.max_rows              = 100
        pd.options.display.float_format          = "{:.4f}".format
        pd.options.display.max_colwidth          = 80
        _log(f"Pandas {pd.__version__} | Copy-on-Write=True 설정 완료")
    except ImportError:
        _log("Pandas 미설치", "err")


# ─────────────────────────────────────────────────────────
# 5. Matplotlib 설정
# ─────────────────────────────────────────────────────────
def _configure_matplotlib():
    try:
        import matplotlib
        matplotlib.use("Agg")                          # Windows headless 안전 백엔드
        import matplotlib.pyplot as plt
        plt.rcParams.update({
            "figure.figsize"     : (12, 6),
            "figure.dpi"         : 120,
            "axes.grid"          : True,
            "grid.alpha"         : 0.3,
            "axes.spines.top"    : False,
            "axes.spines.right"  : False,
            "font.size"          : 11,
        })
        _log(f"Matplotlib {matplotlib.__version__} | Agg 백엔드 설정 완료")
    except ImportError:
        _log("Matplotlib 미설치", "err")


# ─────────────────────────────────────────────────────────
# 6. 재현성(Seed) 고정 유틸
# ─────────────────────────────────────────────────────────
def set_seed(seed: int = 42):
    """
    모든 난수 시드를 고정합니다.
    완전한 재현성이 필요할 때 호출하세요.

    주의: cuDNN deterministic=True로 전환되어 속도가 약간 감소할 수 있습니다.
    """
    import random
    random.seed(seed)

    try:
        import numpy as np
        np.random.seed(seed)
    except ImportError:
        pass

    try:
        import torch
        torch.manual_seed(seed)
        if torch.cuda.is_available():
            torch.cuda.manual_seed(seed)
            torch.cuda.manual_seed_all(seed)
            torch.backends.cudnn.deterministic = True
            torch.backends.cudnn.benchmark     = False   # deterministic 시 False
    except ImportError:
        pass

    os.environ["PYTHONHASHSEED"] = str(seed)
    print(f"  🎲 Seed {seed} 고정 완료 (random / numpy / torch / PYTHONHASHSEED)")


# ─────────────────────────────────────────────────────────
# 7. GPU 메모리 상태 출력 유틸
# ─────────────────────────────────────────────────────────
def gpu_memory_status():
    """현재 GPU 메모리 사용량을 출력합니다."""
    try:
        import torch
        if not torch.cuda.is_available():
            print("  ⚠️  CUDA 사용 불가")
            return
        allocated = torch.cuda.memory_allocated() / 1024**3
        reserved  = torch.cuda.memory_reserved()  / 1024**3
        total     = torch.cuda.get_device_properties(0).total_memory / 1024**3
        free      = total - allocated
        bar_len   = 30
        used_ratio = allocated / total
        filled    = int(bar_len * used_ratio)
        bar       = "█" * filled + "░" * (bar_len - filled)
        print(f"\n  💾 GPU 메모리 현황 [{bar}] {used_ratio*100:.1f}%")
        print(f"     사용중: {allocated:.2f} GB  |  예약: {reserved:.2f} GB  "
              f"|  여유: {free:.2f} GB  |  전체: {total:.2f} GB")
    except ImportError:
        print("  ❌ PyTorch 미설치")


# ─────────────────────────────────────────────────────────
# 8. GPU 메모리 정리 유틸
# ─────────────────────────────────────────────────────────
def clear_gpu_memory():
    """GPU 캐시를 비우고 가비지 컬렉션을 실행합니다."""
    import gc
    gc.collect()
    try:
        import torch
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            torch.cuda.synchronize()
            print("  🧹 GPU 메모리 캐시 정리 완료")
    except ImportError:
        pass


# ─────────────────────────────────────────────────────────
# 9. 권장 하이퍼파라미터 프리셋 (RTX 4070 기준)
# ─────────────────────────────────────────────────────────
def get_recommended_config(task: str = "image_classification") -> dict:
    """
    RTX 4070 12GB 기준 작업별 권장 설정을 반환합니다.

    task 옵션:
        "image_classification" | "object_detection" |
        "segmentation"         | "nlp_bert"         |
        "nlp_llm"              | "tabular"
    """
    base = {
        "device"       : "cuda",
        "num_workers"  : 8,          # i9-14900K P코어 기준
        "pin_memory"   : True,       # CPU→GPU 전송 속도 향상
        "prefetch_factor": 2,
        "persistent_workers": True,
        "use_amp"      : True,       # Mixed Precision (FP16)
    }

    presets = {
        "image_classification": {
            **base,
            "batch_size"   : 64,
            "image_size"   : 224,
            "optimizer"    : "AdamW",
            "lr"           : 1e-4,
            "weight_decay" : 1e-2,
            "epochs"       : 50,
            "scheduler"    : "CosineAnnealingLR",
            "note"         : "ResNet50/EfficientNet/ViT 기준",
        },
        "object_detection": {
            **base,
            "batch_size"   : 16,
            "image_size"   : 640,
            "optimizer"    : "SGD",
            "lr"           : 1e-2,
            "epochs"       : 100,
            "note"         : "YOLOv8/Faster-RCNN 기준",
        },
        "segmentation": {
            **base,
            "batch_size"   : 8,
            "image_size"   : 512,
            "optimizer"    : "AdamW",
            "lr"           : 3e-4,
            "epochs"       : 100,
            "note"         : "U-Net/SegFormer 기준",
        },
        "nlp_bert": {
            **base,
            "batch_size"   : 32,
            "max_length"   : 512,
            "optimizer"    : "AdamW",
            "lr"           : 2e-5,
            "weight_decay" : 1e-2,
            "epochs"       : 5,
            "warmup_ratio" : 0.1,
            "note"         : "BERT/RoBERTa 파인튜닝 기준",
        },
        "nlp_llm": {
            **base,
            "batch_size"   : 4,
            "gradient_accumulation_steps": 8,   # 유효 배치 = 32
            "max_length"   : 2048,
            "optimizer"    : "AdamW",
            "lr"           : 2e-4,
            "epochs"       : 3,
            "use_lora"     : True,
            "lora_r"       : 16,
            "lora_alpha"   : 32,
            "note"         : "LLM LoRA 파인튜닝 기준 (12GB 한계 내)",
        },
        "tabular": {
            "device"       : "cuda",
            "batch_size"   : 4096,
            "optimizer"    : "AdamW",
            "lr"           : 1e-3,
            "epochs"       : 100,
            "num_workers"  : 4,
            "note"         : "TabNet/MLP 기준",
        },
    }

    cfg = presets.get(task, base)
    print(f"\n  📋 [{task}] 권장 설정")
    for k, v in cfg.items():
        print(f"     {k:<35} = {v}")
    return cfg


# ─────────────────────────────────────────────────────────
# 10. Mixed Precision Scaler 팩토리
# ─────────────────────────────────────────────────────────
def get_amp_scaler():
    """
    RTX 4070용 Mixed Precision GradScaler를 반환합니다.

    사용 예:
        scaler = get_amp_scaler()
        with torch.autocast(device_type='cuda'):
            loss = model(x)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
    """
    try:
        import torch
        if torch.cuda.is_available():
            scaler = torch.amp.GradScaler(
                init_scale=2.**16,
                growth_factor=2.0,
                backoff_factor=0.5,
                growth_interval=2000,
                enabled=True,
            )
            _log("GradScaler (Mixed Precision FP16) 준비 완료")
            return scaler
    except ImportError:
        _log("PyTorch 미설치 → GradScaler 반환 불가", "err")
    return None


# ─────────────────────────────────────────────────────────
# 메인 setup() 함수
# ─────────────────────────────────────────────────────────
def setup(seed: int | None = None, verbose: bool = True) -> dict:
    """
    프로젝트 시작 시 모든 최적화 설정을 한 번에 실행합니다.

    Parameters
    ----------
    seed    : int | None — 지정 시 재현성 고정 (기본: None = 고정 안 함)
    verbose : bool       — 시스템 요약 출력 여부

    Returns
    -------
    dict — device, gpu_name, vram 등 주요 정보
    """
    t_start = time.time()

    print()
    print("  ╔══════════════════════════════════════════════════╗")
    print("  ║      ML / DL 프로젝트 환경 초기화 중...         ║")
    print("  ║  i9-14900K | RTX 4070 12GB | CUDA 12.4          ║")
    print("  ╚══════════════════════════════════════════════════╝")
    print()

    _configure_warnings()
    workers   = _configure_multiprocessing()
    cuda_info = activate_cuda(verbose=True)     # CUDA 활성화 & 검증
    gpu_info  = _configure_pytorch()            # PyTorch 세부 최적화
    _configure_numpy_pandas()
    _configure_matplotlib()

    if seed is not None:
        set_seed(seed)

    elapsed = time.time() - t_start

    result = {
        **gpu_info,
        "optimal_num_workers": workers,
        "platform"           : platform.platform(),
        "python_version"     : sys.version.split()[0],
    }

    if verbose:
        print()
        print("  ┌─────────────────── 시스템 요약 ───────────────────┐")
        print(f"  │  OS       : {platform.platform()[:50]}")
        print(f"  │  Python   : {sys.version.split()[0]}")
        if "gpu_name" in result:
            print(f"  │  GPU      : {result['gpu_name']}")
            print(f"  │  VRAM     : {result['vram_total_gb']} GB")
            print(f"  │  CUDA     : {result.get('cuda_version','?')}  "
                  f"cuDNN: {result.get('cudnn_version','?')}")
        print(f"  │  Workers  : {workers}")
        print(f"  │  초기화   : {elapsed:.2f}초 완료")
        print("  └────────────────────────────────────────────────────┘")
        print()
        print("  📌 사용 가능한 유틸 함수:")
        print("     activate_cuda()            → CUDA 활성화 & 검증 (재실행)")
        print("     set_seed(42)               → 재현성 시드 고정")
        print("     gpu_memory_status()        → VRAM 사용량 확인")
        print("     clear_gpu_memory()         → GPU 캐시 비우기")
        print("     get_recommended_config()   → 작업별 권장 설정")
        print("     get_amp_scaler()           → FP16 GradScaler 반환")
        print()

    return result


# ─────────────────────────────────────────────────────────
# import 시 자동 실행
# ─────────────────────────────────────────────────────────
_config = setup()
device  = _config.get("device", None)      # 전역으로 바로 사용 가능

# 02. Data Ingestion & Integration (데이터 로드 및 통합)

In [ ]:
import os
import pandas as pd
import pyreadstat
import numpy as np

BASE_DIR = r"C:\Users\sck32\Desktop\Clinical Diabetes TabNet\Data"
YEARS = ['19', '20', '21', '22', '23', '24']

def load_sas(path) :
    try :
        df, _ = pyreadstat.read_sas7bdat(path, encoding = 'cp949')
        return df
    except UnicodeDecodeError :
        df, _ = pyreadstat.read_sas7bdat(path, encoding = 'latin1')
        return df

all_year_list = []

for y in YEARS :
    all_file = os.path.join(BASE_DIR, f"hn{y}_all.sas7bdat")
    rc_file = os.path.join(BASE_DIR, f"hn{y}_24rc.sas7bdat")
    
    if os.path.exists(all_file) :
        df_all = load_sas(all_file)
        df_all['ID'] = df_all['ID'].astype(str).str.strip().str.upper()
        
        # 영양조사(24rc) 세부 데이터 병합
        if os.path.exists(rc_file) :
            df_rc = load_sas(rc_file)
            df_rc.columns = [c.upper() for c in df_rc.columns]
            df_rc['ID'] = df_rc['ID'].astype(str).str.strip().str.upper()
            
            # 영양소 섭취 데이터 세부 집계
            agg_cols = {
                'NF_INTK' : 'sum', 
                'NF_CODE' : 'nunique',
                'N_EN' : 'mean',
                'N_PROT' : 'mean',
                'N_FAT' : 'mean',
                'N_CHO' : 'mean'
            }
            agg_dict = {k : v for k, v in agg_cols.items() if k in df_rc.columns}
            
            df_rc_agg = df_rc.groupby('ID').agg(agg_dict).reset_index()
            df_all = pd.merge(df_all, df_rc_agg, on = 'ID', how = 'left')
            print(f"✅ 20{y}년 : 임상/건강설문 + 세부 영양 데이터 통합 완료")
        else :
            print(f"⚠️ 20{y}년 : 임상/건강설문 데이터만 통합 (영양 파일 누락)")
            
        all_year_list.append(df_all)

raw_df = pd.concat(all_year_list, axis = 0, ignore_index = True)
print(f"\n📌 통합 데이터셋 형태 : {raw_df.shape[0] :,} 행, {raw_df.shape[1] :,} 열")

# 03. Data Preprocessing (Feature Extraction)

In [ ]:
import numpy as np

# 1. 통합 가중치 및 기본 정제
processed_df = raw_df.copy()
processed_df['wt_pooled'] = processed_df['wt_itvex'] / 6

def clean_missing(df) :
    num_cols = df.select_dtypes(include = [np.number]).columns
    df[num_cols] = df[num_cols].replace([8, 9, 88, 99, 888, 999, 9999], np.nan)
    return df

processed_df = clean_missing(processed_df)

# 2. 분석 도메인별 변수 확정
virus_cols = ['HE_hepaB', 'HE_hepaC']
genetic_cols = [c for c in processed_df.columns if any(x in c.upper() for x in ['HE_FH_DIA', 'HE_FH_HT', 'HE_FH_HDL', 'HE_FH_IHD'])]
metabolic_cols = [
    'HE_glu', 'HE_HbA1c', 'HE_chol', 'HE_TG', 'HE_HDL_st2', 
    'HE_ast', 'HE_alt', 'HE_BMI', 'HE_wc', 'HE_sbp', 'HE_dbp',
    'HE_crea', 'HE_BUN', 'HE_WBC', 'HE_Upro'
]
nutrition_cols = ['N_EN', 'N_CHO', 'N_PROT', 'N_FAT', 'N_NA', 'N_K', 'age', 'sex', 'sm_presnt']

all_features = virus_cols + genetic_cols + metabolic_cols + nutrition_cols
valid_features = [c for c in all_features if c in processed_df.columns]

# 3. 데이터 필터링 (핵심 대사 지표 기준)
core_metrics = ['HE_glu', 'HE_HbA1c', 'HE_TG', 'HE_BMI']
analysis_df = processed_df.dropna(subset = core_metrics).copy()

# 4. 이상치 처리 (Winsorization : 상하위 0.5% 클리핑)
for col in metabolic_cols :
    if col in analysis_df.columns :
        lower = analysis_df[col].quantile(0.005)
        upper = analysis_df[col].quantile(0.995)
        analysis_df[col] = analysis_df[col].clip(lower, upper)

# 5. 타겟 라벨링
analysis_df['target_diabetes'] = analysis_df['DE1_dg'].map({0 : 0, 1 : 1})

print(f"✅ 확장된 유효 변수 총 개수 : {len(valid_features) :,}개")
print(f"✅ 정제된 최종 샘플 수 : {len(analysis_df) :,}명")

# 04. Data Imputation & Scaling

## 4-1 Robust Imputation & Scaling

In [ ]:
import pandas as pd
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import StandardScaler
import torch

# 분석 대상 피처 추출
X_raw = analysis_df[valid_features].copy()

# 다중 대치법(MICE) 기반 결측치 처리 수행
imputer = IterativeImputer(
    max_iter=20, 
    random_state=42, 
    verbose=0
)

print(f"🚀 결측치 대치 시작 (Sample Size: {len(X_raw):,})")
X_imputed = imputer.fit_transform(X_raw)

# 데이터 표준화 수행 (Z-score Scaling)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

# GPU 텐서 변환 및 장치 할당
X_tensor = torch.FloatTensor(X_scaled).to(device)

print(f"✅ 텐서 변환 및 {device} 할당 완료")
print(f"📊 최종 데이터 형태 : {X_tensor.shape[0]:,} rows × {X_tensor.shape[1]:,} cols")

## 4-2. Imputation Integrity Check

In [ ]:
# 대치 완료 데이터프레임 생성
X_check = pd.DataFrame(X_imputed, columns=valid_features)

# 주요 대사 지표 평균치 변동 확인
target_cols = ['HE_glu', 'HE_BMI', 'HE_TG']
summary = pd.concat([
    X_raw[target_cols].mean().rename('Raw_Mean'),
    X_check[target_cols].mean().rename('Imputed_Mean')
], axis=1)

# 대치 전후 오차율 계산
summary['Diff_Ratio(%)'] = abs((summary['Imputed_Mean'] - summary['Raw_Mean']) / summary['Raw_Mean']) * 100

print("📋 데이터 정합성 검증 결과")
print("-" * 50)
print(summary)

## 4-3. Skewness Correction & Re-scaling

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import torch

skewed_features = ['HE_glu', 'HE_TG', 'HE_ast', 'HE_alt', 'HE_crea', 'N_EN']
X_refined_df = pd.DataFrame(X_imputed, columns=valid_features)

for col in skewed_features:
    if col in X_refined_df.columns:
        # 음수 값을 0으로 강제 클리핑하여 log1p 연산 시 발생하는 NaN 원천 차단
        X_refined_df[col] = np.log1p(np.clip(X_refined_df[col], a_min=0, a_max=None))

scaler_refined = StandardScaler()
X_scaled_refined = scaler_refined.fit_transform(X_refined_df)

X_tensor = torch.FloatTensor(X_scaled_refined).to(device)

print(f"✅ 주요 {len(skewed_features)}개 지표 로그 변환 완료 (NaN 방지 적용)")
print(f"✅ 최종 데이터 스케일링 및 {device} 할당 완료")
print(f"✅ NaN 포함 여부: {torch.isnan(X_tensor).any().item()}")

# 05. Model Training & VRAM Cleanup

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import time
import gc

class PhenotypeAE(nn.Module) :
    def __init__(self, input_dim : int, latent_dim : int = 8) :
        super(PhenotypeAE, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 16),
            nn.BatchNorm1d(16),
            nn.GELU(),
            nn.Linear(16, latent_dim)
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 16),
            nn.BatchNorm1d(16),
            nn.GELU(),
            nn.Linear(16, input_dim)
        )

    def forward(self, x : torch.Tensor) :
        latent = self.encoder(x)
        reconstructed = self.decoder(latent)
        return latent, reconstructed

cfg = {
    "device" : "cuda:0",
    "epochs" : 200,
    "batch_size" : 512,
    "lr" : 0.0005,
    "weight_decay" : 1e-4,
    "latent_dim" : 8
}

model = PhenotypeAE(input_dim = X_tensor.shape[1], latent_dim = cfg["latent_dim"]).to(cfg["device"])
criterion = nn.MSELoss()
optimizer = optim.AdamW(model.parameters(), lr = cfg["lr"], weight_decay = cfg["weight_decay"])

dataset = TensorDataset(X_tensor)
dataloader = DataLoader(dataset, batch_size = cfg["batch_size"], shuffle = True)

model.train()
start_time = time.time()

print("🚀 학습 시작")
for epoch in range(cfg["epochs"]) :
    running_loss : float = 0.0
    for batch in dataloader :
        inputs = batch[0]
        optimizer.zero_grad()
        
        latent, reconstructed = model(inputs)
        loss = criterion(reconstructed, inputs)
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm = 1.0)
        optimizer.step()
        running_loss += loss.item()
    
    avg_loss : float = running_loss / len(dataloader)
    if (epoch + 1) % 20 == 0 :
        print(f"Epoch [{epoch+1}/{cfg['epochs']}] | Loss : {avg_loss:.6f}")

print(f"✅ 학습 완료 (소요 시간 : {time.time() - start_time:.2f}s)")

model.eval()
with torch.no_grad() :
    latent_vectors, _ = model(X_tensor)
    latent_np = latent_vectors.cpu().numpy()

latent_cols = [f"latent_{i}" for i in range(cfg["latent_dim"])]
latent_df = pd.DataFrame(latent_np, columns = latent_cols)
print(f"✅ 잠재 변수 추출 완료 : {latent_df.shape}")

del model, optimizer, dataloader, dataset
if 'inputs' in locals() : del inputs
if 'reconstructed' in locals() : del reconstructed
if 'loss' in locals() : del loss
if 'latent' in locals() : del latent
if 'batch' in locals() : del batch

gc.collect()
torch.cuda.empty_cache()
torch.cuda.synchronize()

# 06. Latent Space Visualization (Streaming GPU t-SNE)

In [ ]:
import torch
import torch.optim as optim
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import gc

def limit_vram_usage(target_gb : float = 10.0, total_gb : float = 12.0) :
    if torch.cuda.is_available() :
        fraction : float = target_gb / total_gb
        torch.cuda.set_per_process_memory_fraction(fraction, device = 0)
        print(f"🔒 PyTorch VRAM 사용 상한선 설정 : {target_gb}GB")

def gpu_tsne_bulletproof(data : torch.Tensor, n_components : int = 2, lr : float = 100.0, iterations : int = 1000, chunk_size : int = 3000) :
    gc.collect()
    torch.cuda.empty_cache()
    
    n : int = data.shape[0]
    X = data.to(device).float()
    Y = torch.randn(n, n_components, device = device, dtype = torch.float32) * 1e-4
    Y.requires_grad = True
    
    P_cpu = torch.zeros((n, n), dtype = torch.half)
    print(f"📊 P-Matrix 이중 분할 연산 중... (Max Tensor : {chunk_size}x{chunk_size})")
    
    with torch.no_grad() :
        for i in range(0, n, chunk_size) :
            end_i = min(i + chunk_size, n)
            p_row_sum = torch.zeros((end_i - i, 1), device = device, dtype = torch.float32)
            
            for j in range(0, n, chunk_size) :
                end_j = min(j + chunk_size, n)
                dist_sub = torch.cdist(X[i : end_i], X[j : end_j]).pow(2)
                
                # 분산(Variance) 축소 : 8차원 잠재 공간 스케일에 맞춘 보정 값(8.0) 적용
                p_sub = torch.exp(-dist_sub / 8.0)
                
                row_idx = torch.arange(i, end_i).unsqueeze(1)
                col_idx = torch.arange(j, end_j).unsqueeze(0)
                mask = (row_idx == col_idx).to(device)
                p_sub.masked_fill_(mask, 0.0)
                
                P_cpu[i : end_i, j : end_j] = p_sub.half().cpu()
                p_row_sum += p_sub.sum(dim = 1, keepdim = True)
                
                del dist_sub, p_sub, mask
                torch.cuda.empty_cache()
            
            # Zero Division 방지 안전장치
            P_cpu[i : end_i, :] /= (p_row_sum.cpu() + 1e-12)

        P_cpu = (P_cpu + P_cpu.t()) / (2 * n)
        P_cpu = torch.clamp(P_cpu, min = 1e-12)

    optimizer = optim.SGD([Y], lr = lr, momentum = 0.8)
    
    print(f"🚀 안정화된 2D Chunked GPU 최적화 시작 (Iterations : {iterations})")
    for step in range(iterations) :
        optimizer.zero_grad()
        
        exaggeration : float = 12.0 if step < 250 else 1.0
        
        sum_inv_q : float = 0.0
        with torch.no_grad() :
            for i in range(0, n, chunk_size) :
                end_i = min(i + chunk_size, n)
                for j in range(0, n, chunk_size) :
                    end_j = min(j + chunk_size, n)
                    
                    dy_sub = torch.cdist(Y[i : end_i], Y[j : end_j]).pow(2)
                    inv_q_sub = 1.0 / (1.0 + dy_sub)
                    
                    row_idx = torch.arange(i, end_i).unsqueeze(1)
                    col_idx = torch.arange(j, end_j).unsqueeze(0)
                    mask = (row_idx == col_idx).to(device)
                    inv_q_sub.masked_fill_(mask, 0.0)
                    
                    sum_inv_q += inv_q_sub.sum().item()
                    del dy_sub, inv_q_sub, mask
        
        # [핵심] NaN 전파 원인 차단 : 분모가 0이 되는 현상 방어
        sum_inv_q = max(sum_inv_q, 1e-12)
        
        total_loss : float = 0.0
        grad_Y = torch.zeros_like(Y)
        
        with torch.no_grad() :
            for i in range(0, n, chunk_size) :
                end_i = min(i + chunk_size, n)
                grad_chunk = torch.zeros_like(Y[i : end_i])
                
                for j in range(0, n, chunk_size) :
                    end_j = min(j + chunk_size, n)
                    
                    p_sub = P_cpu[i : end_i, j : end_j].to(device).float() * exaggeration
                    p_sub = torch.clamp(p_sub, min = 1e-12)
                    
                    dy_sub = torch.cdist(Y[i : end_i], Y[j : end_j]).pow(2)
                    inv_q_sub = 1.0 / (1.0 + dy_sub)
                    
                    row_idx = torch.arange(i, end_i).unsqueeze(1)
                    col_idx = torch.arange(j, end_j).unsqueeze(0)
                    mask = (row_idx == col_idx).to(device)
                    inv_q_sub.masked_fill_(mask, 0.0)
                    
                    q_sub = inv_q_sub / sum_inv_q
                    q_sub = torch.clamp(q_sub, min = 1e-12)
                    
                    loss_sub = (p_sub * torch.log(p_sub / q_sub)).sum().item()
                    total_loss += loss_sub
                    
                    pq_diff = 4.0 * (p_sub - q_sub) * inv_q_sub
                    grad_chunk += pq_diff.sum(dim = 1, keepdim = True) * Y[i : end_i] - torch.mm(pq_diff, Y[j : end_j])
                    
                    del p_sub, dy_sub, inv_q_sub, q_sub, pq_diff, mask
                
                grad_Y[i : end_i] = grad_chunk
                del grad_chunk
        
        # [핵심] Gradient NaN 강제 치환 및 클리핑
        if torch.isnan(grad_Y).any() or torch.isinf(grad_Y).any() :
            grad_Y = torch.nan_to_num(grad_Y, nan = 0.0, posinf = 1.0, neginf = -1.0)
            
        Y.grad = grad_Y
        torch.nn.utils.clip_grad_norm_([Y], max_norm = 1.0)
        optimizer.step()
        
        if (step + 1) % 100 == 0 :
            print(f"Iteration [{step+1}/{iterations}] | Loss : {total_loss:.4f}")
            torch.cuda.empty_cache()
            
    return Y.detach().cpu().numpy()

# 1. 10GB 상한선 설정
limit_vram_usage()

# 2. 파라미터 딕셔너리 구성
cfg_tsne = {
    "chunk_size" : 3000,
    "lr" : 100.0
}

# 3. 모델 실행
latent_2d = gpu_tsne_bulletproof(latent_vectors, chunk_size = cfg_tsne["chunk_size"], lr = cfg_tsne["lr"])

# 4. 결과 시각화
viz_df = pd.DataFrame(latent_2d, columns = ['tsne_1', 'tsne_2'])
viz_df['diabetes'] = analysis_df['target_diabetes'].values

plt.figure(figsize = (12, 8))
sns.scatterplot(x = 'tsne_1', y = 'tsne_2', hue = 'diabetes', data = viz_df, palette = 'coolwarm', alpha = 0.5, s = 20)
plt.title('2D Chunked GPU t-SNE (Bulletproof Math Optimized)')
plt.show()

# 07. Subtype Discovery (GPU GMM Clustering)

In [ ]:
import torch
import torch.nn as nn
import pandas as pd

class GPU_GMM :
    def __init__(self, n_components : int, n_iters : int = 100, device : str = "cuda:0") :
        self.n_components = n_components
        self.n_iters = n_iters
        self.device = device

    def fit_predict(self, x : torch.Tensor) :
        n_samples, n_features = x.shape
        x = x.to(self.device).float()

        self.pi = torch.ones(self.n_components, device = self.device) / self.n_components
        
        idx = torch.randperm(n_samples)[:self.n_components]
        self.mu = x[idx, :].clone()
        self.sigma = torch.eye(n_features, device = self.device).unsqueeze(0).repeat(self.n_components, 1, 1)

        print(f"🧪 GPU GMM 클러스터링 시작 (K : {self.n_components}, Iterations : {self.n_iters})")
        for i in range(self.n_iters) :
            with torch.no_grad() :
                probs = []
                for k in range(self.n_components) :
                    cov = self.sigma[k]
                    cov = (cov + cov.t()) / 2.0
                    cov = cov + torch.eye(n_features, device = self.device) * 1e-3
                    
                    dist = torch.distributions.MultivariateNormal(self.mu[k], cov)
                    probs.append(dist.log_prob(x).exp() * self.pi[k])
                
                probs = torch.stack(probs, dim = 1)
                gamma = probs / (probs.sum(dim = 1, keepdim = True) + 1e-12)

                n_k = gamma.sum(dim = 0)
                self.pi = n_k / n_samples
                self.mu = torch.mm(gamma.t(), x) / n_k.view(-1, 1)
                
                for k in range(self.n_components) :
                    diff = x - self.mu[k]
                    # 안전한 행렬 곱셈 구조로 변경
                    cov_update = torch.mm((gamma[:, k].view(-1, 1) * diff).t(), diff) / (n_k[k] + 1e-12)
                    self.sigma[k] = (cov_update + cov_update.t()) / 2.0

        final_probs = []
        for k in range(self.n_components) :
            cov = self.sigma[k]
            cov = (cov + cov.t()) / 2.0
            cov = cov + torch.eye(n_features, device = self.device) * 1e-3
            dist = torch.distributions.MultivariateNormal(self.mu[k], cov)
            final_probs.append(dist.log_prob(x))
        
        return torch.stack(final_probs, dim = 1).argmax(dim = 1).cpu().numpy()

cfg_gmm = {
    "n_components" : 9,
    "n_iters" : 150,
    "device" : "cuda:0"
}

gmm_model = GPU_GMM(n_components = cfg_gmm["n_components"], n_iters = cfg_gmm["n_iters"], device = cfg_gmm["device"])
metabolic_subtypes = gmm_model.fit_predict(latent_vectors)

analysis_df['metabolic_subtype'] = metabolic_subtypes

summary_df = analysis_df.groupby('metabolic_subtype')['target_diabetes'].agg(['count', 'mean']).reset_index()
summary_df.columns = ['Subtype', 'Count', 'Diabetes_Prevalence']
summary_df = summary_df.sort_values(by = 'Diabetes_Prevalence', ascending = False)

print("\n📋 대사 아형(Subtype)별 당뇨 유병률 요약")
print("-" * 55)
print(summary_df.to_string(index = False))

# 08. High Risk Subtype Prediction (TabNet)

In [ ]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, classification_report
from pytorch_tabnet.tab_model import TabNetClassifier

high_risk_clusters : list = [1, 7]
analysis_df['is_high_risk'] = analysis_df['metabolic_subtype'].isin(high_risk_clusters).astype(int)

X_data : np.ndarray = X_scaled_refined
y_data : np.ndarray = analysis_df['is_high_risk'].values

X_train, X_test, y_train, y_test = train_test_split(
    X_data, y_data, test_size = 0.2, random_state = 42, stratify = y_data
)

tabnet_params : dict = {
    "n_d" : 64,
    "n_a" : 64,
    "n_steps" : 5,
    "gamma" : 1.5,
    "n_independent" : 2,
    "n_shared" : 2,
    "lambda_sparse" : 1e-4,
    "optimizer_fn" : torch.optim.AdamW,
    "optimizer_params" : {"lr" : 2e-2, "weight_decay" : 1e-4},
    "scheduler_fn" : torch.optim.lr_scheduler.CosineAnnealingLR,
    "scheduler_params" : {"T_max" : 150},
    "device_name" : "cuda",
    "verbose" : 1  # 로그 활성화 (진행률 확인용)
}

clf = TabNetClassifier(**tabnet_params)

print("🚀 TabNet 고위험군 예측 모델 학습 시작 (num_workers=0 최적화)")

clf.fit(
    X_train = X_train, y_train = y_train,
    eval_set = [(X_train, y_train), (X_test, y_test)],
    eval_name = ['train', 'valid'],
    eval_metric = ['auc'],
    max_epochs = 150,
    patience = 30,
    batch_size = 4096,
    virtual_batch_size = 512,
    num_workers = 0,  # Windows 데드락 원인 차단 및 인메모리 직결
    drop_last = False
)

preds_proba = clf.predict_proba(X_test)[ : , 1]
preds_class = clf.predict(X_test)

auc_score : float = roc_auc_score(y_test, preds_proba)
print(f"\n✅ TabNet 최적화 완료 | Test AUC : {round(auc_score, 4)}")
print("\n📊 Classification Report")
print(classification_report(y_test, preds_class))

feat_importances : np.ndarray = clf.feature_importances_
importance_df = pd.DataFrame({
    'Feature' : valid_features,
    'Importance' : feat_importances
}).sort_values(by = 'Importance', ascending = False)

print("\n🔍 Top 10 주요 임상 지표 (High-Risk 판별 기여도)")
print("-" * 55)
print(importance_df.head(10).to_string(index = False))

# 09. Clinical Evaluation Dashboard (XAI & Metrics)

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.metrics import roc_curve, confusion_matrix, roc_auc_score

def plot_clinical_dashboard_v6(y_true : np.ndarray, y_prob : np.ndarray, y_pred : np.ndarray, imp_df : pd.DataFrame) :
    auc_val : float = roc_auc_score(y_true, y_prob)
    fig, axes = plt.subplots(1, 3, figsize = ( 22, 6 ))
    
    # 1. ROC Curve Analysis
    fpr, tpr, _ = roc_curve(y_true, y_prob)
    axes[ 0 ].plot(fpr, tpr, color = "darkred", lw = 2.5, label = f"TabNet (AUC = {auc_val:.4f})")
    axes[ 0 ].plot([ 0, 1 ], [ 0, 1 ], color = "gray", linestyle = "--", alpha = 0.5)
    axes[ 0 ].fill_between(fpr, tpr, alpha = 0.1, color = "red")
    axes[ 0 ].set_title("ROC Curve Analysis", fontsize = 15, fontweight = "bold")
    axes[ 0 ].set_xlabel("False Positive Rate")
    axes[ 0 ].set_ylabel("True Positive Rate")
    axes[ 0 ].legend(loc = "lower right")
    axes[ 0 ].grid(True, alpha = 0.2)
    
    # 2. Confusion Matrix
    cm : np.ndarray = confusion_matrix(y_true, y_pred)
    cm_ratio : np.ndarray = cm.astype("float") / cm.sum(axis = 1)[ :, np.newaxis ]
    labels : np.ndarray = np.array([ [ f"{v}\n({r:.1%})" for v, r in zip(row_v, row_r) ] for row_v, row_r in zip(cm, cm_ratio) ])
    
    sns.heatmap(cm, annot = labels, fmt = "", cmap = "Blues", ax = axes[ 1 ], cbar = False, 
                annot_kws = { "size" : 14, "fontweight" : "bold" })
    axes[ 1 ].set_title("Confusion Matrix (Count & %)", fontsize = 15, fontweight = "bold")
    axes[ 1 ].set_xticklabels([ "Safe", "High-Risk" ])
    axes[ 1 ].set_yticklabels([ "Safe", "High-Risk" ])
    
    # 3. Feature Importance
    top_10 : pd.DataFrame = imp_df.head(10).copy()
    sns.barplot(x = "Importance", y = "Feature", data = top_10, ax = axes[ 2 ], palette = "viridis", hue = "Feature", legend = False)
    axes[ 2 ].set_title("Top 10 Clinical Biomarkers", fontsize = 15, fontweight = "bold")
    
    for i, v in enumerate(top_10[ "Importance" ]) :
        axes[ 2 ].text(v + 0.002, i, f"{v:.4f}", va = "center", fontsize = 11, fontweight = "bold")
    
    axes[ 2 ].set_xlim(0, top_10[ "Importance" ].max() * 1.15)
    axes[ 2 ].grid(axis = "x", alpha = 0.3)
    
    plt.tight_layout()
    
    target_dir : str = r"C:\Users\sck32\Desktop\Clinical Diabetes TabNet"
    try :
        os.makedirs(target_dir, exist_ok = True)
        os.chdir(target_dir)
        
        save_path : str = "clinical_dashboard.png"
        fig.savefig(save_path, dpi = 300, bbox_inches = "tight")
        print(f"✅ Dashboard saved strictly to : {os.path.abspath(save_path)}")
    except Exception as e :
        print(f"❌ Cannot save image. OS blocked file writing. (Error : {e})")
            
    plt.show()

# 실행
plot_clinical_dashboard_v6(y_test, preds_proba, preds_class, importance_df)

# 10. Pipeline Checkpoint Export (Persistence Fixed)

In [ ]:
import os
import torch
import torch.nn as nn
import joblib
from pathlib import Path

# 1. 모델 클래스 재선언
class PhenotypeAE(nn.Module) :
    def __init__(self, input_dim : int, latent_dim : int = 8) :
        super(PhenotypeAE, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 16),
            nn.BatchNorm1d(16),
            nn.GELU(),
            nn.Linear(16, latent_dim)
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 16),
            nn.BatchNorm1d(16),
            nn.GELU(),
            nn.Linear(16, input_dim)
        )

    def forward(self, x : torch.Tensor) :
        latent = self.encoder(x)
        reconstructed = self.decoder(latent)
        return latent, reconstructed

# 저장 경로 설정
base_dir : str = r"C:\Users\sck32\Desktop\Clinical Diabetes TabNet"
target_path : Path = Path(base_dir) / "metabolic_ai_models"

try :
    target_path.mkdir(parents = True, exist_ok = True)
    export_path : str = str(target_path)
    os.chdir(base_dir) # 커널 경로 강제 동기화
    print(f"📂 모델 체크포인트 수출 시작 : {export_path}")
except Exception as e :
    # 1순위 경로 실패 시 바탕화면으로 강제 우회
    fallback_path : Path = Path.home() / "Desktop" / "metabolic_ai_models"
    fallback_path.mkdir(parents = True, exist_ok = True)
    export_path : str = str(fallback_path)
    print(f"⚠️ 권한 또는 경로 오류로 인해 바탕화면으로 우회합니다 : {export_path}")

# 2. 파이프라인 에셋 저장
if 'model_ae' in locals() :
    torch.save(model_ae.state_dict(), f"{export_path}/ae_weights.pt")
    print(" [1/4] Autoencoder Weights 저장 완료")

if 'gmm_model' in locals() :
    gmm_assets : dict = {
        "mu" : gmm_model.mu.cpu(),
        "sigma" : gmm_model.sigma.cpu(),
        "pi" : gmm_model.pi.cpu()
    }
    torch.save(gmm_assets, f"{export_path}/gmm_metabolic_criteria.pt")
    print(" [2/4] GMM Parameters 저장 완료")

if 'clf' in locals() :
    clf.save_model(f"{export_path}/tabnet_high_risk_model")
    print(" [3/4] TabNet 모델 저장 완료")

if 'scaler_refined' in locals() :
    joblib.dump(scaler_refined, f"{export_path}/clinical_scaler.pkl")
    print(" [4/4] Data Scaler 저장 완료")

print("\n✅ 파이프라인 에셋 저장 프로세스 종료.")

# 11. Real-time Inference Pipeline

In [ ]:
import torch
import joblib
import pandas as pd
import numpy as np
import os
from pathlib import Path
from pytorch_tabnet.tab_model import TabNetClassifier

def diagnose_metabolic_risk(new_patient_data : pd.DataFrame) :
    # 1. 경로 우선순위 설정 (바탕화면 우선 탐색)
    desktop_path : Path = Path.home() / "Desktop" / "metabolic_ai_models"
    local_path : Path = Path(os.getcwd()) / "metabolic_ai_models"
    
    if (desktop_path / "clinical_scaler.pkl").exists() :
        model_dir : Path = desktop_path
    elif (local_path / "clinical_scaler.pkl").exists() :
        model_dir : Path = local_path
    else :
        raise FileNotFoundError(f"❌ 모델 에셋을 찾을 수 없습니다. (확인 경로 : {desktop_path})")

    # 2. 에셋 로드 및 모델 초기화
    scaler = joblib.load(model_dir / "clinical_scaler.pkl")
    
    # RTX 4070 12GB 환경을 고려한 디바이스 설정
    device : str = "cuda" if torch.cuda.is_available() else "cpu"
    
    clf = TabNetClassifier(device_name = device)
    clf.load_model(str(model_dir / "tabnet_high_risk_model.zip"))
    
    # 3. 데이터 전처리 파이프라인 (학습 단계와 완벽 매칭)
    X_raw = new_patient_data[ valid_features ].copy()
    X_raw = X_raw.fillna(0) 
    
    # 왜도 보정 및 음수 클리핑
    skewed_features : list = [ "HE_glu", "HE_TG", "HE_ast", "HE_alt", "HE_crea", "N_EN" ]
    for col in skewed_features :
        if col in X_raw.columns :
            X_raw[ col ] = np.log1p(np.clip(X_raw[ col ], a_min = 0, a_max = None))
    
    # 스케일링 및 FP32 최적화
    X_scaled : np.ndarray = scaler.transform(X_raw).astype(np.float32)
    
    # CUDA Assert 방지용 최종 검사
    if np.isnan(X_scaled).any() or np.isinf(X_scaled).any() :
        X_scaled = np.nan_to_num(X_scaled)
    
    # 4. 정밀 추론 수행
    risk_probs : np.ndarray = clf.predict_proba(X_scaled)[ :, 1 ]
    risk_preds : np.ndarray = clf.predict(X_scaled)
    
    # 5. 결과 리포트 생성
    results : pd.DataFrame = new_patient_data.copy()
    results[ "High_Risk_Probability" ] = risk_probs
    results[ "Diagnosis_Result" ] = [ "High-Risk" if p == 1 else "Safe" for p in risk_preds ]
    
    return results

# 테스트 실행
try :
    test_patients : pd.DataFrame = analysis_df.iloc[ :5 ] 
    diagnosis_report : pd.DataFrame = diagnose_metabolic_risk(test_patients)

    print(f"🔬 신규 환자 정밀 진단 결과 리포트 (Device : {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'})")
    print("-" * 80)
    print(diagnosis_report[ [ "ID", "age", "High_Risk_Probability", "Diagnosis_Result" ] ].to_string(index = False))
    print("-" * 80)
except Exception as e :
    print(f"❌ 진단 프로세스 오류 : {e}")

# 12. Full-scale Diagnosis & CSV Export

In [ ]:
import os

# 1. 전체 데이터 진단 수행
print(f"🚀 전체 데이터({len(analysis_df)}명) 정밀 진단 시작...")
full_diagnosis_results = diagnose_metabolic_risk(analysis_df)

# 2. CSV 파일 저장 (BOM 포함 UTF-8로 엑셀 호환성 확보)
output_filename : str = "final_metabolic_diagnosis_report.csv"
full_diagnosis_results.to_csv(output_filename, index = False, encoding = "utf-8-sig")

print(f"✅ 진단 보고서 저장 완료 : {os.path.abspath(output_filename)}")

# 고위험군 통계 요약
high_risk_count : int = full_diagnosis_results['Diagnosis_Result'].value_counts().get('High-Risk', 0)
print(f"📊 최종 식별된 고위험군 : {high_risk_count}명 ({high_risk_count/len(full_diagnosis_results)*100:.2f}%)")

# 13. Individual Patient Risk Analysis (Local XAI)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def explain_patient_risk(patient_id : str, model_dir : str = "./metabolic_ai_models") :
    # 1. 대상 환자 데이터 추출 및 전처리
    patient_raw = analysis_df[analysis_df['ID'] == patient_id].copy()
    if patient_raw.empty :
        print("❌ 해당 ID의 환자를 찾을 수 없습니다.")
        return

    scaler = joblib.load(f"{model_dir}/clinical_scaler.pkl")
    X_raw = patient_raw[valid_features].copy().fillna(0)
    
    skewed_features = ['HE_glu', 'HE_TG', 'HE_ast', 'HE_alt', 'HE_crea', 'N_EN']
    for col in skewed_features :
        X_raw[col] = np.log1p(np.clip(X_raw[col], a_min = 0, a_max = None))
    
    X_scaled = scaler.transform(X_raw).astype(np.float32)
    
    # 2. TabNet Local Explainability 추출
    explain_matrix, masks = clf.explain(X_scaled)
    
    # 3. 중요도 매핑 및 시각화
    plt.figure(figsize = (12, 6))
    importance_scores = explain_matrix[0]
    local_imp_df = pd.DataFrame({
        'Feature' : valid_features,
        'Contribution' : importance_scores
    }).sort_values(by = 'Contribution', ascending = False)
    
    sns.barplot(x = 'Contribution', y = 'Feature', data = local_imp_df.head(10), palette = "flare")
    
    # 수치 표기 추가
    for i, v in enumerate(local_imp_df.head(10)['Contribution']) :
        plt.text(v + 0.005, i, f'{v:.4f}', va = 'center', fontweight = 'bold')
        
    plt.title(f"Detailed Risk Factor Analysis | Patient ID : {patient_id}", fontsize = 15, fontweight = 'bold')
    plt.xlabel("Local Attention Weight (Model's Focus)")
    plt.grid(axis = 'x', alpha = 0.3)
    plt.tight_layout()
    plt.show()
    
    return local_imp_df.head(10)

# 실행 : 고위험군 환자 정밀 분석
top_factors = explain_patient_risk("A751220401")

# 14. Group Statistical Analysis (High-Risk vs. Safe)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind

# 한글 폰트 설정 (Windows : Malgun Gothic)
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

def perform_group_analysis_v2(df : pd.DataFrame, target_features : list) :
    # 1. 영문 코드 -> 한글 명칭 매핑 사전
    feature_map : dict = {
        'sm_presnt' : '현재 흡연 여부',
        'sex' : '성별',
        'HE_hepaB' : 'B형간염 양성',
        'N_CHO' : '탄수화물 섭취(g)',
        'HE_TG' : '중성지방(mg/dL)',
        'N_PROT' : '단백질 섭취(g)',
        'HE_wc' : '허리둘레(cm)',
        'HE_HDL_st2' : 'HDL 콜레스테롤',
        'HE_chol' : '총 콜레스테롤',
        'HE_ast' : 'AST(간수치)'
    }
    
    # 2. 통계 데이터 계산
    stats = df.groupby('Diagnosis_Result')[target_features].mean().T
    
    # 3. T-Test 수행
    p_values : list = []
    for feat in target_features :
        safe_g = df[df['Diagnosis_Result'] == 'Safe'][feat].dropna()
        risk_g = df[df['Diagnosis_Result'] == 'High-Risk'][feat].dropna()
        _, p_val = ttest_ind(safe_g, risk_g, equal_var = False)
        p_values.append(p_val)

    # 4. 시각화 데이터 구성
    melted_df = df.melt(id_vars = 'Diagnosis_Result', value_vars = target_features)
    melted_df['variable'] = melted_df['variable'].map(feature_map)
    
    # 5. 시각화 실행
    plt.figure(figsize = (16, 8))
    ax = sns.barplot(
        x = 'variable', y = 'value', hue = 'Diagnosis_Result', 
        data = melted_df, palette = 'coolwarm', edgecolor = 'black'
    )
    
    # 그래프 상단에 수치(Mean) 표시
    for p in ax.patches :
        height = p.get_height()
        if height > 0 :
            ax.text(
                p.get_x() + p.get_width() / 2., height + (height * 0.01),
                f'{height:.2f}', ha = "center", fontsize = 10, fontweight = 'bold'
            )
    
    plt.title("고위험군 vs 일반군 주요 임상 지표 비교 (집단 평균)", fontsize = 16, fontweight = 'bold', pad = 20)
    plt.xlabel("임상 지표 (Biomarkers)", fontsize = 12)
    plt.ylabel("평균 수치 (Mean Value)", fontsize = 12)
    plt.legend(title = "진단 결과", loc = 'upper right')
    plt.xticks(rotation = 0) # 한글 명칭이므로 가독성을 위해 회전 제거
    plt.grid(axis = 'y', alpha = 0.2)
    plt.tight_layout()
    plt.show()

    # 결과 테이블 생성
    summary_df = pd.DataFrame({
        '지표명' : [feature_map.get(f, f) for f in target_features],
        '일반군 평균' : stats['Safe'].values,
        '고위험군 평균' : stats['High-Risk'].values,
        'P-Value' : p_values
    })
    
    return summary_df

# 실행
top_10_features = [
    'sm_presnt', 'sex', 'HE_hepaB', 'N_CHO', 'HE_TG', 
    'N_PROT', 'HE_wc', 'HE_HDL_st2', 'HE_chol', 'HE_ast'
]
group_comparison_v2 = perform_group_analysis_v2(full_diagnosis_results, top_10_features)

print("📋 집단 간 통계적 유의성 분석 리포트")
print("-" * 80)
print(group_comparison_v2.to_string(index = False))

# 15. Multi Disease Comparative Analysis

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_auc_score
from scipy.stats import spearmanr

# 하드웨어 최적화 폰트 설정
plt.rcParams[ "font.family" ] = "Malgun Gothic"
plt.rcParams[ "axes.unicode_minus" ] = False

def run_final_disease_comparison_labeled(df : pd.DataFrame, target_map : dict) :
    # 1. AI 위험도 등급(Decile) 생성 (128GB RAM 최적화 연산)
    df = df.copy()
    df[ "Risk_Decile" ] = pd.qcut(df[ "High_Risk_Probability" ], 10, labels = False, duplicates = "drop")
    
    results = []
    
    # 2. 시각화 초기 설정
    plt.figure(figsize = (16, 9))
    
    # 3. 질환별 분석 및 시각화
    for col_name, label in target_map.items() :
        if col_name not in df.columns :
            continue
            
        temp_df = df[[ col_name, "High_Risk_Probability", "Risk_Decile" ]].dropna()
        y_true = (temp_df[ col_name ] >= (3 if "HE_" in col_name else 1)).astype(int)
        y_prob = temp_df[ "High_Risk_Probability" ].values
        
        # 통계 지표 산출
        auc = roc_auc_score(y_true, y_prob)
        corr, _ = spearmanr(y_true, y_prob)
        
        results.append({
            "질환명" : label,
            "AUC" : round(auc, 4),
            "Spearman_Corr" : round(corr, 4),
            "유병자수" : int(y_true.sum())
        })
        
        # 등급별 유병률 계산 및 시각화
        temp_stats = df.copy()
        temp_stats[ "is_target" ] = (temp_stats[ col_name ] >= (3 if "HE_" in col_name else 1)).astype(int)
        trend = temp_stats.groupby( "Risk_Decile" )[ "is_target" ].mean()
        
        # 라인 플롯
        line = plt.plot(trend.index, trend.values, marker = "o", label = f"{label} (AUC : {auc:.4f})", lw = 3, markersize = 8)
        color = line[ 0 ].get_color()
        
        # 수치 표시 (Data Labels)
        for x, y in zip(trend.index, trend.values) :
            plt.text(
                x, y + 0.015, f"{y:.3f}", 
                ha = "center", va = "bottom", 
                fontsize = 10, fontweight = "bold", color = color
            )

    # 4. 그래프 디테일 설정
    plt.title( "AI 위험 등급별 실제 질환 유병률 추이 (정밀 수치 포함)", fontsize = 18, fontweight = "bold", pad = 20 )
    plt.xlabel( "AI Risk Decile (0 : 저위험 -> 9 : 고위험)", fontsize = 12 )
    plt.ylabel( "실제 유병률 (Prevalence Rate)", fontsize = 12 )
    plt.xticks(range( 10 ))
    plt.ylim(0, 1.1) # 가독성을 위해 상단 여백 확보
    plt.legend(loc = "upper right", fontsize = 11, frameon = True, shadow = True)
    plt.grid(True, axis = "y", alpha = 0.3)
    plt.tight_layout()
    plt.show()
    
    return pd.DataFrame(results)

# 분석 실행
target_disease_map = {
    "diabetes" : "당뇨병 (학습 타겟)",
    "HE_HP" : "고혈압",
    "HE_chol" : "고콜레스테롤혈증"
}

summary_table = run_final_disease_comparison_labeled(full_diagnosis_results, target_disease_map)

print( "\n📊 다중 질환 상관성 정밀 분석 리포트" )
print( "-" * 85 )
print(summary_table.to_string(index = False))

# 16. Multi Disease Risk Weight Re-adjustment

In [ ]:
import numpy as np
import pandas as pd

def adjust_metabolic_weights(df : pd.DataFrame, stats_df : pd.DataFrame) :
    # 1. 질환별 가중치 추출 (상관계수의 절대값 기반)
    # Spearman_Corr가 NaN인 고콜레스테롤은 최소 가중치(0.01) 부여
    weights = {
        'diabetes' : 1.0, # 기준점
        'HE_HP' : abs(stats_df.loc[stats_df['질환명'] == '고혈압', 'Spearman_Corr'].values[0]),
        'HE_chol' : 0.05   # 포화 데이터에 대한 보정 가중치
    }
    
    # 가중치 정규화
    total_w = sum(weights.values())
    norm_weights = { k : v / total_w for k, v in weights.items() }
    
    # 2. 질환별 개별 위험도 산출
    # 고혈압은 역상관 관계이므로 (1 - Prob)를 활용해 대사적 불일치 보정
    p_dm = df['High_Risk_Probability'].values
    p_hp = 1 - p_dm # 역상관 관계 반영 (혈압 특이적 위험도 시뮬레이션)
    p_chol = np.full_like(p_dm, 1.0) # 전원 유병 상태 반영
    
    # 3. 최종 통합 대사 위험도 계산 (FP16 최적화 연산)
    df['Adjusted_Total_Risk'] = (
        (p_dm * norm_weights['diabetes']) +
        (p_hp * norm_weights['HE_HP']) +
        (p_chol * norm_weights['HE_chol'])
    ).astype(np.float16)
    
    # 4. 재조정 결과에 따른 위험군 재분류 (상위 15% 기준)
    threshold = np.percentile(df['Adjusted_Total_Risk'], 85)
    df['Final_Metabolic_Grade'] = df['Adjusted_Total_Risk'].apply(
        lambda x : 'Critical' if x >= threshold else ('Warning' if x >= 0.5 else 'Normal')
    )
    
    return df, norm_weights

# 가중치 재조정 실행
full_diagnosis_results, final_weights = adjust_metabolic_weights(full_diagnosis_results, summary_table)

print("⚖️ 질환별 최종 반영 가중치 (Normalized)")
for k, v in final_weights.items() :
    print(f" - {k} : {v:.4f}")

print("\n📊 가중치 재조정 후 위험 등급 분포")
print(full_diagnosis_results['Final_Metabolic_Grade'].value_counts())

# 17. Metabolic Risk Distribution

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# 폰트 및 스타일 설정
plt.rcParams["font.family"] = "Malgun Gothic"
plt.rcParams["axes.unicode_minus"] = False

plt.figure(figsize = (14, 7))

# 1. KDE Plot 생성 (원본 vs 재조정)
sns.kdeplot(full_diagnosis_results['High_Risk_Probability'], 
            label = 'Raw Diabetes Prob', fill = True, alpha = 0.3, color = '#1f77b4')
sns.kdeplot(full_diagnosis_results['Adjusted_Total_Risk'], 
            label = 'Adjusted Total Risk', fill = True, alpha = 0.4, color = '#ff7f0e')

# 2. 통계치 계산
mean_raw = full_diagnosis_results['High_Risk_Probability'].mean()
mean_adj = full_diagnosis_results['Adjusted_Total_Risk'].mean()
# Critical 등급의 최소 점수를 임계치로 설정
threshold = full_diagnosis_results.loc[full_diagnosis_results['Final_Metabolic_Grade'] == 'Critical', 
                                     'Adjusted_Total_Risk'].min()

# 3. 평균선(Mean) 및 수치 표시
plt.axvline(mean_raw, color = '#1f77b4', linestyle = '--', lw = 2)
plt.text(mean_raw + 0.02, 7, f'Raw Mean: {mean_raw:.3f}', 
         color = '#1f77b4', fontweight = 'bold', fontsize = 11)

plt.axvline(mean_adj, color = '#ff7f0e', linestyle = '--', lw = 2)
plt.text(mean_adj + 0.02, 9, f'Adj Mean: {mean_adj:.3f}', 
         color = '#ff7f0e', fontweight = 'bold', fontsize = 11)

# 4. 임계치(Critical Threshold) 및 위험 구역 표시
plt.axvline(threshold, color = 'red', linestyle = ':', lw = 2.5)
plt.fill_betweenx(y = [0, 11], x1 = threshold, x2 = 1.1, color = 'red', alpha = 0.05)
plt.text(threshold + 0.01, 4, f'Critical Threshold: {threshold:.3f}', 
         color = 'red', fontweight = 'bold', rotation = 90, va = 'center')

# 5. 그래프 세부 설정
plt.title("Metabolic Risk Distribution Analysis | Raw vs. Adjusted", fontsize = 16, fontweight = 'bold', pad = 20)
plt.xlabel("Risk Score (0.0 ~ 1.0)", fontsize = 12)
plt.ylabel("Density (Population Frequency)", fontsize = 12)
plt.xlim(-0.1, 1.1)
plt.ylim(0, 11)
plt.legend(loc = 'upper right', frameon = True, shadow = True)
plt.grid(axis = 'y', alpha = 0.2)
plt.tight_layout()
plt.show()

# 18. Model Integrity & Reliability Verification

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.metrics import brier_score_loss
from sklearn.calibration import calibration_curve

# 1. 폰트 충돌 방지를 위한 설정 순서 최적화
sns.set_style( "whitegrid" )

# Windows 시스템 기본 폰트 강제 재할당
plt.rcParams[ "font.family" ] = "Malgun Gothic"
plt.rcParams[ "axes.unicode_minus" ] = False

# 2. 임상 지표 한글 매핑 딕셔너리
clinical_ko_map : dict = {
    "sm_presnt" : "현재 흡연 여부", "sex" : "성별", "HE_hepaB" : "B형간염 양성",
    "N_CHO" : "탄수화물 섭취", "HE_TG" : "중성지방", "N_PROT" : "단백질 섭취",
    "HE_wc" : "허리둘레", "HE_HDL_st2" : "HDL 콜레스테롤", "HE_chol" : "총 콜레스테롤",
    "HE_ast" : "AST (간수치)", "HE_HbA1c" : "당화혈색소", "HE_glu" : "공복혈당",
    "HE_BMI" : "체질량지수 (BMI)"
}

# --- [A] 최정예 모델 중요도 (Top 10 Selection) ---
def plot_importance_solid(df : pd.DataFrame) :
    plt.figure(figsize = (12, 8))
    
    df[ "Feature_Ko" ] = df[ "Feature" ].map(lambda x : clinical_ko_map.get(x, x))
    
    # 선명한 단색 팔레트 적용 (bright)
    ax = sns.barplot(
        x = "Importance", 
        y = "Feature_Ko", 
        data = df, 
        palette = "bright", 
        hue = "Feature_Ko", 
        legend = False
    )
    
    for p in ax.patches :
        w : float = p.get_width()
        if w > 0 :
            ax.annotate(f"{w:.4f}", (w + 0.005, p.get_y() + p.get_height() / 2),
                        ha = "left", va = "center", fontsize = 10, fontweight = "bold")
    
    plt.title( "최정예 모델 중요도 분석 (Top 10 Selection)", fontsize = 18, fontweight = "bold", pad = 25 )
    plt.xlabel( "기여도 (Importance Score)", fontsize = 12 )
    plt.ylabel( "핵심 임상 지표", fontsize = 12 )
    plt.xlim(0, df[ "Importance" ].max() * 1.15)
    plt.tight_layout()
    plt.show()

# --- [B] Data Leakage 상관계수 점검 ---
def plot_leakage_solid(data : dict) :
    plt.figure(figsize = (10, 6))
    leakage_df : pd.DataFrame = pd.DataFrame(list(data.items()), columns = [ "Feature", "Correlation" ])
    leakage_df[ "Feature_Ko" ] = leakage_df[ "Feature" ].map(lambda x : clinical_ko_map.get(x, x))
    
    # 구분이 명확한 단색 팔레트 적용 (Set1)
    ax = sns.barplot(
        x = "Correlation", 
        y = "Feature_Ko", 
        data = leakage_df, 
        palette = "Set1", 
        hue = "Feature_Ko", 
        legend = False
    )
    
    for p in ax.patches :
        w : float = p.get_width()
        ax.annotate(f"r = {w:.4f}", (w + 0.002, p.get_y() + p.get_height() / 2),
                    ha = "left", va = "center", fontsize = 10, fontweight = "bold")

    plt.axvline(x = 0.9, color = "#e74c3c", linestyle = "--", lw = 2)
    plt.title( "데이터 누수 점검 : 진단 지표 상관계수", fontsize = 16, fontweight = "bold", pad = 25 )
    plt.xlabel( "피어슨 상관계수 (r)", fontsize = 12 )
    plt.ylabel( "진단 기준 지표", fontsize = 12 )
    plt.xlim(0, 1.0)
    plt.tight_layout()
    plt.show()

# 데이터 로드 및 실행
top_10_results : pd.DataFrame = pd.DataFrame({
    "Feature" : [ "sm_presnt", "sex", "HE_hepaB", "N_CHO", "HE_TG", "N_PROT", "HE_wc", "HE_HDL_st2", "HE_chol", "HE_ast" ],
    "Importance" : [ 0.3385, 0.0842, 0.0722, 0.0685, 0.0622, 0.0521, 0.0383, 0.0345, 0.0301, 0.0281 ]
})
leakage_results : dict = { "HE_HbA1c" : 0.0401, "HE_glu" : 0.0506, "HE_BMI" : 0.0669, "HE_wc" : 0.1346 }

plot_importance_solid(top_10_results)
plot_leakage_solid(leakage_results)

# 19. SHAP Value (의사결정 방향성)

In [ ]:
import shap
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

# 하드웨어 및 폰트 환경 설정
device : str = "cuda" if torch.cuda.is_available() else "cpu"
plt.rcParams[ "font.family" ] = "Malgun Gothic"
plt.rcParams[ "axes.unicode_minus" ] = False

# TabNet 출력을 단일 텐서로 변환하는 래퍼
class TabNetWrapper(nn.Module) :
    def __init__(self, model) :
        super(TabNetWrapper, self).__init__()
        self.model = model
    def forward(self, x) :
        return self.model(x)[ 0 ]

# 1. SHAP Interaction Values 산출 (GPU 가속)
background_data : torch.Tensor = torch.as_tensor(shap.kmeans(X_train, 50).data).float().to(device)
wrapped_model : nn.Module = TabNetWrapper(clf.network).to(device).eval()

# KernelExplainer를 통한 Interaction Approximation
explainer : shap.KernelExplainer = shap.KernelExplainer(lambda x : wrapped_model(torch.as_tensor(x).float().to(device)).cpu().detach().numpy(), background_data.cpu().numpy())
shap_interaction_values : np.ndarray = explainer.shap_values(X_test[ :100 ], check_additivity = False)

# 2. 수치 라벨링 Interaction Plot 함수
def plot_labeled_interaction(interaction_vals : np.ndarray, data : np.ndarray, feature_idx : list) :
    fig = plt.figure(figsize = (10, 12))
    
    # Interaction Summary Plot 출력
    shap.summary_plot(
        interaction_vals[ :, feature_idx, feature_idx ], 
        data[ :100, feature_idx ],
        feature_names = [ clinical_ko_map.get(valid_features[ i ], valid_features[ i ]) for i in feature_idx ],
        show = False
    )

    ax = plt.gca()
    # 이상치(Extreme Points) 탐색 및 수치 강제 표기
    for i, idx in enumerate(feature_idx) :
        vals = interaction_vals[ :, idx, idx ]
        extreme_points = np.where(np.abs(vals) > 0.5)[ 0 ]
        
        for e_idx in extreme_points :
            v = vals[ e_idx ]
            ax.text(
                v, i, f" {v:.2f}", 
                va = "center", ha = "left" if v > 0 else "right",
                fontsize = 9, fontweight = "bold", color = "crimson"
            )

    plt.title( "임상 지표별 당뇨 위험 상호작용 분석 (Labeled)", fontsize = 16, fontweight = "bold", pad = 20 )
    plt.xlabel( "SHAP interaction value", fontsize = 12 )
    plt.tight_layout()
    plt.show()

# 3. 분석 실행 (B형간염 및 HE_hepaC 인덱스 기준)
target_indices : list = [ valid_features.index( "HE_hepaB" ), valid_features.index( "HE_hepaC" ) ]
plot_labeled_interaction(shap_interaction_values, X_test, target_indices)

# 20. Advanced Clinical Validation : Interpretability, Utility, and Robustness Analysis

In [ ]:
import shap
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.stats.outliers_influence import variance_inflation_factor

# 하드웨어 가속 및 폰트 전역 설정
device : str = "cuda" if torch.cuda.is_available() else "cpu"
plt.rcParams[ "font.family" ] = "Malgun Gothic"
plt.rcParams[ "axes.unicode_minus" ] = False

# 1. SHAP Interaction (GPU 가속 래핑)
class TabNetWrapper(nn.Module) :
    def __init__(self, model) :
        super(TabNetWrapper, self).__init__()
        self.model = model
    def forward(self, x) :
        return self.model(x)[ 0 ]

wrapped_model : nn.Module = TabNetWrapper(clf.network).to(device).eval()
background : np.ndarray = shap.kmeans(X_train, 50).data
explainer : shap.KernelExplainer = shap.KernelExplainer(
    lambda x : wrapped_model(torch.as_tensor(x).float().to(device)).cpu().detach().numpy(), 
    background
)
shap_interaction_values : np.ndarray = explainer.shap_values(X_test[ :100 ], check_additivity = False)

# 2. Labeled Interaction Plot
def plot_labeled_interaction(interaction_vals, data, feature_idx) :
    plt.figure(figsize = (10, 8))
    shap.summary_plot(
        interaction_vals[ :, feature_idx, feature_idx ], 
        data[ :100, feature_idx ],
        feature_names = [ clinical_ko_map.get(valid_features[ i ], valid_features[ i ]) for i in feature_idx ],
        show = False
    )
    ax = plt.gca()
    for i, idx in enumerate(feature_idx) :
        vals = interaction_vals[ :, idx, idx ]
        extreme = np.where(np.abs(vals) > 0.5)[ 0 ]
        for e in extreme :
            v = vals[ e ]
            ax.text(v, i, f" {v:.2f}", va = "center", ha = "left" if v > 0 else "right", 
                    fontsize = 9, fontweight = "bold", color = "crimson")
    plt.title( "임상 지표별 당뇨 위험 상호작용 분석 (Labeled)", fontsize = 16, fontweight = "bold", pad = 20 )
    plt.show()

# 3. Decision Curve Analysis (DCA) with Labeling
def plot_dca_pytorch(df) :
    target_candidates : list = [ "HE_DM", "is_high_risk", "target", "label" ]
    target_col : str = next((c for c in target_candidates if c in df.columns), df.columns[ -1 ])
    
    y_raw = pd.to_numeric(df[ target_col ], errors = "coerce" ).fillna(0)
    y_true : torch.Tensor = torch.as_tensor((y_raw >= (3 if "HE_" in target_col else 1)).astype(int).values).float().to(device)
    y_prob : torch.Tensor = torch.as_tensor(df[ "High_Risk_Probability" ].values).float().to(device)
    
    thresholds : torch.Tensor = torch.linspace(0, 0.99, 100).to(device)
    nb_model, nb_all = [], []
    n : int = len(y_true)
    prev : float = torch.mean(y_true).item()

    for p_t in thresholds :
        tp : float = torch.sum((y_prob >= p_t) & (y_true == 1)).item()
        fp : float = torch.sum((y_prob >= p_t) & (y_true == 0)).item()
        nb : float = (tp / n) - (fp / n) * (p_t.item() / (1 - p_t.item()))
        nb_model.append( nb )
        nb_all.append( prev - (1 - prev) * (p_t.item() / (1 - p_t.item())) )

    plt.figure(figsize = (11, 8))
    ax = plt.gca()
    
    # 곡선 렌더링
    ax.plot(thresholds.cpu(), nb_model, color = "#0047AB", lw = 3, label = "TabNet 모델")
    ax.plot(thresholds.cpu(), nb_all, color = "#A9A9A9", ls = "--", label = "전원 치료 전략")
    ax.axhline(y = 0, color = "black", lw = 1.2, label = "치료 없음 전략")

    # 최대 순이익(Max Net Benefit) 지점 자동 탐색 및 라벨링
    max_nb = max(nb_model)
    max_idx = nb_model.index(max_nb)
    max_t = thresholds[ max_idx ].item()
    
    # 최대 이득 지점에 포인트 표시 및 주석 추가
    ax.plot(max_t, max_nb, 'o', color = "crimson", markersize = 8)
    ax.annotate(f"최대 순이익 : {max_nb:.4f}\n(임계치 : {max_t:.2f})",
                xy = (max_t, max_nb), xytext = (max_t + 0.15, max_nb + 0.02),
                arrowprops = dict(arrowstyle = "->", connectionstyle = "arc3", color = "crimson"),
                fontsize = 10, fontweight = 'bold', color = "crimson")

    # 주요 임계치(0.1, 0.2, 0.5) 수치 라벨 추가
    for t_val in [ 0.1, 0.2, 0.5 ] :
        idx = (np.abs(thresholds.cpu().numpy() - t_val)).argmin()
        val = nb_model[ idx ]
        if val > 0 :
            ax.text(t_val, val + 0.005, f"{val:.4f}", ha = 'center', va = 'bottom', 
                    fontsize = 9, fontweight = 'bold', color = "#0047AB")

    plt.title( "임상적 의사결정 곡선 분석 (DCA)", fontsize = 18, fontweight = "bold", pad = 25 )
    plt.xlabel( "위험 임계치 (Threshold)", fontsize = 12 )
    plt.ylabel( "순이익 (Net Benefit)", fontsize = 12 )
    plt.ylim(-0.05, max(nb_model) * 1.5)
    plt.legend(loc = "upper right")
    plt.grid(alpha = 0.2)
    plt.tight_layout()
    plt.show()

# 4. VIF Analysis (Independence)
def run_vif_check() :
    if isinstance(top_10_features, pd.DataFrame) :
        vif_features : list = top_10_features[ "Feature" ].tolist()
    elif isinstance(top_10_features, list) :
        if len(top_10_features) > 0 and isinstance(top_10_features[ 0 ], dict) :
            vif_features : list = [ f[ "Feature" ] for f in top_10_features ]
        else :
            vif_features : list = top_10_features
    else :
        vif_features : list = list(top_10_features)

    X_vif_df : pd.DataFrame = pd.DataFrame(X_test, columns = valid_features)[ vif_features ]
    X_vif_df = X_vif_df.apply(pd.to_numeric, errors = "coerce" ).fillna(0)
    
    vif_report : pd.DataFrame = pd.DataFrame()
    vif_report[ "임상 지표" ] = [ clinical_ko_map.get(f, f) for f in X_vif_df.columns ]
    vif_report[ "VIF" ] = [ variance_inflation_factor(X_vif_df.values, i) for i in range(X_vif_df.shape[ 1 ]) ]
    
    print( "\n⚖️ 지표 간 독립성 검증 (VIF)" )
    print( "-" * 50 )
    print( vif_report.sort_values(by = "VIF", ascending = False).to_string(index = False) )

# 분석 실행 파이프라인
target_idx : list = [ valid_features.index( "HE_hepaB" ), valid_features.index( "HE_hepaC" ) ]
plot_labeled_interaction(shap_interaction_values, X_test, target_idx)
plot_dca_pytorch(full_diagnosis_results)
run_vif_check()

# 21. Advanced Clinical Validation

In [ ]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as st
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix
from sklearn.linear_model import LogisticRegression
from joblib import Parallel, delayed

# 이전 파이프라인의 변수들을 현재 검증 스크립트 변수명으로 맵핑
y_true : np.ndarray = y_test
y_prob_tabnet : np.ndarray = preds_proba

device : str = "cuda" if torch.cuda.is_available() else "cpu"
plt.rcParams[ "font.family" ] = "Malgun Gothic"
plt.rcParams[ "axes.unicode_minus" ] = False

# 1. DeLong's Test
def compute_midrank(x) :
    J = np.argsort(x)
    Z = x[ J ]
    N = len(x)
    T = np.zeros(N, dtype = float)
    i = 0
    while i < N :
        j = i
        while j < N and Z[ j ] == Z[ i ] :
            j += 1
        T[ i : j ] = 0.5 * (i + j - 1)
        i = j
    T2 = np.empty(N, dtype = float)
    T2[ J ] = T + 1
    return T2

def delong_roc_test(y_true : np.ndarray, y_prob1 : np.ndarray, y_prob2 : np.ndarray) :
    pos = y_prob1[ y_true == 1 ]
    neg = y_prob1[ y_true == 0 ]
    m, n = len(pos), len(neg)
    
    tx1, ty1 = compute_midrank(pos), compute_midrank(neg)
    tz1 = compute_midrank(np.concatenate((pos, neg)))
    vx1 = (tz1[ :m ] - tx1) / n
    vy1 = 1.0 - (tz1[ m: ] - ty1) / m
    
    pos2 = y_prob2[ y_true == 1 ]
    neg2 = y_prob2[ y_true == 0 ]
    tx2, ty2 = compute_midrank(pos2), compute_midrank(neg2)
    tz2 = compute_midrank(np.concatenate((pos2, neg2)))
    vx2 = (tz2[ :m ] - tx2) / n
    vy2 = 1.0 - (tz2[ m: ] - ty2) / m

    S11 = np.cov(vx1, vx2, rowvar = False)[ 0, 1 ] * m + np.cov(vy1, vy2, rowvar = False)[ 0, 1 ] * n
    S11 /= (m + n)
    
    auc1, auc2 = roc_auc_score(y_true, y_prob1), roc_auc_score(y_true, y_prob2)
    z = (auc1 - auc2) / np.sqrt(max(S11, 1e-8))
    p_value = 2 * (1 - st.norm.cdf(abs(z)))
    return auc1, auc2, p_value

lr_model = LogisticRegression(max_iter = 1000, n_jobs = -1)
lr_model.fit(X_train, y_train)
y_prob_lr : np.ndarray = lr_model.predict_proba(X_test)[ :, 1 ]

auc_tabnet, auc_lr, p_val_delong = delong_roc_test(y_true, y_prob_tabnet, y_prob_lr)
print( f"\n[1] DeLong's Test : TabNet ({auc_tabnet:.4f}) vs LR ({auc_lr:.4f}) | p-value : {p_val_delong:.4e}" )

# 2. Youden's Index
fpr, tpr, thresholds_roc = roc_curve(y_true, y_prob_tabnet)
youden_idx = np.argmax(tpr - fpr)
optimal_threshold : float = thresholds_roc[ youden_idx ]

y_pred_opt : np.ndarray = (y_prob_tabnet >= optimal_threshold).astype(int)
tn, fp, fn, tp = confusion_matrix(y_true, y_pred_opt).ravel()

sensitivity : float = tp / (tp + fn)
specificity : float = tn / (tn + fp)
ppv : float = tp / (tp + fp) if (tp + fp) > 0 else 0.0
npv : float = tn / (tn + fn) if (tn + fn) > 0 else 0.0

print( f"\n[2] Youden's Index Optimal Threshold : {optimal_threshold:.4f}" )
print( f"    - Sensitivity : {sensitivity:.4f} | Specificity : {specificity:.4f}" )
print( f"    - PPV         : {ppv:.4f} | NPV         : {npv:.4f}" )

# 3. Bootstrapping
n_bootstraps : int = 1000
rng = np.random.RandomState(42)

def bootstrap_iter(i) :
    indices = rng.randint(0, len(y_prob_tabnet), len(y_prob_tabnet))
    if len(np.unique(y_true[ indices ])) < 2 :
        return None
    return roc_auc_score(y_true[ indices ], y_prob_tabnet[ indices ])

bootstrapped_scores = Parallel(n_jobs = -1)(delayed(bootstrap_iter)(i) for i in range(n_bootstraps))
bootstrapped_scores = [ s for s in bootstrapped_scores if s is not None ]

sorted_scores = np.array(bootstrapped_scores)
sorted_scores.sort()
ci_lower : float = sorted_scores[ int(0.025 * len(sorted_scores)) ]
ci_upper : float = sorted_scores[ int(0.975 * len(sorted_scores)) ]

print( f"\n[3] Bootstrapping (1000 iter) 95% CI : {ci_lower:.4f} - {ci_upper:.4f}" )

# 4. Subgroup Analysis
def get_mask(feature : str, val : float) :
    if feature in valid_features :
        idx = valid_features.index(feature)
        return X_test[ :, idx ] == val
    return np.zeros(len(y_true), dtype = bool)

subgroups : dict = {
    "All Patients" : (np.ones(len(y_true), dtype = bool), "Total"),
    "Male" : (get_mask( "sex", 1 ), "Gender"),
    "Female" : (get_mask( "sex", 2 ), "Gender"),
    "HepaB (+)" : (get_mask( "HE_hepaB", 1 ), "Liver"),
    "HepaB (-)" : (get_mask( "HE_hepaB", 0 ), "Liver"),
    "Smoker" : (get_mask( "sm_presnt", 1 ), "Lifestyle"),
    "Non-Smoker" : (get_mask( "sm_presnt", 0 ), "Lifestyle"),
}

results_sub = []
for name, (mask, category) in subgroups.items() :
    if np.sum(mask) > 10 and len(np.unique(y_true[ mask ])) > 1 :
        score = roc_auc_score(y_true[ mask ], y_prob_tabnet[ mask ])
        results_sub.append((name, score, np.sum(mask)))

plt.figure(figsize = ( 10, 6 ))
names = [ r[ 0 ] for r in results_sub ][ ::-1 ]
aucs = [ r[ 1 ] for r in results_sub ][ ::-1 ]
counts = [ r[ 2 ] for r in results_sub ][ ::-1 ]

plt.scatter(aucs, range(len(names)), color = "#0047AB", s = 100, zorder = 3)
plt.axvline(x = auc_tabnet, color = "crimson", linestyle = "--", label = "Overall AUC")

for i, (auc_val, count) in enumerate(zip(aucs, counts)) :
    plt.text(auc_val + 0.01, i, f"{auc_val:.3f} (N={count})", va = "center", fontsize = 10)

plt.yticks(range(len(names)), names, fontsize = 12)
plt.xlabel( "AUROC Score", fontsize = 12 )
plt.title( "Subgroup AUROC Analysis", fontsize = 16, fontweight = "bold", pad = 20 )
plt.legend()
plt.grid(axis = "x", linestyle = "--", alpha = 0.5)
plt.tight_layout()
plt.show()

# 22. Comprehensive Model Integrity Analysis : Local Interpretability, Calibration, and Feature Robustness

In [ ]:
import shap
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.text as mtext
from matplotlib import font_manager, rc
from sklearn.metrics import brier_score_loss, roc_auc_score
from sklearn.calibration import calibration_curve

# 1. 폰트 및 유니코드 마이너스 설정
font_path : str = "C:/Windows/Fonts/malgun.ttf"
font_name : str = font_manager.FontProperties(fname = font_path).get_name()
rc('font', family = font_name)
plt.rcParams[ "axes.unicode_minus" ] = False 

device : str = "cuda" if torch.cuda.is_available() else "cpu"

# Y축 레이블 강제 맵핑 및 전역 텍스트 치환
def force_fix_plot_tofu(fig, ax) :
    fig.canvas.draw()
    
    # 1. Y축 레이블 좌표 고정 및 명시적 치환
    ticks : list = ax.get_yticks()
    labels : list = [ item.get_text() for item in ax.get_yticklabels() ]
    new_labels : list = [ label.replace('\u2212', '-').replace('−', '-').replace('ㅁ', '-') for label in labels ]
    
    ax.set_yticks(ticks)
    ax.set_yticklabels(new_labels)
    
    # 2. 내부 텍스트(막대 수치 등) 치환
    for text_obj in fig.findobj(mtext.Text) :
        t : str = text_obj.get_text()
        if t :
            cleaned : str = t.replace('\u2212', '-').replace('−', '-').replace('ㅁ', '-')
            text_obj.set_text(cleaned)

# 1. Local XAI Case Study (Waterfall Plot)
y_true_arr : np.ndarray = y_test.values if isinstance(y_test, pd.Series) else np.array(y_test)
y_prob : np.ndarray = clf.predict_proba(X_test)[ :, 1 ]
y_pred : np.ndarray = (y_prob >= 0.3965).astype(int)

tp_idx : int = np.where((y_true_arr == 1) & (y_pred == 1))[ 0 ][ 0 ]
fn_idx : int = np.where((y_true_arr == 1) & (y_pred == 0))[ 0 ][ 0 ]

explainer_local = shap.KernelExplainer(lambda x : clf.predict_proba(x)[ :, 1 ], shap.kmeans(X_train, 50))
shap_vals_local : np.ndarray = explainer_local.shap_values(X_test[ [ tp_idx, fn_idx ] ])

def plot_local_analysis(shap_values : np.ndarray, base_value : float, data : np.ndarray, title : str) :
    explanation = shap.Explanation(
        values = shap_values, 
        base_values = base_value, 
        data = data, 
        feature_names = [ clinical_ko_map.get(f, f) for f in valid_features ]
    )
    fig, ax = plt.subplots(figsize = (10, 8))
    shap.plots.waterfall(explanation, max_display = 10, show = False)
    
    plt.title(title, fontsize = 16, fontweight = "bold", pad = 25)
    plt.tight_layout()
    
    force_fix_plot_tofu(fig, ax)
    plt.show()

plot_local_analysis(shap_vals_local[ 0 ], explainer_local.expected_value, X_test[ tp_idx ], "True Positive Case Analysis")
plot_local_analysis(shap_vals_local[ 1 ], explainer_local.expected_value, X_test[ fn_idx ], "False Negative Case Analysis")

# 2. Probability Calibration with Point Annotations
prob_true, prob_pred = calibration_curve(y_true_arr, y_prob, n_bins = 10, strategy = "quantile")
brier : float = brier_score_loss(y_true_arr, y_prob)

fig_cal, ax_cal = plt.subplots(figsize = (9, 9))
ax_cal.plot(prob_pred, prob_true, marker = "o", lw = 3, color = "#0047AB", label = f"TabNet (Brier : {brier:.4f})")
ax_cal.plot([ 0, 1 ], [ 0, 1 ], ls = "--", color = "black", alpha = 0.7, label = "Ideal")

for x, y in zip(prob_pred, prob_true) :
    ax_cal.text(x, y + 0.02, f"P:{x:.3f}\nO:{y:.3f}", ha = "center", va = "bottom", 
                fontsize = 9, fontweight = "bold", color = "#0047AB",
                bbox = dict(boxstyle = "round,pad=0.3", fc = "white", ec = "#0047AB", alpha = 0.8))

ax_cal.set_title( "예측 확률 교정 곡선 (Calibration Analysis)", fontsize = 18, fontweight = "bold", pad = 25 )
ax_cal.set_xlabel( "평균 예측 확률 (Mean Predicted Probability)", fontsize = 12 )
ax_cal.set_ylabel( "실제 관측 비율 (Fraction of Positives)", fontsize = 12 )
ax_cal.legend(loc = "lower right")
ax_cal.grid(alpha = 0.2)
plt.tight_layout()

force_fix_plot_tofu(fig_cal, ax_cal)
plt.show()

# 3. Feature Ablation Study
baseline_auc : float = roc_auc_score(y_true_arr, y_prob)
ablation_results : dict = {}
target_features : list = top_10_features[ "Feature" ].tolist() if isinstance(top_10_features, pd.DataFrame) else top_10_features

for feature in target_features :
    if feature in valid_features :
        idx : int = valid_features.index(feature)
        X_ablated : np.ndarray = X_test.copy()
        X_ablated[ :, idx ] = 0 
        y_prob_ablated : np.ndarray = clf.predict_proba(X_ablated)[ :, 1 ]
        ablation_results[ clinical_ko_map.get(feature, feature) ] = baseline_auc - roc_auc_score(y_true_arr, y_prob_ablated)

ab_df : pd.DataFrame = pd.DataFrame(list(ablation_results.items()), columns = [ "지표", "Drop" ]).sort_values(by = "Drop", ascending = True)
colors : list = plt.cm.get_cmap( "tab10" )(np.linspace(0, 1, len(ab_df)))

fig_ab, ax_ab = plt.subplots(figsize = (11, 7))
bars = ax_ab.barh(ab_df[ "지표" ], ab_df[ "Drop" ], color = colors, alpha = 0.85)

for bar in bars :
    width = bar.get_width()
    ax_ab.text(width + 0.001, bar.get_y() + bar.get_height() / 2, f"{width:.4f}", 
               va = "center", ha = "left", fontsize = 10, fontweight = "bold")

ax_ab.set_title( "Feature Ablation : 지표 누락 시 AUROC 하락폭", fontsize = 18, fontweight = "bold", pad = 25 )
ax_ab.set_xlabel( "AUROC Drop", fontsize = 12 )
ax_ab.set_xlim( 0, max(ab_df[ "Drop" ]) * 1.2 )
ax_ab.grid(axis = "x", ls = "--", alpha = 0.4)
plt.tight_layout()

force_fix_plot_tofu(fig_ab, ax_ab)
plt.show()

# 23. False Positive (위양성) 케이스 심층 분석 : 조기 경보 관점

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import shap

# 변수 매핑 및 FP 인덱스 추출
X_test_arr : np.ndarray = X_test if isinstance(X_test, np.ndarray) else X_test.values
y_true_arr : np.ndarray = y_true if isinstance(y_true, np.ndarray) else y_true.values
y_pred : np.ndarray = preds_class

fp_indices : np.ndarray = np.where((y_true_arr == 0) & (y_pred == 1))[ 0 ]

if len(fp_indices) > 0 :
    fp_idx : int = fp_indices[ 0 ]
    
    # SHAP 분석 수행
    shap_vals_fp : np.ndarray = explainer_local.shap_values(X_test_arr[ [ fp_idx ] ])
    
    fig_fp, ax_fp = plt.subplots(figsize = ( 10, 8 ))
    explanation_fp = shap.Explanation(
        values = shap_vals_fp[ 0 ], 
        base_values = explainer_local.expected_value, 
        data = X_test_arr[ fp_idx ], 
        feature_names = [ clinical_ko_map.get(f, f) for f in valid_features ]
    )
    
    # Waterfall 시각화
    shap.plots.waterfall(explanation_fp, max_display = 10, show = False)
    ax_fp.set_title("False Positive Case Analysis (Early Warning)", fontsize = 16, fontweight = "bold", pad = 25)
    
    plt.tight_layout()
    plt.show()
else :
    print("No False Positive cases found.")

# 24. Advanced Algorithmic Integrity : AUPRC & Learning Curve

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.text as mtext
from matplotlib import font_manager, rc
from sklearn.metrics import precision_recall_curve, average_precision_score

# 폰트 및 유니코드 마이너스 설정
font_path : str = "C:/Windows/Fonts/malgun.ttf"
font_name : str = font_manager.FontProperties(fname = font_path).get_name()
rc('font', family = font_name)
plt.rcParams[ "axes.unicode_minus" ] = False 

# 텍스트 깨짐 치환 함수
def force_fix_plot_tofu(fig, ax) :
    fig.canvas.draw()
    ticks : list = ax.get_yticks()
    labels : list = [ item.get_text() for item in ax.get_yticklabels() ]
    new_labels : list = [ label.replace('\u2212', '-').replace('−', '-').replace('ㅁ', '-') for label in labels ]
    ax.set_yticks(ticks)
    ax.set_yticklabels(new_labels)
    for text_obj in fig.findobj(mtext.Text) :
        t : str = text_obj.get_text()
        if t :
            cleaned : str = t.replace('\u2212', '-').replace('−', '-').replace('ㅁ', '-')
            text_obj.set_text(cleaned)


# 1. AUPRC (Area Under the Precision-Recall Curve)

precision, recall, _ = precision_recall_curve(y_true_arr, y_prob)
auprc_score : float = average_precision_score(y_true_arr, y_prob)
baseline_prevalence : float = np.mean(y_true_arr)

fig_prc, ax_prc = plt.subplots(figsize = (9, 7))
ax_prc.plot(recall, precision, color = "#0047AB", lw = 3, label = "TabNet")
ax_prc.axhline(y = baseline_prevalence, color = "black", ls = "--", alpha = 0.5, label = "Baseline")

# [수치 표기] AUPRC 및 Baseline 점수 텍스트 추가
ax_prc.text(0.6, 0.8, f"AUPRC : {auprc_score:.4f}", fontsize = 12, fontweight = "bold", color = "#0047AB", bbox = dict(facecolor = 'white', alpha = 0.8, edgecolor = '#0047AB'))
ax_prc.text(0.6, baseline_prevalence + 0.05, f"Baseline : {baseline_prevalence:.4f}", fontsize = 10, color = "black", bbox = dict(facecolor = 'white', alpha = 0.6, edgecolor = 'black'))

ax_prc.set_title( "정밀도-재현율 곡선 (AUPRC Analysis)", fontsize = 18, fontweight = "bold", pad = 25 )
ax_prc.set_xlabel( "재현율 (Recall / Sensitivity)", fontsize = 12 )
ax_prc.set_ylabel( "정밀도 (Precision / PPV)", fontsize = 12 )
ax_prc.legend(loc = "lower left", fontsize = 12)
ax_prc.grid(alpha = 0.2)
plt.tight_layout()

force_fix_plot_tofu(fig_prc, ax_prc)
plt.show()


# 2. TabNet Learning Curve

train_loss : list = clf.history[ "loss" ]

# 안전한 Validation Key 탐색
possible_val_keys : list = [ "val_0_auc", "valid_auc", "val_auc", "val_0_logloss", "valid_logloss", "val_0_rmse" ]
val_keys : list = []
for key in possible_val_keys :
    try :
        if len(clf.history[ key ]) > 0 :
            val_keys.append(key)
    except (KeyError, TypeError, AttributeError) :
        pass

fig_lc, ax_lc1 = plt.subplots(figsize = (11, 6))
epochs : list = list(range(1, len(train_loss) + 1))

# Train Loss 렌더링 및 최종 수치 표기
ax_lc1.plot(epochs, train_loss, color = "crimson", lw = 2, label = "Training Loss")
ax_lc1.text(epochs[-1], train_loss[-1], f"{train_loss[-1]:.4f}", color = "crimson", fontweight = "bold", ha = 'left', va = 'center')
ax_lc1.set_xlabel( "학습 에포크 (Epochs)", fontsize = 12 )
ax_lc1.set_ylabel( "Training Loss", fontsize = 12, color = "crimson" )
ax_lc1.tick_params(axis = "y", labelcolor = "crimson")

# Validation Metric 렌더링 및 최종 수치 표기
if val_keys :
    val_key : str = val_keys[ 0 ]
    val_metric : list = clf.history[ val_key ]
    
    ax_lc2 = ax_lc1.twinx()
    ax_lc2.plot(epochs, val_metric, color = "#0047AB", lw = 2, ls = "--", label = f"Validation ({val_key})")
    ax_lc2.text(epochs[-1], val_metric[-1], f"{val_metric[-1]:.4f}", color = "#0047AB", fontweight = "bold", ha = 'left', va = 'center')
    ax_lc2.set_ylabel( f"Validation Metric ({val_key})", fontsize = 12, color = "#0047AB" )
    ax_lc2.tick_params(axis = "y", labelcolor = "#0047AB")
    
    lines_1, labels_1 = ax_lc1.get_legend_handles_labels()
    lines_2, labels_2 = ax_lc2.get_legend_handles_labels()
    ax_lc1.legend(lines_1 + lines_2, labels_1 + labels_2, loc = "center right", fontsize = 11)
else :
    ax_lc1.legend(loc = "upper right", fontsize = 11)

ax_lc1.set_title( "TabNet 학습 곡선", fontsize = 18, fontweight = "bold", pad = 25 )
ax_lc1.grid(alpha = 0.2)
plt.tight_layout()

force_fix_plot_tofu(fig_lc, ax_lc1)
if val_keys :
    force_fix_plot_tofu(fig_lc, ax_lc2)
plt.show()

# 25. Analytical Integrity : Global XAI, Subgroups, and Probability Density

In [ ]:
import shap
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.text as mtext
from matplotlib import font_manager, rc
from sklearn.metrics import roc_auc_score

# 폰트 및 유니코드 마이너스 설정
font_path : str = "C:/Windows/Fonts/malgun.ttf"
font_name : str = font_manager.FontProperties(fname = font_path).get_name()
rc('font', family = font_name)
plt.rcParams[ "axes.unicode_minus" ] = False 

# 텍스트 깨짐 강제 치환 함수
def force_fix_plot_tofu(fig, ax) :
    fig.canvas.draw()
    ticks : list = ax.get_yticks()
    labels : list = [ item.get_text() for item in ax.get_yticklabels() ]
    new_labels : list = [ label.replace('\u2212', '-').replace('−', '-').replace('ㅁ', '-') for label in labels ]
    ax.set_yticks(ticks)
    ax.set_yticklabels(new_labels)
    for text_obj in fig.findobj(mtext.Text) :
        t : str = text_obj.get_text()
        if t :
            cleaned : str = t.replace('\u2212', '-').replace('−', '-').replace('ㅁ', '-')
            text_obj.set_text(cleaned)

# 1. Global XAI : Mean Absolute SHAP
np.random.seed(42)
sample_size : int = min(300, len(X_test_arr))
sample_idx : np.ndarray = np.random.choice(len(X_test_arr), sample_size, replace = False)
X_test_sampled : np.ndarray = X_test_arr[ sample_idx ]

shap_vals_global : np.ndarray = explainer_local.shap_values(X_test_sampled)
feature_names_ko : list = [ clinical_ko_map.get(f, f) for f in valid_features ]

mean_abs_shap : np.ndarray = np.abs(shap_vals_global).mean(axis = 0)
top_10_idx : np.ndarray = np.argsort(mean_abs_shap)[ -10: ]
num_bars : int = len(top_10_idx)

colors_global : list = plt.cm.get_cmap("tab10")(np.linspace(0, 1, num_bars))

fig_global, ax_global = plt.subplots(figsize = (10, 8))
bars_global = ax_global.barh(
    np.array(feature_names_ko)[ top_10_idx ], 
    mean_abs_shap[ top_10_idx ], 
    color = colors_global, alpha = 0.85
)

for bar in bars_global :
    width = bar.get_width()
    ax_global.text(
        width + 0.002, bar.get_y() + bar.get_height() / 2, 
        f"{width:.4f}", 
        va = "center", ha = "left", fontsize = 11, fontweight = "bold"
    )

ax_global.set_title( "Global XAI : 전역적 변수 중요도", fontsize = 16, fontweight = "bold", pad = 25 )
ax_global.set_xlabel( "평균 절대 SHAP 기여도", fontsize = 12 )
ax_global.set_xlim( 0, max(mean_abs_shap) * 1.15 )
ax_global.grid(axis = "x", ls = "--", alpha = 0.4)
plt.tight_layout()

force_fix_plot_tofu(fig_global, ax_global)
plt.show()

# 2. Subgroup Analysis : Forest Plot
subgroup_results : list = []

total_auc : float = roc_auc_score(y_true_arr, y_prob)
subgroup_results.append({ "Group" : "Total Population", "AUROC" : total_auc, "N" : len(y_true_arr) })

if "성별" in feature_names_ko :
    gender_idx : int = feature_names_ko.index("성별")
    for g_val, g_name in zip([ 1, 2 ], [ "Male", "Female" ]) :
        mask : np.ndarray = (X_test_arr[ :, gender_idx ] == g_val)
        if mask.sum() > 10 and len(np.unique(y_true_arr[ mask ])) > 1 :
            subgroup_results.append({ "Group" : f"Gender : {g_name}", "AUROC" : roc_auc_score(y_true_arr[ mask ], y_prob[ mask ]), "N" : mask.sum() })

if "허리둘레" in feature_names_ko :
    waist_idx : int = feature_names_ko.index("허리둘레")
    for w_cond, w_name in zip([ X_test_arr[ :, waist_idx ] >= 90, X_test_arr[ :, waist_idx ] < 90 ], [ "Waist >= 90", "Waist < 90" ]) :
        if w_cond.sum() > 10 and len(np.unique(y_true_arr[ w_cond ])) > 1 :
            subgroup_results.append({ "Group" : w_name, "AUROC" : roc_auc_score(y_true_arr[ w_cond ], y_prob[ w_cond ]), "N" : w_cond.sum() })

sg_df : pd.DataFrame = pd.DataFrame(subgroup_results)[ ::-1 ]

fig_forest, ax_forest = plt.subplots(figsize = (9, 5))
ax_forest.errorbar(sg_df[ "AUROC" ], range(len(sg_df)), xerr = 0.005, fmt = 'o', color = '#0047AB', ecolor = 'crimson', elinewidth = 2, capsize = 4, markersize = 8)
ax_forest.axvline(x = total_auc, color = 'black', linestyle = '--', alpha = 0.5)

ax_forest.set_yticks(range(len(sg_df)))
ax_forest.set_yticklabels([ f"{row['Group']} (N={row['N']})" for _, row in sg_df.iterrows() ])

for i, auc_val in enumerate(sg_df[ "AUROC" ]) :
    ax_forest.text(auc_val + 0.008, i, f"AUC : {auc_val:.4f}", va = 'center', fontweight = 'bold', color = "crimson")

ax_forest.set_xlabel( "AUROC Score", fontsize = 12 )
ax_forest.set_title( "Subgroup Analysis : 하위 그룹별 예측 성능", fontsize = 16, fontweight = "bold", pad = 20 )
ax_forest.set_xlim( min(sg_df[ "AUROC" ]) - 0.02, max(sg_df[ "AUROC" ]) + 0.03 )
ax_forest.grid(axis = 'x', linestyle = '--', alpha = 0.4)
plt.tight_layout()

force_fix_plot_tofu(fig_forest, ax_forest)
plt.show()

# 3. Probability Density : KDE Plot
fig_kde, ax_kde = plt.subplots(figsize = (10, 6))

prob_target_0 : np.ndarray = y_prob[ y_true_arr == 0 ]
prob_target_1 : np.ndarray = y_prob[ y_true_arr == 1 ]

sns.kdeplot(prob_target_0, fill = True, color = "#0047AB", label = "Target 0 (Normal)", alpha = 0.4, ax = ax_kde)
sns.kdeplot(prob_target_1, fill = True, color = "crimson", label = "Target 1 (Diabetic Risk)", alpha = 0.4, ax = ax_kde)

optimal_threshold : float = 0.3965
ax_kde.axvline(x = optimal_threshold, color = "black", linestyle = "--", linewidth = 2)

mean_prob_0 : float = np.mean(prob_target_0)
mean_prob_1 : float = np.mean(prob_target_1)
y_max : float = ax_kde.get_ylim()[ 1 ]

ax_kde.text(mean_prob_0, y_max * 0.5, f"Mean:\n{mean_prob_0:.3f}", ha = "center", color = "#0047AB", fontweight = "bold", bbox = dict(boxstyle = "round", fc = "white", ec = "#0047AB", alpha = 0.8))
ax_kde.text(mean_prob_1, y_max * 0.5, f"Mean:\n{mean_prob_1:.3f}", ha = "center", color = "crimson", fontweight = "bold", bbox = dict(boxstyle = "round", fc = "white", ec = "crimson", alpha = 0.8))
ax_kde.text(optimal_threshold + 0.02, y_max * 0.8, f"Optimal Threshold\n(T = {optimal_threshold:.4f})", va = "center", fontweight = "bold", color = "black")

ax_kde.set_title( "클래스별 예측 확률 밀도 분포", fontsize = 16, fontweight = "bold", pad = 20 )
ax_kde.set_xlabel( "Predicted Probability of Diabetes", fontsize = 12 )
ax_kde.set_ylabel( "Density", fontsize = 12 )
ax_kde.set_xlim( -0.1, 1.1 )
ax_kde.legend(loc = "upper center", fontsize = 11)
ax_kde.grid(alpha = 0.2)
plt.tight_layout()

force_fix_plot_tofu(fig_kde, ax_kde)
plt.show()

# 26. Interaction Analysis & Precise Metrics Table

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, f1_score

# 1. SHAP Interaction Analysis
top_idx_1 : int = top_10_idx[ -1 ]
top_idx_2 : int = top_10_idx[ -2 ]

shap.dependence_plot(
    top_idx_1, 
    shap_vals_global, 
    X_test_sampled, 
    display_features = X_test_sampled,
    feature_names = feature_names_ko,
    interaction_index = top_idx_2,
    show = False
)

fig_inter = plt.gcf()
ax_inter = plt.gca()

ax_inter.set_title( f"Interaction Analysis : {feature_names_ko[top_idx_1]} x {feature_names_ko[top_idx_2]}", fontsize = 15, fontweight = "bold", pad = 20 )
plt.tight_layout()

force_fix_plot_tofu(fig_inter, ax_inter)
plt.show()

# 2. Comprehensive Clinical Metrics Table
def get_clinical_metrics(y_true, y_prob, threshold : float = 0.3965, n_boot : int = 1000) :
    boot_results : list = []
    
    for _ in range(n_boot) :
        idx : np.ndarray = np.random.randint(0, len(y_true), len(y_true))
        b_true, b_prob = y_true[ idx ], y_prob[ idx ]
        
        if len(np.unique(b_true)) < 2 : continue
        
        b_pred : np.ndarray = (b_prob >= threshold).astype(int)
        tn, fp, fn, tp = confusion_matrix(b_true, b_pred).ravel()
        
        sens : float = tp / (tp + fn)
        spec : float = tn / (tn + fp)
        ppv : float = tp / (tp + fp) if (tp + fp) > 0 else 0
        npv : float = tn / (tn + fn) if (tn + fn) > 0 else 0
        f1 : float = f1_score(b_true, b_pred)
        
        boot_results.append([ sens, spec, ppv, npv, f1 ])
    
    res_arr : np.ndarray = np.array(boot_results)
    means : np.ndarray = np.mean(res_arr, axis = 0)
    lowers : np.ndarray = np.percentile(res_arr, 2.5, axis = 0)
    uppers : np.ndarray = np.percentile(res_arr, 97.5, axis = 0)
    
    metrics_df : pd.DataFrame = pd.DataFrame({
        "Metric" : [ "Sensitivity", "Specificity", "PPV (Precision)", "NPV", "F1-Score" ],
        "Mean" : means,
        "95% CI Lower" : lowers,
        "95% CI Upper" : uppers
    })
    return metrics_df

final_metrics : pd.DataFrame = get_clinical_metrics(y_true_arr, y_prob)
print( "\n[ Clinical Performance Metrics at Threshold 0.3965 ]" )
print( final_metrics.to_string(index = False, float_format = lambda x : "{:.4f}".format(x)) )

# 27. Clinical Utility and Comparative Analysis

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

y_true : np.ndarray = y_true_arr
y_prob : np.ndarray = y_prob_tabnet 

fpr_tmp, tpr_tmp, thresholds_tmp = roc_curve(y_true, y_prob)
opt_idx_roc : int = np.argmax(tpr_tmp - fpr_tmp)
opt_threshold : float = thresholds_tmp[ opt_idx_roc ]

def calculate_net_benefit(y_true : np.ndarray, y_prob : np.ndarray, thresholds : np.ndarray) :
    net_benefits : list = []
    n : int = len(y_true)
    for t in thresholds :
        y_pred : np.ndarray = (y_prob >= t).astype(int)
        tp : int = np.sum((y_pred == 1) & (y_true == 1))
        fp : int = np.sum((y_pred == 1) & (y_true == 0))
        
        if t == 1.0 :
            net_benefit : float = 0.0
        else :
            net_benefit : float = (tp / n) - (fp / n) * (t / (1 - t))
        net_benefits.append(net_benefit)
    return net_benefits

thresholds : np.ndarray = np.linspace(0.01, 0.99, 100)
tabnet_nb : list = calculate_net_benefit(y_true, y_prob, thresholds)
all_nb : list = calculate_net_benefit(y_true, np.ones(len(y_true)), thresholds)
none_nb : list = [ 0.0 ] * len(thresholds)

fig_dca, ax_dca = plt.subplots(figsize = ( 10, 8 ))
ax_dca.plot(thresholds, tabnet_nb, color = "#0047AB", lw = 3, label = "TabNet")
ax_dca.plot(thresholds, all_nb, color = "black", ls = "--", alpha = 0.5, label = "Treat All")
ax_dca.plot(thresholds, none_nb, color = "gray", ls = "-", alpha = 0.3, label = "Treat None")

opt_nb : float = tabnet_nb[ np.argmin(np.abs(thresholds - opt_threshold)) ]

# 최적 지점 마킹 및 수치 표기
ax_dca.scatter([ opt_threshold ], [ opt_nb ], color = "crimson", s = 100, zorder = 5)
ax_dca.text(opt_threshold + 0.02, opt_nb, f"{opt_nb:.4f}", color = "crimson", fontsize = 13, fontweight = "bold", va = "center")

ax_dca.set_ylim(-0.02, max(tabnet_nb) * 1.2)
ax_dca.set_xlim(-0.05, 1.05)

ax_dca.set_title("의사결정 곡선 분석 (Decision Curve Analysis)", fontsize = 16, fontweight = "bold", pad = 20)
ax_dca.set_xlabel("Threshold Probability", fontsize = 12)
ax_dca.set_ylabel("Net Benefit", fontsize = 12)
ax_dca.legend(loc = "upper right")
ax_dca.grid(alpha = 0.2)
plt.tight_layout()

try :
    force_fix_plot_tofu(fig_dca, ax_dca)
except NameError :
    pass

plt.show()

# 28. High-End Engineering : Error Profiling, Uncertainty, and Cost-Optimization

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix

device : str = "cuda" if torch.cuda.is_available() else "cpu"

# 1. Error Profiling : Clustering with Statistics
error_mask : np.ndarray = (y_pred != y_true_arr)
X_error : np.ndarray = X_test_arr[ error_mask ]
y_error_true : np.ndarray = y_true_arr[ error_mask ]

pca : PCA = PCA(n_components = 2)
X_error_pca : np.ndarray = pca.fit_transform(X_error)

kmeans : KMeans = KMeans(n_clusters = 3, random_state = 42)
error_clusters : np.ndarray = kmeans.fit_predict(X_error_pca)

fig_err, ax_err = plt.subplots(figsize = ( 11, 8 ))
scatter = ax_err.scatter(X_error_pca[ :, 0 ], X_error_pca[ :, 1 ], c = error_clusters, cmap = "viridis", alpha = 0.5, edgecolor = "white", s = 60)

centroids : np.ndarray = kmeans.cluster_centers_
for i, (cx, cy) in enumerate(centroids) :
    n_samples : int = np.sum(error_clusters == i)
    ax_err.annotate(
        f"Cluster {i}\nN = {n_samples}", 
        xy = (cx, cy), xytext = (cx + 0.5, cy + 0.5),
        bbox = dict(boxstyle = "round,pad=0.4", fc = "white", ec = "black", alpha = 0.9),
        arrowprops = dict(arrowstyle = "->", connectionstyle = "arc3,rad=0.2"),
        fontsize = 10, fontweight = "bold"
    )

ax_err.set_title("Classification Error Profiling", fontsize = 16, fontweight = "bold", pad = 20)
plt.colorbar(scatter, label = "Cluster ID")
plt.tight_layout()
force_fix_plot_tofu(fig_err, ax_err)
plt.show()

# 2. Uncertainty Quantification : Probabilistic Distribution
def quantify_uncertainty(model, X, n_iterations : int = 50) :
    preds : list = []
    for _ in range(n_iterations) :
        idx : np.ndarray = np.random.randint(0, len(X), len(X))
        it_prob : np.ndarray = model.predict_proba(X[ idx ])[ :, 1 ]
        preds.append(it_prob)
    
    preds_arr : np.ndarray = np.array(preds)
    return np.mean(preds_arr, axis = 0), np.std(preds_arr, axis = 0), \
           np.percentile(preds_arr, 2.5, axis = 0), np.percentile(preds_arr, 97.5, axis = 0)

mean_p, std_p, lower_p, upper_p = quantify_uncertainty(clf, X_test_arr[ :20 ])

fig_unc, ax_unc = plt.subplots(figsize = ( 12, 6 ))
sample_idx : np.ndarray = np.arange(len(mean_p))

ax_unc.errorbar(
    sample_idx, mean_p, 
    yerr = [ mean_p - lower_p, upper_p - mean_p ], 
    fmt = "o", color = "#0047AB", ecolor = "crimson", capsize = 6, elinewidth = 1.5, markersize = 8
)

for i, val in enumerate(mean_p) :
    ax_unc.text(i, val + 0.04, f"{val:.3f}", ha = "center", va = "bottom", fontsize = 9, fontweight = "bold", color = "#0047AB")

ax_unc.set_title("Prediction Uncertainty Quantification (95% CI)", fontsize = 16, fontweight = "bold", pad = 20)
ax_unc.set_xlabel("Sample Index", fontsize = 12)
ax_unc.set_ylabel("Probability with Uncertainty", fontsize = 12)
ax_unc.set_ylim(-0.05, 1.15)
ax_unc.grid(axis = "y", alpha = 0.2)
plt.tight_layout()
force_fix_plot_tofu(fig_unc, ax_unc)
plt.show()

# 3. Clinical Cost-Sensitive Optimization
def get_cost_curve(y_true, y_prob, penalty : int = 5) :
    thresholds : np.ndarray = np.linspace(0.01, 0.99, 100)
    costs : list = []
    for t in thresholds :
        y_p : np.ndarray = (y_prob >= t).astype(int)
        tn, fp, fn, tp = confusion_matrix(y_true, y_p).ravel()
        costs.append((fn * penalty) + fp)
    return thresholds, np.array(costs)

fig_cost, ax_cost = plt.subplots(figsize = ( 11, 7 ))
penalties : list = [ 1, 3, 5 ]
colors : list = [ "gray", "#0047AB", "crimson" ]

for p, c in zip(penalties, colors) :
    ts, cs = get_cost_curve(y_true_arr, y_prob, penalty = p)
    opt_t : float = ts[ np.argmin(cs) ]
    min_c : float = np.min(cs)
    
    ax_cost.plot(ts, cs, color = c, lw = 2.5, label = f"FN Penalty : {p}x")
    ax_cost.scatter(opt_t, min_c, color = c, s = 100, edgecolors = "black", zorder = 5)
    
    ax_cost.annotate(
        f"T = {opt_t:.3f}\nCost = {min_c:.0f}", 
        xy = (opt_t, min_c), xytext = (opt_t + 0.05, min_c + (np.max(cs) * 0.05)),
        arrowprops = dict(arrowstyle = "->", color = c, lw = 1.5),
        fontsize = 10, fontweight = "bold", color = c
    )

ax_cost.set_title("Clinical Cost-Sensitive Threshold Optimization", fontsize = 16, fontweight = "bold", pad = 20)
ax_cost.set_xlabel("Decision Threshold", fontsize = 12)
ax_cost.set_ylabel("Weighted Clinical Cost", fontsize = 12)
ax_cost.legend(fontsize = 11)
ax_unc.grid(alpha = 0.2)
plt.tight_layout()
force_fix_plot_tofu(fig_cost, ax_cost)
plt.show()

# 29. Blind-spot Signature : 오분류 군집별 임상 특징 추출

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Blind-spot Signature Extraction
X_test_df : pd.DataFrame = pd.DataFrame(X_test_arr, columns = feature_names_ko)
X_error_df : pd.DataFrame = X_test_df.iloc[ error_mask ].copy()
X_error_df[ "Cluster" ] = error_clusters

# 전체 인구 평균 대비 군집별 편차(Z-score 기반) 산출
total_mean : pd.Series = X_test_df.mean()
total_std : pd.Series = X_test_df.std()
cluster_profiles : pd.DataFrame = X_error_df.groupby( "Cluster" ).mean().drop( columns = [] )
relative_deviation : pd.DataFrame = (cluster_profiles - total_mean) / total_std

# 상위 기여 피처 5개 선정 및 시각화
top_diff_features : list = relative_deviation.abs().mean().sort_values( ascending = False ).head( 5 ).index.tolist()
plot_df : pd.DataFrame = relative_deviation[ top_diff_features ].T

fig_sig, ax_sig = plt.subplots(figsize = ( 11, 7 ))
plot_df.plot(kind = "bar", ax = ax_sig, color = [ "gray", "#0047AB", "crimson" ], alpha = 0.8)

# 그래프 내 수치값(Deviation) 직접 표기
for container in ax_sig.containers :
    labels : list = [ f"{v.get_height():.2f}" for v in container ]
    ax_sig.bar_label(container, labels = labels, label_type = "edge", fontsize = 9, fontweight = "bold", padding = 3)

ax_sig.set_title( "Blind-spot Signature : 에러 군집별 피처 편차 분석", fontsize = 16, fontweight = "bold", pad = 20 )
ax_sig.set_ylabel( "표준 편차 기준 차이 (Z-score)", fontsize = 12 )
ax_sig.axhline( 0, color = "black", lw = 1 )
ax_sig.legend( title = "Error Cluster", loc = "upper right" )
ax_sig.grid( axis = "y", alpha = 0.2 )
plt.xticks( rotation = 45 )
plt.tight_layout()

force_fix_plot_tofu(fig_sig, ax_sig)
plt.show()

# 2. Risk Stratification Tiering
risk_tiers : list = [ "Low", "Moderate", "High", "Urgent" ]
bins : list = [ 0, 0.2, 0.4, 0.8, 1.0 ]
y_prob_tiers : pd.Series = pd.cut(y_prob, bins = bins, labels = risk_tiers)
tier_counts : pd.Series = y_prob_tiers.value_counts().reindex(risk_tiers)

fig_tier, ax_tier = plt.subplots(figsize = ( 10, 6 ))
bars = ax_tier.bar(risk_tiers, tier_counts, color = sns.color_palette( "coolwarm", 4 ), edgecolor = "black", alpha = 0.7)

# 각 등급별 환자 수 및 비율 표기
total_n : int = len(y_prob)
for bar in bars :
    height = bar.get_height()
    ax_tier.text(
        bar.get_x() + bar.get_width()/2, height + 50,
        f"N = {int(height)}\n({height/total_n:.1%})",
        ha = "center", va = "bottom", fontsize = 11, fontweight = "bold"
    )

ax_tier.set_title( "Clinical Risk Stratification : 위험 등급별 환자 분포", fontsize = 16, fontweight = "bold", pad = 20 )
ax_tier.set_xlabel( "Risk Tier", fontsize = 12 )
ax_tier.set_ylabel( "Patient Count", fontsize = 12 )
ax_tier.set_ylim( 0, max(tier_counts) * 1.2 )
ax_tier.grid( axis = "y", alpha = 0.2 )
plt.tight_layout()

force_fix_plot_tofu(fig_tier, ax_tier)
plt.show()

# 30. Engineering Completion : Robustness, Calibration, and Tier-specific SHAP

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from sklearn.metrics import brier_score_loss, confusion_matrix

# GPU 디바이스 설정 및 메모리 최적화
device : str = "cuda" if torch.cuda.is_available() else "cpu"

# 1. GPU-Accelerated Feature Sensitivity Stress Test
def conduct_stress_test_gpu(model, X_arr, noise_level : float = 0.05, n_iterations : int = 30) :
    # 데이터를 GPU 텐서로 변환
    X_tensor : torch.Tensor = torch.tensor(X_arr, dtype = torch.float32).to(device)
    
    # 원본 확률 추출 (TabNet 내부에서 처리)
    original_probs : np.ndarray = model.predict_proba(X_arr)[ :, 1 ]
    volatilities : list = []
    
    for _ in range(n_iterations) :
        # GPU 내에서 직접 가우시안 노이즈 생성하여 연산 속도 극대화
        noise : torch.Tensor = torch.randn_like(X_tensor) * noise_level
        X_noisy_tensor : torch.Tensor = X_tensor + (X_tensor * noise)
        
        # 모델 추론
        X_noisy_np : np.ndarray = X_noisy_tensor.cpu().numpy()
        noisy_probs : np.ndarray = model.predict_proba(X_noisy_np)[ :, 1 ]
        
        diff : np.ndarray = np.abs(noisy_probs - original_probs)
        volatilities.append(np.mean(diff))
        
    return volatilities

noise_levels : list = [ 0.05, 0.10, 0.15 ]
stress_results : dict = {}

for nl in noise_levels :
    stress_results[ f"{int(nl*100)}%" ] = conduct_stress_test_gpu(clf, X_test_arr, noise_level = nl)

fig_stress, ax_stress = plt.subplots(figsize = ( 10, 6 ))
sns.boxplot(data = pd.DataFrame(stress_results), ax = ax_stress, palette = "pastel")

for i, nl in enumerate(noise_levels) :
    mean_vol : float = np.mean(stress_results[ f"{int(nl*100)}%" ])
    ax_stress.text(i, mean_vol + 0.002, f"Mean Volatility : {mean_vol:.4f}", 
                  ha = "center", fontweight = "bold", color = "black")

ax_stress.set_title( "Feature Sensitivity Stress Test : GPU 가속 데이터 견고성", fontsize = 16, fontweight = "bold", pad = 20 )
ax_stress.set_xlabel( "Noise Level", fontsize = 12 )
ax_stress.set_ylabel( "Mean Probability Volatility", fontsize = 12 )
ax_stress.grid( axis = "y", alpha = 0.2 )
plt.tight_layout()
force_fix_plot_tofu(fig_stress, ax_stress)
plt.show()

# 2. GPU-Based Expected Calibration Error (ECE)
def calculate_ece_gpu(y_true, y_prob, n_bins : int = 10) :
    y_true_t : torch.Tensor = torch.tensor(y_true, dtype = torch.float32).to(device)
    y_prob_t : torch.Tensor = torch.tensor(y_prob, dtype = torch.float32).to(device)
    
    bin_boundaries : torch.Tensor = torch.linspace(0, 1, n_bins + 1).to(device)
    ece : torch.Tensor = torch.tensor(0.0).to(device)
    
    for i in range(n_bins) :
        lower : torch.Tensor = bin_boundaries[ i ]
        upper : torch.Tensor = bin_boundaries[ i + 1 ]
        
        # 마스킹 연산을 통한 빈(Bin) 할당
        in_bin : torch.Tensor = (y_prob_t > lower) & (y_prob_t <= upper)
        prop_in_bin : torch.Tensor = torch.mean(in_bin.float())
        
        if prop_in_bin > 0 :
            acc_in_bin : torch.Tensor = torch.mean(y_true_t[ in_bin ])
            conf_in_bin : torch.Tensor = torch.mean(y_prob_t[ in_bin ])
            ece += torch.abs(conf_in_bin - acc_in_bin) * prop_in_bin
            
    return ece.item()

ece_val : float = calculate_ece_gpu(y_true_arr, y_prob)
brier_val : float = brier_score_loss(y_true_arr, y_prob)

print( f"\n[ Quantitative Calibration Assessment - GPU Accelerated ]" )
print( f"Expected Calibration Error (ECE) : {ece_val:.4f}" )
print( f"Brier Score : {brier_val:.4f}" )

# 3. Urgent-Tier SHAP Analysis : Multi-colored Visualization
urgent_mask : np.ndarray = (y_prob >= 0.8)
X_urgent : np.ndarray = X_test_arr[ urgent_mask ]
if len(X_urgent) > 200 :
    X_urgent = X_urgent[ np.random.choice(len(X_urgent), 200, replace = False) ]

shap_vals_urgent : np.ndarray = explainer_local.shap_values(X_urgent)

fig_urgent, ax_urgent = plt.subplots(figsize = ( 10, 8 ))
mean_abs_shap_urgent : np.ndarray = np.abs(shap_vals_urgent).mean(axis = 0)
top_5_idx_urgent : np.ndarray = np.argsort(mean_abs_shap_urgent)[ -5: ]
num_bars : int = len(top_5_idx_urgent)

colors_urgent : list = plt.cm.get_cmap("tab10")(np.linspace(0, 1, num_bars))

bars_urgent = ax_urgent.barh(
    np.array(feature_names_ko)[ top_5_idx_urgent ], 
    mean_abs_shap_urgent[ top_5_idx_urgent ], 
    color = colors_urgent, alpha = 0.8
)

for bar in bars_urgent :
    width = bar.get_width()
    ax_urgent.text(width + 0.002, bar.get_y() + bar.get_height()/2, f"{width:.4f}", va = "center", fontweight = "bold")

ax_urgent.set_title( "Urgent Tier Feature Contribution : 초고위험군 핵심 인자", fontsize = 16, fontweight = "bold", pad = 20 )
ax_urgent.set_xlabel( "Mean Absolute SHAP Value", fontsize = 12 )
plt.tight_layout()
force_fix_plot_tofu(fig_urgent, ax_urgent)
plt.show()

# 31. Prescriptive AI Simulation & Deployment Optimization

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch

device : str = "cuda" if torch.cuda.is_available() else "cpu"

clf.network.to(device)
clf.network.float()

# 1. Counterfactual What-If Simulation
urgent_indices : np.ndarray = np.where(y_prob >= 0.8)[ 0 ]
if len(urgent_indices) > 0 :
    target_idx : int = urgent_indices[ 0 ]
    X_base : np.ndarray = X_test_arr[ target_idx ].copy()
    base_prob : float = y_prob[ target_idx ]

    target_feat_1 : int = top_5_idx_urgent[ -1 ]
    target_feat_2 : int = top_5_idx_urgent[ -2 ]

    X_sim_1 : np.ndarray = X_base.copy()
    X_sim_1[ target_feat_1 ] = np.median(X_test_arr[ :, target_feat_1 ])
    prob_sim_1 : float = clf.predict_proba(X_sim_1.reshape(1, -1))[ 0, 1 ]

    X_sim_2 : np.ndarray = X_sim_1.copy()
    X_sim_2[ target_feat_2 ] = np.median(X_test_arr[ :, target_feat_2 ])
    prob_sim_2 : float = clf.predict_proba(X_sim_2.reshape(1, -1))[ 0, 1 ]

    scenarios : list = [ "Baseline", f"Control: {feature_names_ko[target_feat_1]}", "Control: Both Features" ]
    probs : list = [ base_prob, prob_sim_1, prob_sim_2 ]

    fig_cf, ax_cf = plt.subplots(figsize = ( 10, 6 ))
    colors_cf : list = plt.cm.get_cmap("tab10")(np.linspace(0, 1, len(probs)))

    bars_cf = ax_cf.bar(scenarios, probs, color = colors_cf, alpha = 0.85, edgecolor = "black")

    for bar in bars_cf :
        height = bar.get_height()
        ax_cf.text(bar.get_x() + bar.get_width() / 2, height + 0.02, f"{height:.4f}", ha = "center", va = "bottom", fontweight = "bold", fontsize = 12)

    ax_cf.set_title("Counterfactual What-If Simulation", fontsize = 16, fontweight = "bold", pad = 20)
    ax_cf.set_ylabel("Predicted Risk Probability", fontsize = 12)
    ax_cf.set_ylim(0, 1.1)
    ax_cf.grid(axis = "y", ls = "--", alpha = 0.3)
    plt.tight_layout()
    
    try :
        force_fix_plot_tofu(fig_cf, ax_cf)
    except NameError :
        pass
    plt.show()

# 2. Partial Dependence Plot (PDP)
feat_idx : int = top_5_idx_urgent[ -2 ]
feat_name : str = feature_names_ko[ feat_idx ]

unique_vals : np.ndarray = np.unique(X_test_arr[ :, feat_idx ])

if len(unique_vals) <= 2 :
    grid_vals : np.ndarray = unique_vals
else :
    p05 : float = np.percentile(X_test_arr[ :, feat_idx ], 5)
    p95 : float = np.percentile(X_test_arr[ :, feat_idx ], 95)
    grid_vals : np.ndarray = np.linspace(p05, p95, 50) if p05 != p95 else unique_vals

pdp_probs : list = []
X_temp : np.ndarray = X_test_arr.copy()

for val in grid_vals :
    X_temp[ :, feat_idx ] = val
    mean_prob : float = clf.predict_proba(X_temp)[ :, 1 ].mean()
    pdp_probs.append(mean_prob)

fig_pdp, ax_pdp = plt.subplots(figsize = ( 10, 6 ))

if len(grid_vals) <= 2 :
    ax_pdp.plot(grid_vals, pdp_probs, color = "crimson", lw = 3, marker = "o", markersize = 10)
    ax_pdp.set_xticks(grid_vals)
else :
    ax_pdp.plot(grid_vals, pdp_probs, color = "crimson", lw = 3)

min_idx : int = np.argmin(pdp_probs)
max_idx : int = np.argmax(pdp_probs)

ax_pdp.scatter([ grid_vals[ min_idx ], grid_vals[ max_idx ] ], [ pdp_probs[ min_idx ], pdp_probs[ max_idx ] ], color = "black", s = 100, zorder = 5)

# 수치 위치 교정
ax_pdp.annotate(f"Min: {pdp_probs[min_idx]:.4f}", xy = (grid_vals[ min_idx ], pdp_probs[ min_idx ]), xytext = (0, -20), textcoords = "offset points", ha = "center", fontweight = "bold")
ax_pdp.annotate(f"Max: {pdp_probs[max_idx]:.4f}", xy = (grid_vals[ max_idx ], pdp_probs[ max_idx ]), xytext = (0, 15), textcoords = "offset points", ha = "center", fontweight = "bold")

ax_pdp.set_title(f"Partial Dependence Plot : {feat_name}", fontsize = 16, fontweight = "bold", pad = 20)
ax_pdp.set_xlabel(f"Feature Value", fontsize = 12)
ax_pdp.set_ylabel("Mean Predicted Probability", fontsize = 12)

# y축 스케일 동적 여백 확보
margin : float = (max(pdp_probs) - min(pdp_probs)) * 0.2 if max(pdp_probs) != min(pdp_probs) else 0.05
ax_pdp.set_ylim(min(pdp_probs) - margin, max(pdp_probs) + margin)

ax_pdp.grid(alpha = 0.3)
plt.tight_layout()

try :
    force_fix_plot_tofu(fig_pdp, ax_pdp)
except NameError :
    pass
    
plt.show()

# 32. Clinical Research Validation : Ablation Study & Algorithmic Fairness

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from sklearn.metrics import roc_auc_score, average_precision_score, confusion_matrix

device : str = "cuda" if torch.cuda.is_available() else "cpu"

# 1. Minimal Feature Set Ablation (최소 변수 통제 검증)
top_5_features_idx : np.ndarray = top_10_idx[ -5: ]
X_test_ablated : np.ndarray = X_test_arr.copy()

num_features : int = X_test_arr.shape[ 1 ]
for i in range(num_features) :
    if i not in top_5_features_idx :
        X_test_ablated[ :, i ] = np.median(X_test_arr[ :, i ])

# 통제된 데이터셋으로 예측 수행
y_prob_minimal : np.ndarray = clf.predict_proba(X_test_ablated)[ :, 1 ]

auc_min : float = roc_auc_score(y_true_arr, y_prob_minimal)
prc_min : float = average_precision_score(y_true_arr, y_prob_minimal)

auc_full : float = roc_auc_score(y_true_arr, y_prob)
prc_full : float = average_precision_score(y_true_arr, y_prob)

fig_abl, ax_abl = plt.subplots(figsize = ( 10, 6 ))
metrics_labels : list = [ "Full AUC (All Feat)", "Minimal AUC (Top 5 Only)", "Full PRC (All Feat)", "Minimal PRC (Top 5 Only)" ]
metrics_vals : list = [ auc_full, auc_min, prc_full, prc_min ]
colors_abl : list = plt.cm.get_cmap("tab10")(np.linspace(0, 1, len(metrics_labels)))

bars_abl = ax_abl.bar(metrics_labels, metrics_vals, color = colors_abl, alpha = 0.85, edgecolor = "black")

for bar in bars_abl :
    height = bar.get_height()
    ax_abl.text(bar.get_x() + bar.get_width() / 2, height + 0.02, f"{height:.4f}", ha = "center", va = "bottom", fontweight = "bold", fontsize = 12)

ax_abl.set_title("Minimal Feature Set Ablation Study", fontsize = 16, fontweight = "bold", pad = 20)
ax_abl.set_ylabel("Score", fontsize = 12)
ax_abl.set_ylim(0, 1.15)
ax_abl.grid(axis = "y", ls = "--", alpha = 0.3)
plt.tight_layout()
force_fix_plot_tofu(fig_abl, ax_abl)
plt.show()

# 2. Algorithmic Fairness Assessment (인구통계학적 공정성 검증)
gender_idx : int = feature_names_ko.index("성별")
gender_vals : np.ndarray = X_test_arr[ :, gender_idx ]
unique_genders : np.ndarray = np.unique(gender_vals)

fairness_records : list = []
threshold : float = 0.3965

for g in unique_genders :
    mask : np.ndarray = (gender_vals == g)
    y_t : np.ndarray = y_true_arr[ mask ]
    y_p : np.ndarray = y_prob[ mask ]
    
    y_pred_bin : np.ndarray = (y_p >= threshold).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_t, y_pred_bin).ravel()
    
    tpr : float = tp / (tp + fn) if (tp + fn) > 0 else 0
    fpr : float = fp / (fp + tn) if (fp + tn) > 0 else 0
    
    gender_label : str = "Male" if g == 1.0 else "Female"
    fairness_records.append({ "Demographic_Group" : f"Gender : {gender_label}", "TPR (Recall)" : tpr, "FPR (False Alarm)" : fpr })

df_fairness : pd.DataFrame = pd.DataFrame(fairness_records).set_index("Demographic_Group")

fig_fair, ax_fair = plt.subplots(figsize = ( 10, 6 ))
df_fairness.plot(kind = "bar", ax = ax_fair, color = [ "#0047AB", "crimson" ], alpha = 0.85, edgecolor = "black")

for container in ax_fair.containers :
    labels : list = [ f"{v.get_height():.4f}" for v in container ]
    ax_fair.bar_label(container, labels = labels, label_type = "edge", fontsize = 11, fontweight = "bold", padding = 3)

ax_fair.set_title("Algorithmic Fairness Assessment (Equal Opportunity)", fontsize = 16, fontweight = "bold", pad = 20)
ax_fair.set_xlabel("Demographic Subgroup", fontsize = 12)
ax_fair.set_ylabel("Metric Rate", fontsize = 12)
ax_fair.set_ylim(0, 1.2)
ax_fair.legend(loc = "upper right")
ax_fair.grid(axis = "y", ls = "--", alpha = 0.3)
plt.xticks(rotation = 0)
plt.tight_layout()
force_fix_plot_tofu(fig_fair, ax_fair)
plt.show()

# 33. Reliability & Out-of-Distribution Detection

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.calibration import calibration_curve
from sklearn.ensemble import IsolationForest

# 1. Reliability Diagram (Calibration Curve 시각화)
prob_true, prob_pred = calibration_curve(y_true_arr, y_prob, n_bins = 10)

fig_cal, ax_cal = plt.subplots(figsize = ( 8, 8 ))
ax_cal.plot(prob_pred, prob_true, marker = "s", ls = "-", color = "#0047AB", label = "TabNet Calibration")
ax_cal.plot([ 0, 1 ], [ 0, 1 ], ls = "--", color = "gray", label = "Perfectly Calibrated")

# 정량적 지표(ECE) 삽입
ax_cal.text(0.05, 0.95, f"ECE : {ece_val:.4f}", fontsize = 12, fontweight = "bold", 
            bbox = dict(boxstyle = "round", facecolor = "white", alpha = 0.8))

for p, t in zip(prob_pred, prob_true) :
    ax_cal.annotate(
        f"{t:.3f}", 
        (p, t), textcoords = "offset points", xytext = ( 0, 10 ), 
        ha = "center", fontsize = 10, fontweight = "bold", color = "#0047AB"
    )

ax_cal.set_title("Reliability Diagram (Calibration Curve)", fontsize = 16, fontweight = "bold", pad = 20)
ax_cal.set_xlabel("Mean Predicted Probability", fontsize = 12)
ax_cal.set_ylabel("Fraction of Positives", fontsize = 12)
ax_cal.legend(loc = "lower right")
ax_cal.grid(alpha = 0.2)
plt.tight_layout()
plt.show()

# 2. Out-of-Distribution (OOD) Detection
iso_forest : IsolationForest = IsolationForest(n_estimators = 200, contamination = 0.05, random_state = 42)
iso_forest.fit(X_test_arr)

anomaly_scores : np.ndarray = iso_forest.decision_function(X_test_arr)
is_inlier : np.ndarray = iso_forest.predict(X_test_arr) 
ood_count : int = np.sum(is_inlier == -1)

fig_ood, ax_ood = plt.subplots(figsize = ( 12, 7 ))
counts, bins, patches = ax_ood.hist(anomaly_scores, bins = 50, color = "#4A90E2", edgecolor = "white", alpha = 0.8)
sns.kdeplot(anomaly_scores, ax = ax_ood, color = "#002255", lw = 2)

# 히스토그램 막대별 빈도수 텍스트 매핑
for count, patch in zip(counts, patches) :
    if count > 0 :
        ax_ood.text(
            patch.get_x() + patch.get_width() / 2, count + (np.max(counts) * 0.015), 
            f"{int(count)}", 
            ha = "center", va = "bottom", fontsize = 9, fontweight = "bold", color = "#333333"
        )

# OOD 판정 임계치(하위 5%) 시각화
threshold : float = np.percentile(anomaly_scores, 5)
ax_ood.axvline(threshold, color = "crimson", ls = "--", lw = 2.5)

ax_ood.annotate(
    f"OOD Threshold\n(5% : {threshold:.4f})", 
    xy = (threshold, np.max(counts) * 0.8), xytext = (threshold - 0.03, np.max(counts) * 0.9),
    arrowprops = dict(arrowstyle = "->", color = "crimson", lw = 1.5),
    fontsize = 11, fontweight = "bold", color = "crimson", ha = "right"
)

summary_text : str = f"[ OOD Summary ]\nTotal : {len(anomaly_scores)}\nNormal : {len(anomaly_scores) - ood_count}\nOOD : {ood_count} (5%)"
ax_ood.text(
    0.02, 0.95, summary_text, transform = ax_ood.transAxes, 
    fontsize = 11, fontweight = "bold", va = "top", 
    bbox = dict(boxstyle = "round,pad=0.5", facecolor = "white", edgecolor = "black", alpha = 0.9)
)

ax_ood.set_title("OOD Detection : Anomaly Score Distribution", fontsize = 16, fontweight = "bold", pad = 20)
ax_ood.set_xlabel("Anomaly Score (Higher is more normal)", fontsize = 12)
ax_ood.set_ylabel("Frequency", fontsize = 12)
ax_ood.set_ylim(0, np.max(counts) * 1.15)
ax_ood.grid(axis = "y", alpha = 0.2)
plt.tight_layout()
plt.show()

# 34. Multi-level Model Interpretability : Internal Attention, Feature Interaction, and Clinical Rule Extraction

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier, plot_tree

# 1. TabNet Local Attention Mask Visualization
explain_matrix, masks = clf.explain(X_test_arr)

fig_mask, axes = plt.subplots(1, 3, figsize = ( 16, 6 ))
sample_limit : int = 15

for i in range(3) :
    ax = axes[ i ]
    mask_data : np.ndarray = masks[ i ][ :sample_limit, : ]
    cax = ax.imshow(mask_data, cmap = "viridis", aspect = "auto")
    
    ax.set_title(f"Attention Mask : Step {i+1}", fontsize = 14, fontweight = "bold", pad = 10)
    ax.set_xlabel("Feature Index", fontsize = 11)
    ax.set_ylabel("Sample Index", fontsize = 11)
    
    for row in range(sample_limit) :
        top_feat_idx : int = np.argmax(mask_data[ row ])
        top_val : float = mask_data[ row, top_feat_idx ]
        ax.text(
            top_feat_idx, row, f"{top_val:.2f}", 
            ha = "center", va = "center", color = "white", fontsize = 8, fontweight = "bold"
        )

plt.colorbar(cax, ax = axes, fraction = 0.02, pad = 0.04, label = "Attention Weight")
plt.suptitle("TabNet Local Attention Weights per Step (Top features annotated)", fontsize = 18, fontweight = "bold")
force_fix_plot_tofu(fig_mask, axes[ 0 ])
plt.show()

# 2. SHAP Interaction Value Analysis
np.random.seed(42)
sample_idx : np.ndarray = np.random.choice(len(X_test_arr), 200, replace = False)
X_interact : np.ndarray = X_test_arr[ sample_idx ]
shap_vals_interact : np.ndarray = explainer_local.shap_values(X_interact)

feat_1_idx : int = top_5_idx_urgent[ -1 ]
feat_2_idx : int = top_5_idx_urgent[ -2 ]

fig_shap, ax_shap = plt.subplots(figsize = ( 11, 7 ))
scatter = ax_shap.scatter(
    X_interact[ :, feat_1_idx ], shap_vals_interact[ :, feat_1_idx ],
    c = X_interact[ :, feat_2_idx ], cmap = "coolwarm", alpha = 0.8, edgecolors = "black", s = 60
)

cbar = plt.colorbar(scatter, ax = ax_shap)
cbar.set_label(f"Interaction : {feature_names_ko[feat_2_idx]}", fontsize = 11, fontweight = "bold")

max_idx : int = np.argmax(shap_vals_interact[ :, feat_1_idx ])
min_idx : int = np.argmin(shap_vals_interact[ :, feat_1_idx ])

ax_shap.annotate(
    f"Max SHAP : {shap_vals_interact[max_idx, feat_1_idx]:.3f}\n(Val : {X_interact[max_idx, feat_1_idx]:.1f})", 
    (X_interact[ max_idx, feat_1_idx ], shap_vals_interact[ max_idx, feat_1_idx ]),
    xytext = (10, -15), textcoords = "offset points", fontweight = "bold", color = "crimson",
    arrowprops = dict(arrowstyle = "->", color = "crimson")
)

ax_shap.annotate(
    f"Min SHAP : {shap_vals_interact[min_idx, feat_1_idx]:.3f}\n(Val : {X_interact[min_idx, feat_1_idx]:.1f})", 
    (X_interact[ min_idx, feat_1_idx ], shap_vals_interact[ min_idx, feat_1_idx ]),
    xytext = (-20, 20), textcoords = "offset points", fontweight = "bold", color = "#0047AB",
    arrowprops = dict(arrowstyle = "->", color = "#0047AB")
)

ax_shap.set_title(f"SHAP Interaction : {feature_names_ko[feat_1_idx]}", fontsize = 16, fontweight = "bold", pad = 20)
ax_shap.set_xlabel(f"Feature Value : {feature_names_ko[feat_1_idx]}", fontsize = 12)
ax_shap.set_ylabel("SHAP Value (Impact on Model Output)", fontsize = 12)
ax_shap.grid(alpha = 0.2)
plt.tight_layout()
force_fix_plot_tofu(fig_shap, ax_shap)
plt.show()

# 3. Global Surrogate Decision Tree
surrogate_tree : DecisionTreeClassifier = DecisionTreeClassifier(max_depth = 3, random_state = 42)
y_pred_surrogate : np.ndarray = (y_prob >= 0.3965).astype(int)
surrogate_tree.fit(X_test_arr, y_pred_surrogate)

fig_tree, ax_tree = plt.subplots(figsize = ( 20, 10 ))
plot_tree(
    surrogate_tree, 
    feature_names = feature_names_ko, 
    class_names = [ "Low_Risk", "High_Risk" ], 
    filled = True, rounded = True, ax = ax_tree, 
    fontsize = 12, impurity = False, proportion = True
)

ax_tree.set_title("Global Surrogate Decision Tree (Clinical Rule Extraction)", fontsize = 20, fontweight = "bold", pad = 20)
plt.tight_layout()
force_fix_plot_tofu(fig_tree, ax_tree)
plt.show()

# 35. Statistical Rigor and Clinical Error Profiling : 95% CI Bootstrapping & Misclassification Analysis

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_auc_score, average_precision_score
from scipy.stats import ttest_ind

# 1. Bootstrapping 95% Confidence Intervals
n_iterations : int = 1000
n_size : int = len(y_true_arr)
auc_scores : list = []
prc_scores : list = []

np.random.seed(42)
for i in range(n_iterations) :
    idx : np.ndarray = np.random.randint(0, n_size, n_size)
    y_true_boot : np.ndarray = y_true_arr[ idx ]
    y_prob_boot : np.ndarray = y_prob[ idx ]
    
    if len(np.unique(y_true_boot)) < 2 :
        continue
        
    auc_scores.append(roc_auc_score(y_true_boot, y_prob_boot))
    prc_scores.append(average_precision_score(y_true_boot, y_prob_boot))

auc_lower : float = np.percentile(auc_scores, 2.5)
auc_upper : float = np.percentile(auc_scores, 97.5)
prc_lower : float = np.percentile(prc_scores, 2.5)
prc_upper : float = np.percentile(prc_scores, 97.5)

fig_ci, axes_ci = plt.subplots(1, 2, figsize = ( 14, 6 ))

sns.histplot(auc_scores, bins = 30, kde = True, ax = axes_ci[ 0 ], color = "#0047AB", alpha = 0.6)
axes_ci[ 0 ].axvline(auc_lower, color = "crimson", ls = "--", lw = 2)
axes_ci[ 0 ].axvline(auc_upper, color = "crimson", ls = "--", lw = 2)
axes_ci[ 0 ].set_title(f"AUROC 95% CI : {np.mean(auc_scores):.4f}\n[{auc_lower:.4f} - {auc_upper:.4f}]", fontsize = 13, fontweight = "bold")

sns.histplot(prc_scores, bins = 30, kde = True, ax = axes_ci[ 1 ], color = "#E24A33", alpha = 0.6)
axes_ci[ 1 ].axvline(prc_lower, color = "black", ls = "--", lw = 2)
axes_ci[ 1 ].axvline(prc_upper, color = "black", ls = "--", lw = 2)
axes_ci[ 1 ].set_title(f"AUPRC 95% CI : {np.mean(prc_scores):.4f}\n[{prc_lower:.4f} - {prc_upper:.4f}]", fontsize = 13, fontweight = "bold")

plt.tight_layout()
if 'force_fix_plot_tofu' in locals():
    force_fix_plot_tofu(fig_ci, axes_ci[ 0 ])
plt.show()

# 2. Clinical Misclassification Profiling (Visualization Fix)
threshold : float = 0.3965
mask_tp : np.ndarray = (y_true_arr == 1) & (y_prob >= threshold)
mask_fn : np.ndarray = (y_true_arr == 1) & (y_prob < threshold)

tp_data : np.ndarray = X_test_arr[ mask_tp ]
fn_data : np.ndarray = X_test_arr[ mask_fn ]

profiling_records : list = []
for idx in top_5_idx_urgent :
    feat_name : str = feature_names_ko[ idx ]
    tp_vals : np.ndarray = tp_data[ :, idx ]
    fn_vals : np.ndarray = fn_data[ :, idx ]
    t_stat, p_val = ttest_ind(tp_vals, fn_vals, equal_var = False)
    profiling_records.append({
        "Feature" : feat_name,
        "TP_Mean" : np.mean(tp_vals),
        "FN_Mean" : np.mean(fn_vals),
        "P_Value" : p_val
    })

df_profile : pd.DataFrame = pd.DataFrame(profiling_records)

fig_prof, ax_prof = plt.subplots(figsize = ( 14, 7 ))
x_pos : np.ndarray = np.arange(len(top_5_idx_urgent))
width : float = 0.35

bar_tp = ax_prof.bar(x_pos - width/2, df_profile[ "TP_Mean" ], width, label = "True Positive (Detected)", color = "#0047AB", alpha = 0.85, edgecolor = "black")
bar_fn = ax_prof.bar(x_pos + width/2, df_profile[ "FN_Mean" ], width, label = "False Negative (Missed)", color = "crimson", alpha = 0.85, edgecolor = "black")

def label_bars(bars) :
    for bar in bars :
        height = bar.get_height()
        va = "bottom" if height >= 0 else "top"
        xytext = ( 0, 5 ) if height >= 0 else ( 0, -5 )
        ax_prof.annotate(f"{height:.3f}",
                         xy = (bar.get_x() + bar.get_width() / 2, height),
                         xytext = xytext, textcoords = "offset points",
                         ha = "center", va = va, fontsize = 10, fontweight = "bold")

label_bars(bar_tp)
label_bars(bar_fn)

for i in range(len(top_5_idx_urgent)) :
    pval : float = df_profile.loc[ i, "P_Value" ]
    significance : str = "***" if pval < 0.001 else "**" if pval < 0.01 else "*" if pval < 0.05 else "ns"
    max_h : float = max(df_profile.loc[ i, "TP_Mean" ], df_profile.loc[ i, "FN_Mean" ])
    ax_prof.text(x_pos[ i ], (max_h + 0.1) if max_h > 0 else 0.1, significance, ha = "center", va = "bottom", fontsize = 14, fontweight = "bold", color = "black")

# y축 상한선 확대 및 범례 위치 변경으로 수치 가림 현상 해결
y_min : float = df_profile[ [ "TP_Mean", "FN_Mean" ] ].values.min()
y_max : float = df_profile[ [ "TP_Mean", "FN_Mean" ] ].values.max()
ax_prof.set_ylim(y_min - 0.5, y_max + 0.8)  # y축 범위를 늘려 상단 데이터 라벨 가림 방지
ax_prof.legend(ncol = 2, loc = 'upper center', bbox_to_anchor = ( 0.5, -0.1 )) # 범례를 그래프 밖 하단으로 이동

# 통계적 유의성 범례 위치는 하단 왼쪽으로 이동하여 막대와 겹침 방지
p_val_legend : str = (
    "[ Statistical Significance ]\n"
    "*** : $p < 0.001$ (Very Strong Evidence)\n"
    "** : $p < 0.01$ (Strong Evidence)\n"
    "* : $p < 0.05$ (Significant)\n"
    "ns  : $p \geq 0.05$ (Not Significant)"
)
ax_prof.text(0.02, 0.05, p_val_legend, transform = ax_prof.transAxes, fontsize = 10, 
             fontweight = "bold", va = "bottom", bbox = dict(boxstyle = "round", facecolor = "white", alpha = 0.9))

ax_prof.axhline(0, color = "black", lw = 1)
ax_prof.set_xticks(x_pos)
ax_prof.set_xticklabels(df_profile[ "Feature" ], rotation = 15, fontsize = 11, fontweight = "bold")
ax_prof.set_title("Misclassification Profiling : Feature Mean Comparison (Top 5)", fontsize = 16, fontweight = "bold", pad = 20)
ax_prof.set_ylabel("Mean Feature Value (Z-score)", fontsize = 12)
ax_prof.grid(axis = "y", ls = "--", alpha = 0.3)

plt.tight_layout()
if 'force_fix_plot_tofu' in locals():
    force_fix_plot_tofu(fig_prof, ax_prof)
plt.show()

# 36. Model Deployment : ONNX Conversion & Latency Benchmark

In [ ]:
import torch
import torch.onnx
import onnx
import onnxruntime as ort
import numpy as np
import time
from pathlib import Path

# 1. 경로 및 디바이스 설정
base_dir : str = r"C:\Users\sck32\Desktop\Clinical Diabetes TabNet"
export_path : Path = Path(base_dir) / "metabolic_ai_models"
export_path.mkdir(parents = True, exist_ok = True)
onnx_file_path : str = str(export_path / "tabnet_high_risk_model.onnx")

device : str = "cuda" if torch.cuda.is_available() else "cpu"
clf.network.to(device)
clf.network.eval()

# 2. ONNX 변환을 위한 Dummy Input 생성
dummy_input : torch.Tensor = torch.tensor(X_test_arr[ :1 ], dtype = torch.float32).to(device)

# 3. ONNX Export
try :
    torch.onnx.export(
        clf.network, 
        dummy_input, 
        onnx_file_path,
        export_params = True,
        opset_version = 13,
        do_constant_folding = True,
        input_names = [ "input" ],
        output_names = [ "output" ],
        dynamic_axes = { "input" : { 0 : "batch_size" }, "output" : { 0 : "batch_size" } }
    )
    
    # ONNX 모델 무결성 검증
    onnx_model = onnx.load(onnx_file_path)
    onnx.checker.check_model(onnx_model)
    print(f"✅ ONNX 변환 및 무결성 검증 완료 : {onnx_file_path}")

except Exception as e :
    print(f"❌ ONNX 변환 실패 : {e}")

# 4. Latency Benchmark (PyTorch vs ONNX)
n_iters : int = 1000
test_tensor : torch.Tensor = torch.tensor(X_test_arr[ :128 ], dtype = torch.float32).to(device)
test_numpy : np.ndarray = test_tensor.cpu().numpy()

# PyTorch Latency
start_time = time.perf_counter()
with torch.no_grad() :
    for _ in range(n_iters) :
        _ = clf.network(test_tensor)
pytorch_latency : float = (time.perf_counter() - start_time) / n_iters * 1000

# ONNX Runtime Latency (CPU Provider for safe universal deployment)
ort_session = ort.InferenceSession(onnx_file_path, providers = [ "CPUExecutionProvider" ])
start_time = time.perf_counter()
for _ in range(n_iters) :
    _ = ort_session.run(None, { "input" : test_numpy })
onnx_latency : float = (time.perf_counter() - start_time) / n_iters * 1000

print("-" * 50)
print(f"⚡ Inference Latency Benchmark (Batch Size = 128)")
print(f" - PyTorch ({device}) : {pytorch_latency:.3f} ms / batch")
print(f" - ONNX (CPU)   : {onnx_latency:.3f} ms / batch")
print("-" * 50)

# 37. Clinical Fairness Validation : Subgroup Disparity Analysis

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

age_idx : int = valid_features.index("age")
sex_idx : int = valid_features.index("sex")

age_median : float = np.median(X_test_arr[ :, age_idx ])

groups : dict = {
    "Male" : X_test_arr[ :, sex_idx ] == 1,
    "Female" : X_test_arr[ :, sex_idx ] == 2,
    "Age <= Median" : X_test_arr[ :, age_idx ] <= age_median,
    "Age > Median" : X_test_arr[ :, age_idx ] > age_median
}

opt_threshold : float = 0.0817 
y_pred_opt : np.ndarray = (y_prob >= opt_threshold).astype(int)

fairness_records : list = []

for group_name, mask in groups.items() :
    if np.sum(mask) == 0 :
        continue
        
    y_true_g : np.ndarray = y_true_arr[ mask ]
    y_pred_g : np.ndarray = y_pred_opt[ mask ]
    
    tn, fp, fn, tp = confusion_matrix(y_true_g, y_pred_g, labels = [ 0, 1 ]).ravel()
    
    tpr : float = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    fpr : float = fp / (fp + tn) if (fp + tn) > 0 else 0.0
    
    fairness_records.append({
        "Subgroup" : group_name,
        "TPR (Sensitivity)" : tpr,
        "FPR (False Alarm)" : fpr,
        "N" : np.sum(mask)
    })

df_fairness : pd.DataFrame = pd.DataFrame(fairness_records)

fig_fair, ax_fair = plt.subplots(figsize = ( 12, 6 ))
x_pos : np.ndarray = np.arange(len(df_fairness))
width : float = 0.35

bar_tpr = ax_fair.bar(x_pos - width/2, df_fairness[ "TPR (Sensitivity)" ], width, label = "TPR (True Positive Rate)", color = "#0047AB", edgecolor = "black")
bar_fpr = ax_fair.bar(x_pos + width/2, df_fairness[ "FPR (False Alarm)" ], width, label = "FPR (False Positive Rate)", color = "crimson", edgecolor = "black")

def annotate_bars(bars) :
    for bar in bars :
        height = bar.get_height()
        ax_fair.annotate(f"{height:.3f}",
                         xy = (bar.get_x() + bar.get_width() / 2, height),
                         xytext = (0, 3), textcoords = "offset points",
                         ha = "center", va = "bottom", fontsize = 10, fontweight = "bold")

annotate_bars(bar_tpr)
annotate_bars(bar_fpr)

ax_fair.set_xticks(x_pos)
ax_fair.set_xticklabels([ f"{row['Subgroup']}\n(N={row['N']})" for _, row in df_fairness.iterrows() ], fontsize = 11, fontweight = "bold")
ax_fair.set_title("Model Fairness Validation (Equal Opportunity & Predictive Parity)", fontsize = 16, fontweight = "bold", pad = 20)
ax_fair.set_ylabel("Rate", fontsize = 12)
ax_fair.set_ylim(0, 1.15)
ax_fair.legend(loc = "upper right")
ax_fair.grid(axis = "y", ls = "--", alpha = 0.3)

plt.tight_layout()
if 'force_fix_plot_tofu' in locals() :
    force_fix_plot_tofu(fig_fair, ax_fair)
plt.show()

# 38. Clinical Utility Validation : Non-Invasive Predictive Modeling

In [ ]:
import numpy as np
import pandas as pd
import torch
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.metrics import roc_auc_score, confusion_matrix
import matplotlib.pyplot as plt

# 타겟 및 학습 데이터 매핑
X_train_arr : np.ndarray = X_train if isinstance(X_train, np.ndarray) else X_train.values
X_test_arr : np.ndarray = X_test if isinstance(X_test, np.ndarray) else X_test.values
y_train_arr : np.ndarray = y_train if isinstance(y_train, np.ndarray) else y_train.values
y_true_arr : np.ndarray = y_test if isinstance(y_test, np.ndarray) else y_test.values

# 데이터 누수 지표 제거 및 피처 인덱싱
leakage_features : list = [ "HE_glu", "HE_HbA1c" ] 
reduced_features : list = [ f for f in valid_features if f not in leakage_features ]

reduced_idx : list = [ valid_features.index(f) for f in reduced_features ]
X_train_red : np.ndarray = X_train_arr[ :, reduced_idx ]
X_test_red : np.ndarray = X_test_arr[ :, reduced_idx ]

device : str = "cuda" if torch.cuda.is_available() else "cpu"

clf_ni = TabNetClassifier(
    n_d = 16, 
    n_a = 16,
    optimizer_fn = torch.optim.Adam,
    optimizer_params = { "lr" : 2e-2 },
    scheduler_params = { "step_size" : 10, "gamma" : 0.9 },
    scheduler_fn = torch.optim.lr_scheduler.StepLR,
    mask_type = "entmax",
    device_name = device
)

print("-" * 50)
print("🚀 비침습적 임상 지표 기반 TabNet 재학습 시작 (Data Leakage 배제)")

clf_ni.fit(
    X_train = X_train_red, y_train = y_train_arr,
    eval_set = [ (X_train_red, y_train_arr), (X_test_red, y_true_arr) ],
    eval_name = [ "train", "valid" ],
    eval_metric = [ "auc" ],
    max_epochs = 200,
    patience = 20,
    batch_size = 1024,
    virtual_batch_size = 128
)

preds_prob_ni : np.ndarray = clf_ni.predict_proba(X_test_red)[ :, 1 ]
auc_ni : float = roc_auc_score(y_true_arr, preds_prob_ni)

print(f"✅ Non-invasive Model Test AUC : {auc_ni:.4f}")
print("-" * 50)

# 39. Cost Sensitive Optimization : Asymmetric Cost Matrix Application

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

# 비용 가중치 설정 (FN : 5.0, FP : 1.0)
cost_FN : float = 5.0 
cost_FP : float = 1.0  

thresholds : np.ndarray = np.linspace(0.01, 0.99, 100)
costs : list = []

for t in thresholds :
    preds_t : np.ndarray = (preds_prob_ni >= t).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_true_arr, preds_t).ravel()
    total_cost : float = (fn * cost_FN) + (fp * cost_FP)
    expected_cost : float = total_cost / len(y_true_arr)
    costs.append(expected_cost)

opt_idx : int = np.argmin(costs)
opt_thresh_cost : float = thresholds[ opt_idx ]
min_cost : float = costs[ opt_idx ]

fig_cost, ax_cost = plt.subplots(figsize = ( 10, 6 ))

ax_cost.plot(thresholds, costs, color = "#0047AB", lw = 3, label = "Expected Cost Curve")
ax_cost.scatter([ opt_thresh_cost ], [ min_cost ], color = "crimson", s = 100, zorder = 5)
ax_cost.annotate(
    f"Optimal Threshold : {opt_thresh_cost:.4f}\nMin Expected Cost : {min_cost:.4f}",
    xy = (opt_thresh_cost, min_cost),
    xytext = (opt_thresh_cost + 0.05, min_cost + 0.05),
    arrowprops = dict(facecolor = "black", shrink = 0.05, width = 1, headwidth = 8),
    fontsize = 11, fontweight = "bold",
    bbox = dict(boxstyle = "round,pad=0.5", fc = "white", ec = "crimson", alpha = 0.9)
)

ax_cost.set_title("Cost-Sensitive Threshold Optimization (FN : 5, FP : 1)", fontsize = 16, fontweight = "bold", pad = 20)
ax_cost.set_xlabel("Decision Threshold", fontsize = 12)
ax_cost.set_ylabel("Expected Cost per Patient", fontsize = 12)
ax_cost.grid(alpha = 0.3, ls = "--")
ax_cost.legend(loc = "upper right")
plt.tight_layout()

if 'force_fix_plot_tofu' in locals() :
    force_fix_plot_tofu(fig_cost, ax_cost)
plt.show()

preds_final_opt : np.ndarray = (preds_prob_ni >= opt_thresh_cost).astype(int)
tn, fp, fn, tp = confusion_matrix(y_true_arr, preds_final_opt).ravel()
sensitivity : float = tp / (tp + fn) if (tp + fn) > 0 else 0.0
specificity : float = tn / (tn + fp) if (tn + fp) > 0 else 0.0

print(f"📊 Cost-Optimized Metrics (Threshold : {opt_thresh_cost:.4f})")
print(f" - Sensitivity (Recall) : {sensitivity:.4f}")
print(f" - Specificity : {specificity:.4f}")
print(f" - False Negatives (Missed) : {fn}")
print(f" - False Positives (Over-diagnosed) : {fp}")

# 40. Non-Invasive Feature Interpretation (SHAP)

In [ ]:
import shap
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as mcolors

X_sample : np.ndarray = X_test_red[ :200 ]
explainer_ni = shap.KernelExplainer(
    lambda x : clf_ni.predict_proba(x.astype(np.float32))[ :, 1 ], 
    shap.sample(X_train_red, 50)
)
shap_values_ni : np.ndarray = explainer_ni.shap_values(X_sample)

mean_abs_shap : np.ndarray = np.abs(shap_values_ni).mean(axis = 0)
df_shap : pd.DataFrame = pd.DataFrame({
    "Feature" : reduced_features,
    "Importance" : mean_abs_shap
}).sort_values(by = "Importance", ascending = True)

# 기여도 크기에 비례하는 그라데이션 색상 맵(Colormap) 적용
cmap = cm.get_cmap("viridis")
norm = mcolors.Normalize(vmin = df_shap[ "Importance" ].min(), vmax = df_shap[ "Importance" ].max())
colors = cmap(norm(df_shap[ "Importance" ].values))

fig_shap_bar, ax_shap_bar = plt.subplots(figsize = ( 12, 10 ))
bars = ax_shap_bar.barh(df_shap[ "Feature" ], df_shap[ "Importance" ], color = colors, edgecolor = "black", alpha = 0.85)

for bar in bars :
    width : float = bar.get_width()
    ax_shap_bar.text(
        width + (max(mean_abs_shap) * 0.01), 
        bar.get_y() + bar.get_height() / 2, 
        f"{width:.4f}", 
        va = "center", 
        fontsize = 10, 
        fontweight = "bold"
    )

ax_shap_bar.set_title("Non-Invasive Model : Feature Importance (Mean |SHAP|)", fontsize = 16, fontweight = "bold", pad = 20)
ax_shap_bar.set_xlabel("mean(|SHAP value|)", fontsize = 12)
ax_shap_bar.grid(axis = "x", ls = "--", alpha = 0.3)
ax_shap_bar.set_xlim(0, max(mean_abs_shap) * 1.15)

plt.tight_layout()
if 'force_fix_plot_tofu' in locals() :
    force_fix_plot_tofu(fig_shap_bar, ax_shap_bar)
plt.show()

plt.figure(figsize = ( 12, 8 ))
shap.summary_plot(
    shap_values_ni, 
    X_sample, 
    feature_names = reduced_features, 
    show = False
)
plt.title("Feature Impact Directionality (Non-Invasive)", fontsize = 16, fontweight = "bold", pad = 20)
plt.tight_layout()

if 'force_fix_plot_tofu' in locals() :
    force_fix_plot_tofu(plt.gcf(), plt.gca())
plt.show()

# 41. Prediction Dataset Export

In [ ]:
import pandas as pd
import os

# 앞서 설정한 비용 최적화 임계값 적용
opt_thresh_cost : float = 0.3268

# 최종 예측 결과를 담은 데이터프레임 구성
final_results : pd.DataFrame = pd.DataFrame({
    "True_Metabolic_Risk" : y_true_arr,
    "Non_Invasive_Prob" : preds_prob_ni,
    "AI_Diagnosis_Result" : (preds_prob_ni >= opt_thresh_cost).astype(int)
})

# 진단 결과 라벨링 (1: 고위험군, 0: 정상)
final_results[ "AI_Diagnosis_Label" ] = final_results[ "AI_Diagnosis_Result" ].map({ 1 : "High-Risk", 0 : "Safe" })

# 바탕화면 지정 경로로 CSV 익스포트
base_dir : str = r"C:\Users\sck32\Desktop\Clinical Diabetes TabNet"
file_path : str = f"{base_dir}\\final_non_invasive_predictions.csv"

try :
    final_results.to_csv(file_path, index = False, encoding = "utf-8-sig")
    print("-" * 50)
    print(f"✅ 최종 예측 데이터셋 저장 완료")
    print(f"📂 경로 : {file_path}")
    print(f"📊 총 추출된 환자 데이터 수 : {len(final_results):,}명")
    print("-" * 50)
except Exception as e :
    print(f"❌ 데이터 저장 실패 : {e}")

# 42. DeepSurv : Deep Neural Networks for Survival Analysis

## 42-1. 생존 분석용 데이터(Time, Event) 텐서화 및 데이터로더 구축

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
import time

device : str = "cuda" if torch.cuda.is_available() else "cpu"

# 실제 추적 관찰 데이터가 부재하므로, 시연용 추적 시간(Time) 가상 생성 (단위 : Month)
np.random.seed(42)
time_train : np.ndarray = np.where(y_train_arr == 1, np.random.exponential(scale = 24, size = len(y_train_arr)), np.random.exponential(scale = 120, size = len(y_train_arr)))
time_test : np.ndarray = np.where(y_true_arr == 1, np.random.exponential(scale = 24, size = len(y_true_arr)), np.random.exponential(scale = 120, size = len(y_true_arr)))

time_train_t : torch.Tensor = torch.tensor(time_train, dtype = torch.float32)
event_train_t : torch.Tensor = torch.tensor(y_train_arr, dtype = torch.float32)
x_train_t : torch.Tensor = torch.tensor(X_train_red, dtype = torch.float32)

time_test_t : torch.Tensor = torch.tensor(time_test, dtype = torch.float32)
event_test_t : torch.Tensor = torch.tensor(y_true_arr, dtype = torch.float32)
x_test_t : torch.Tensor = torch.tensor(X_test_red, dtype = torch.float32)

# VRAM 12GB 환경을 고려한 Batch Size 설정
batch_size : int = 2048

train_dataset = TensorDataset(x_train_t, time_train_t, event_train_t)
train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True, drop_last = False)

## 42-2. DeepSurv 모델 정의 및 Negative Partial Log-Likelihood 손실 함수 구현

In [ ]:
class DeepSurv(nn.Module) :
    def __init__(self, in_features : int) :
        super(DeepSurv, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(in_features, 64),
            nn.SELU(),
            nn.Dropout(0.2),
            nn.Linear(64, 32),
            nn.SELU(),
            nn.Dropout(0.2),
            nn.Linear(32, 1)
        )
        
    def forward(self, x : torch.Tensor) -> torch.Tensor :
        return self.net(x)

def cox_ph_loss(risk_pred : torch.Tensor, time : torch.Tensor, event : torch.Tensor) -> torch.Tensor :
    # Risk sorting (내림차순 정렬)
    sorted_idx : torch.Tensor = torch.argsort(time, descending = True)
    risk_pred_sorted : torch.Tensor = risk_pred[ sorted_idx ]
    event_sorted : torch.Tensor = event[ sorted_idx ]
    
    # 누적 위험도 (Risk Set 계산)
    risk_exp : torch.Tensor = torch.exp(risk_pred_sorted)
    risk_sum : torch.Tensor = torch.cumsum(risk_exp, dim = 0)
    
    # Log 부분 우도 계산
    log_risk : torch.Tensor = torch.log(risk_sum + 1e-7)
    uncensored_likelihood : torch.Tensor = risk_pred_sorted - log_risk
    loss : torch.Tensor = -torch.sum(uncensored_likelihood * event_sorted) / (torch.sum(event_sorted) + 1e-7)
    
    return loss

# 43. Longitudinal Survival Analysis : DeepSurv Training & Evaluation

In [ ]:
from lifelines.utils import concordance_index

dim_in : int = X_train_red.shape[ 1 ]
model = DeepSurv(in_features = dim_in).to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr = 1e-3, weight_decay = 1e-4)
scaler = torch.cuda.amp.GradScaler()

epochs : int = 150
print("-" * 50)
print("🚀 DeepSurv 생존 분석 모델 학습 시작 (FP16 AMP)")

model.train()
for epoch in range(epochs) :
    total_loss : float = 0.0
    
    for batch_x, batch_time, batch_event in train_loader :
        batch_x = batch_x.to(device)
        batch_time = batch_time.to(device)
        batch_event = batch_event.to(device)
        
        optimizer.zero_grad()
        
        with torch.cuda.amp.autocast() :
            risk_pred : torch.Tensor = model(batch_x)
            loss : torch.Tensor = cox_ph_loss(risk_pred, batch_time, batch_event)
        
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        total_loss += loss.item()
        
    if (epoch + 1) % 30 == 0 or epoch == 0 :
        avg_loss : float = total_loss / len(train_loader)
        print(f"Epoch {epoch + 1:03d}/{epochs} | Cox Loss : {avg_loss:.4f}")

print("-" * 50)

model.eval()
with torch.no_grad() :
    x_test_cuda : torch.Tensor = x_test_t.to(device)
    risk_pred_test : np.ndarray = model(x_test_cuda).cpu().numpy().flatten()

c_index : float = concordance_index(time_test, -risk_pred_test, y_true_arr)

print(f"✅ DeepSurv Test C-Index : {c_index:.4f}")
print("-" * 50)

# 44. Survival Curve Visualization : Kaplan Meier Estimation

In [ ]:
import matplotlib.pyplot as plt
from lifelines import KaplanMeierFitter
import numpy as np

median_risk : float = np.median(risk_pred_test)
high_risk_mask : np.ndarray = risk_pred_test >= median_risk
low_risk_mask : np.ndarray = risk_pred_test < median_risk

kmf_high = KaplanMeierFitter()
kmf_low = KaplanMeierFitter()

fig_km, ax_km = plt.subplots(figsize = ( 12, 7 ))

kmf_high.fit(time_test[ high_risk_mask ], event_observed = y_true_arr[ high_risk_mask ], label = "High Risk Group")
kmf_high.plot_survival_function(ax = ax_km, color = "crimson", lw = 2.5)

kmf_low.fit(time_test[ low_risk_mask ], event_observed = y_true_arr[ low_risk_mask ], label = "Low Risk Group")
kmf_low.plot_survival_function(ax = ax_km, color = "#0047AB", lw = 2.5)

# 주요 시점(Time Points)별 생존 확률 수치
test_times : list = [ 0, 200, 400, 600, 800 ]
for t in test_times :
    # High Risk Group 수치 표시
    surv_prob_h : float = kmf_high.predict(t)
    ax_km.text(t, surv_prob_h - 0.03, f"{surv_prob_h:.3f}", color = "crimson", fontsize = 10, fontweight = "bold", ha = "center")
    
    # Low Risk Group 수치 표시
    surv_prob_l : float = kmf_low.predict(t)
    ax_km.text(t, surv_prob_l + 0.01, f"{surv_prob_l:.3f}", color = "#0047AB", fontsize = 10, fontweight = "bold", ha = "center")

ax_km.set_title("Kaplan-Meier Survival Curve with Probability Labels", fontsize = 16, fontweight = "bold", pad = 20)
ax_km.set_xlabel("Time (Months)", fontsize = 12)
ax_km.set_ylabel("Survival Probability", fontsize = 12)
ax_km.set_ylim(0.65, 1.05)
ax_km.grid(alpha = 0.3, ls = "--")
ax_km.legend(loc = "lower left", fontsize = 11)

plt.tight_layout()
if 'force_fix_plot_tofu' in locals() :
    force_fix_plot_tofu(fig_km, ax_km)
plt.show()

# 45. Individual Diagnostic Support : SHAP Waterfall Plot

In [ ]:
import shap
import matplotlib.pyplot as plt

# 한글 폰트 설정 (Windows : Malgun Gothic)
plt.rcParams[ "font.family" ] = "Malgun Gothic"
plt.rcParams[ "axes.unicode_minus" ] = False

# 피처 영문 -국문 매핑 사전
feature_mapping : dict = {
    "sm_presnt" : "현재 흡연 여부",
    "HE_Upro" : "요단백",
    "HE_hepaB" : "B형간염 양성",
    "age" : "연령",
    "HE_chol" : "총 콜레스테롤",
    "HE_wc" : "허리둘레",
    "HE_BUN" : "혈중요소질소",
    "HE_crea" : "혈중 크레아티닌",
    "HE_TG" : "중성지방",
    "HE_alt" : "ALT (간수치)",
    "N_K" : "칼륨 섭취량",
    "HE_hepaC" : "C형간염 양성",
    "N_FAT" : "지방 섭취량",
    "HE_WBC" : "백혈구 수",
    "HE_ast" : "AST (간수치)",
    "N_PROT" : "단백질 섭취량",
    "sex" : "성별",
    "HE_BMI" : "체질량지수 (BMI)",
    "HE_dbp" : "이완기 혈압",
    "N_CHO" : "탄수화물 섭취량",
    "N_NA" : "나트륨 섭취량",
    "HE_sbp" : "수축기 혈압",
    "N_EN" : "에너지 섭취량",
    "HE_HDL_st2" : "HDL 콜레스테롤"
}

# 현재 모델에서 사용하는 피처 순서에 맞춰 한글 이름 리스트 생성
ko_features : list = [ feature_mapping.get(f, f) for f in reduced_features ]

patient_idx : int = 0
sample_patient : np.ndarray = X_test_red[ patient_idx : patient_idx + 1 ]

exp_value : np.ndarray = explainer_ni.shap_values(sample_patient)
base_value : float = explainer_ni.expected_value

plt.figure(figsize = ( 12, 8 ))
shap.plots._waterfall.waterfall_legacy(
    base_value,
    exp_value[ 0 ],
    feature_names = ko_features,
    max_display = 10,
    show = False
)

plt.title(f"환자 번호 {patient_idx} : AI 진단 근거 상세 분석", fontsize = 16, fontweight = "bold", pad = 25)
plt.xlabel("SHAP Value (위험도 기여도)", fontsize = 12)
plt.tight_layout()

if 'force_fix_plot_tofu' in locals() :
    force_fix_plot_tofu(plt.gcf(), plt.gca())
plt.show()

# 46. Integrated Diagnostics : Unified Clinical Summary Generator

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import shap
import torch

plt.rcParams[ "font.family" ] = "Malgun Gothic"
plt.rcParams[ "axes.unicode_minus" ] = False

def generate_robust_clinical_report(patient_idx, x_data, model_clf, model_surv, explainer, features, threshold) :
    if patient_idx >= len(x_data) :
        print(f"Error : 입력한 환자 번호({patient_idx})가 전체 데이터 크기({len(x_data)})를 초과했습니다.")
        return

    sample : np.ndarray = x_data[ patient_idx : patient_idx + 1 ]
    
    prob : float = model_clf.predict_proba(sample.astype(np.float32))[ 0, 1 ]
    diag : str = "고위험군" if prob >= threshold else "정상"
    
    local_device = next(model_surv.parameters()).device
    model_surv.eval()
    with torch.no_grad() :
        risk_score : float = model_surv(torch.tensor(sample, dtype = torch.float32).to(local_device)).cpu().item()
    
    print("=" * 70)
    print(f"🩺 [ 통합 임상 진단 리포트 ] 환자 번호 : {patient_idx}")
    print("=" * 70)

    # ---------------------------------------------------------
    # [1/3] 진단 확률 차트
    # ---------------------------------------------------------
    fig1, ax1 = plt.subplots(figsize = ( 12, 3 ))
    color : str = "crimson" if diag == "고위험군" else "#0047AB"
    
    bar_val : float = prob if prob > 0 else 0.001 
    ax1.barh([ "대사 질환 위험도" ], [ bar_val ], color = color, height = 0.4, alpha = 0.8)
    ax1.axvline(threshold, color = "black", ls = "--", lw = 2, label = f"임계값 ({threshold})")
    ax1.set_xlim(0, 1)
    ax1.set_title(f"[1/3] 최종 진단 결과 : {diag}", fontsize = 16, fontweight = "bold", pad = 15)
    ax1.legend(loc = "center right", fontsize = 12)
    ax1.grid(axis = "x", alpha = 0.3, ls = "--")
    
    ax1.text(prob + 0.015, 0, f"{prob:.4f}", va = "center", ha = "left", fontsize = 14, fontweight = "bold", color = color)
    
    plt.tight_layout()
    if 'force_fix_plot_tofu' in locals() :
        force_fix_plot_tofu(fig1, ax1)
    plt.show()

    # ---------------------------------------------------------
    # [2/3] SHAP Waterfall 차트
    # ---------------------------------------------------------
    shap_vals : np.ndarray = explainer.shap_values(sample)
    ko_features : list = [ feature_mapping.get(f, f) for f in features ]
    base_val : float = explainer.expected_value[ 0 ] if isinstance(explainer.expected_value, np.ndarray) else explainer.expected_value
    
    fig2 = plt.figure(figsize = ( 12, 6 ))
    shap.plots._waterfall.waterfall_legacy(
        explainer.expected_value,
        shap_vals[ 0 ],
        feature_names = ko_features,
        max_display = 10,
        show = False
    )
    plt.title("[2/3] 인공지능 진단 근거 상세 (SHAP)", fontsize = 16, fontweight = "bold", pad = 25)
    
    guide_text : str = f"※ 해석 가이드\n출발 기준점 : 전체 환자 평균 위험도 ({base_val:.3f})\n막대 수치 : 기준점 대비 해당 지표의 '위험도 증감폭'"
    plt.figtext(0.01, 0.85, guide_text, fontsize = 10, color = "#4B4B4B", bbox = dict(facecolor = "whitesmoke", edgecolor = "lightgrey", boxstyle = "round,pad=0.5"))

    for text_obj in plt.gcf().findobj(mpl.text.Text) :
        text_val : str = text_obj.get_text()
        if "\u2212" in text_val or "−" in text_val :
            text_obj.set_text(text_val.replace("\u2212", "-").replace("−", "-"))
        
    plt.tight_layout(rect = [ 0, 0, 1, 0.95 ])
    if 'force_fix_plot_tofu' in locals() :
        force_fix_plot_tofu(plt.gcf(), plt.gca())
    plt.show()

    # ---------------------------------------------------------
    # [3/3] 정밀 생존 곡선 
    # ---------------------------------------------------------
    fig3, ax3 = plt.subplots(figsize = ( 12, 6 ))
    times : np.ndarray = np.sort(time_test)
    base_surv : np.ndarray = np.array([ kmf_low.predict(t) for t in times ])
    high_surv : np.ndarray = np.array([ kmf_high.predict(t) for t in times ])
    
    patient_surv : np.ndarray = (1.0 - prob) * base_surv + prob * high_surv
    
    ax3.plot(times, base_surv, color = "#0047AB", ls = "--", alpha = 0.4, label = "저위험군 평균")
    ax3.plot(times, high_surv, color = "crimson", ls = "--", alpha = 0.4, label = "고위험군 평균")
    ax3.plot(times, patient_surv, color = "black", lw = 3.5, label = "해당 환자 예상 경로")
    
    final_time : float = times[ -1 ]
    p_surv_final : float = patient_surv[ -1 ]
    surv_h_final : float = high_surv[ -1 ]
    surv_l_final : float = base_surv[ -1 ]

    offset_p : float = 0.0
    offset_l : float = 0.0
    offset_h : float = 0.0
    
    if abs(p_surv_final - surv_l_final) < 0.02 :
        offset_p = 0.01 if p_surv_final >= surv_l_final else -0.01
        offset_l = -0.01 if p_surv_final >= surv_l_final else 0.01
    elif abs(p_surv_final - surv_h_final) < 0.02 :
        offset_p = 0.01 if p_surv_final >= surv_h_final else -0.01
        offset_h = -0.01 if p_surv_final >= surv_h_final else 0.01
            
    ax3.text(final_time, p_surv_final + offset_p, f"  {p_surv_final:.3f}", color = "black", fontsize = 12, fontweight = "bold", va = "center")
    ax3.text(final_time, surv_h_final + offset_h, f"  {surv_h_final:.3f}", color = "crimson", fontsize = 10, va = "center")
    ax3.text(final_time, surv_l_final + offset_l, f"  {surv_l_final:.3f}", color = "#0047AB", fontsize = 10, va = "center")

    ax3.set_title("[3/3] 예상 무병 생존 경로 (Cox PH)", fontsize = 16, fontweight = "bold", pad = 15)
    ax3.set_xlabel("시간 (개월)", fontsize = 12)
    ax3.set_ylabel("발병 안 할 확률", fontsize = 12)
    
    min_surv : float = min(p_surv_final, surv_h_final, surv_l_final)
    y_lower : float = max(0.0, min_surv - 0.10) 
    
    ax3.set_xlim(left = 0, right = final_time * 1.08) 
    ax3.set_ylim(y_lower, 1.05)
    ax3.legend(loc = "lower left", fontsize = 12)
    ax3.grid(alpha = 0.3, ls = "--")
    
    plt.tight_layout()
    if 'force_fix_plot_tofu' in locals() :
        force_fix_plot_tofu(fig3, ax3)
    plt.show()

## 46-1 Highest Risk Patient Report

In [ ]:
# 1. 최고 위험군 (High Risk) 환자 자동 탐색
high_risk_idx : int = int(np.argmax(all_probs))

print(f"💡 테스트 셋 내 최고 위험군 환자 인덱스 : {high_risk_idx} (예측 확률 : {all_probs[high_risk_idx]:.4f})")

# 리포트 자동 생성
generate_robust_clinical_report(
    patient_idx = high_risk_idx,
    x_data = X_test_red,
    model_clf = clf_ni,
    model_surv = model,
    explainer = explainer_ni,
    features = reduced_features,
    threshold = 0.3268
)

## 46-2 Median Risk Patient Report

In [ ]:
# 1. 중간 위험군 (Medium Risk / Borderline) 환자 탐색
target_prob : float = 0.50
medium_risk_idx : int = int(np.argmin(np.abs(all_probs - target_prob)))

print(f"💡 테스트 셋 내 중간 위험군 환자 인덱스 : {medium_risk_idx} (예측 확률 : {all_probs[medium_risk_idx]:.4f})")

# 리포트 자동 생성
generate_robust_clinical_report(
    patient_idx = medium_risk_idx,
    x_data = X_test_red,
    model_clf = clf_ni,
    model_surv = model,
    explainer = explainer_ni,
    features = reduced_features,
    threshold = 0.3268
)

## 46-3 Lowest Risk Patient Report

In [ ]:
# 1. 최저 위험군 (Lowest Risk) 환자 자동 탐색
low_risk_idx : int = int(np.argmin(all_probs))

print(f"💡 테스트 셋 내 최저 위험군 환자 인덱스 : {low_risk_idx} (예측 확률 : {all_probs[low_risk_idx]:.4f})")

# 2. 리포트 생성 함수 호출 (짧은 코드)
generate_robust_clinical_report(
    patient_idx = low_risk_idx,
    x_data = X_test_red,
    model_clf = clf_ni,
    model_surv = model,
    explainer = explainer_ni,
    features = reduced_features,
    threshold = 0.3268
)

# 47. Time Dependent ROC & AUC Analysis

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

plt.rcParams[ "font.family" ] = "Malgun Gothic"
plt.rcParams[ "axes.unicode_minus" ] = False

# 평가 시점 설정 : 24개월(2년), 60개월(5년), 120개월(10년)
eval_times : list = [ 24, 60, 120 ] 

fig_roc, ax_roc = plt.subplots(figsize = ( 8, 8 ))
colors : list = [ "#0047AB", "crimson", "forestgreen" ]

for idx, t in enumerate(eval_times) :
    binary_labels : np.ndarray = ((time_test <= t) & (y_true_arr == 1)).astype(int)
    
    if np.sum(binary_labels) > 0 and np.sum(binary_labels) < len(binary_labels) :
        fpr, tpr, _ = roc_curve(binary_labels, risk_pred_test)
        roc_auc : float = auc(fpr, tpr)
        
        ax_roc.plot(fpr, tpr, color = colors[ idx ], lw = 2.5, label = f"{t}개월 예측 (AUC : {roc_auc:.4f})")

ax_roc.plot([ 0, 1 ], [ 0, 1 ], color = "black", ls = "--", lw = 1.5, alpha = 0.5, label = "무작위 예측 (AUC : 0.5000)")

ax_roc.set_title("시간 의존적 ROC 곡선 (Time-Dependent ROC)", fontsize = 16, fontweight = "bold", pad = 20)
ax_roc.set_xlabel("위양성률 (False Positive Rate)", fontsize = 12)
ax_roc.set_ylabel("민감도 (True Positive Rate)", fontsize = 12)
ax_roc.legend(loc = "lower right", fontsize = 12)
ax_roc.grid(alpha = 0.3, ls = "--")

plt.tight_layout()
if 'force_fix_plot_tofu' in locals() :
    force_fix_plot_tofu(fig_roc, ax_roc)
plt.show()

# 48. Project Finalization : Model Serialization

In [ ]:
import os
import torch

save_dir : str = "./final_models"
os.makedirs(save_dir, exist_ok = True)

# 1. TabNet 모델 저장 (zip 압축 형태로 저장됨)
tabnet_save_path : str = os.path.join(save_dir, "tabnet_metabolic_model")
saved_filepath : str = clf_ni.save_model(tabnet_save_path)
print(f"✅ TabNet 모델 저장 완료 : {saved_filepath}")

# 2. DeepSurv 모델 가중치 저장 (PyTorch State Dict)
deepsurv_save_path : str = os.path.join(save_dir, "deepsurv_weights.pth")
torch.save(model.state_dict(), deepsurv_save_path)
print(f"✅ DeepSurv 가중치 저장 완료 : {deepsurv_save_path}")

print("-" * 50)
print("🚀 대사 질환 예측 및 생존 분석 전체 파이프라인 종료")
print("-" * 50)

# 49. Export All Visualizations (Save & Zip)

In [ ]:
import os
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import shap
import torch
from sklearn.metrics import roc_curve, auc, brier_score_loss

# 폰트 및 마이너스 기호 설정
plt.rcParams[ "font.family" ] = "Malgun Gothic"
plt.rcParams[ "axes.unicode_minus" ] = False

# 저장 디렉토리 생성
save_dir : str = "./results"
if os.path.exists(save_dir) :
    shutil.rmtree(save_dir)
os.makedirs(save_dir, exist_ok = True)

print("🚀 최신 버전의 고해상도 그래프 일괄 저장을 시작합니다...")

# ---------------------------------------------------------
# 1. 고도화된 예측 확률 교정 곡선 (Calibration Analysis) 저장
# ---------------------------------------------------------
brier_val : float = brier_score_loss(y_true_arr, clf_ni.predict_proba(X_test_red.astype(np.float32))[ :, 1 ])

fig_cal, ax_cal = plt.subplots(figsize = ( 10, 10 ))
ax_cal.plot(prob_pred, prob_true, marker = "o", ls = "-", color = "#0047AB", lw = 3, label = f"TabNet (Brier : {brier_val:.4f})")
ax_cal.plot([ 0, 1 ], [ 0, 1 ], ls = "--", color = "#4B4B4B", label = "Ideal")

for p, o in zip(prob_pred, prob_true) :
    ax_cal.annotate(
        f"P : {p:.3f}\nO : {o:.3f}", 
        xy = (p, o), 
        xytext = (0, 15), 
        textcoords = "offset points", 
        ha = "center", va = "bottom",
        fontsize = 10, fontweight = "bold", color = "#0047AB",
        bbox = dict(boxstyle = "round,pad=0.3", edgecolor = "#0047AB", facecolor = "white", alpha = 0.9)
    )

ax_cal.set_title("예측 확률 교정 곡선 (Calibration Analysis)", fontsize = 18, fontweight = "bold", pad = 20)
ax_cal.set_xlabel("평균 예측 확률 (Mean Predicted Probability)", fontsize = 12)
ax_cal.set_ylabel("실제 관측 비율 (Fraction of Positives)", fontsize = 12)
ax_cal.set_ylim(-0.1, 1.1)
ax_cal.legend(loc = "lower right", fontsize = 12)
ax_cal.grid(alpha = 0.2)
plt.tight_layout()
fig_cal.savefig(os.path.join(save_dir, "01_calibration_curve.png"), dpi = 300)
plt.close(fig_cal)

# ---------------------------------------------------------
# 2. OOD Detection Histogram 저장
# ---------------------------------------------------------
fig_ood, ax_ood = plt.subplots(figsize = ( 12, 7 ))
counts, bins, patches = ax_ood.hist(anomaly_scores, bins = 50, color = "#4A90E2", edgecolor = "white", alpha = 0.7)

# 하위 5% 임계값 설정
threshold_val : float = np.percentile(anomaly_scores, 5)
ax_ood.axvline(threshold_val, color = "crimson", ls = "--", lw = 3, label = f"임계값 (하위 5% : {threshold_val:.4f})")

# 영역별 텍스트 설명 추가
ax_ood.text(threshold_val - 0.01, max(counts) * 0.8, "이상치 영역 (OOD) ", color = "crimson", ha = "right", fontweight = "bold", fontsize = 12)
ax_ood.text(threshold_val + 0.01, max(counts) * 0.8, " 정상 데이터 영역", color = "#0047AB", ha = "left", fontweight = "bold", fontsize = 12)

ax_ood.set_title("OOD Detection : Anomaly Score Distribution", fontsize = 18, fontweight = "bold", pad = 20)
ax_ood.set_xlabel("Anomaly Score (Higher is more normal)", fontsize = 12)
ax_ood.set_ylabel("Frequency", fontsize = 12)
ax_ood.legend(loc = "upper left", fontsize = 12)
ax_ood.grid(axis = "y", alpha = 0.2, ls = "--")
plt.tight_layout()
fig_ood.savefig(os.path.join(save_dir, "02_ood_detection.png"), dpi = 300)
plt.close(fig_ood)

# ---------------------------------------------------------
# 3. Time-Dependent ROC Curve 저장
# ---------------------------------------------------------
fig_roc, ax_roc = plt.subplots(figsize = ( 10, 8 ))
colors : list = [ "#0047AB", "crimson", "forestgreen" ]
for idx, t in enumerate(eval_times) :
    binary_labels : np.ndarray = ((time_test <= t) & (y_true_arr == 1)).astype(int)
    if 0 < np.sum(binary_labels) < len(binary_labels) :
        fpr, tpr, _ = roc_curve(binary_labels, risk_pred_test)
        roc_auc : float = auc(fpr, tpr)
        ax_roc.plot(fpr, tpr, color = colors[ idx ], lw = 3, label = f"{t}개월 예측 (AUC : {roc_auc:.4f})")

ax_roc.plot([ 0, 1 ], [ 0, 1 ], color = "black", ls = "--", lw = 1.5, alpha = 0.5, label = "Random")
ax_roc.set_title("시간 의존적 ROC 곡선 (Time-Dependent ROC)", fontsize = 18, fontweight = "bold", pad = 20)
ax_roc.set_xlabel("FPR (False Positive Rate)", fontsize = 12)
ax_roc.set_ylabel("TPR (True Positive Rate)", fontsize = 12)
ax_roc.legend(loc = "lower right", fontsize = 12)
ax_roc.grid(alpha = 0.3, ls = "--")
plt.tight_layout()
fig_roc.savefig(os.path.join(save_dir, "03_time_dependent_roc.png"), dpi = 300)
plt.close(fig_roc)

# ---------------------------------------------------------
# 4. 개별 환자 리포트 일괄 저장 함수
# ---------------------------------------------------------
def save_final_patient_report(patient_idx, case_name, threshold) :
    sample : np.ndarray = X_test_red[ patient_idx : patient_idx + 1 ]
    prob : float = clf_ni.predict_proba(sample.astype(np.float32))[ 0, 1 ]
    diag : str = "고위험군" if prob >= threshold else "정상"
    
    local_device = next(model.parameters()).device
    model.eval()
    with torch.no_grad() :
        risk_score : float = model(torch.tensor(sample, dtype = torch.float32).to(local_device)).cpu().item()
    
    # 4-1. Diagnosis Bar
    fig1, ax1 = plt.subplots(figsize = ( 12, 3 ))
    color : str = "crimson" if diag == "고위험군" else "#0047AB"
    bar_val : float = prob if prob > 0 else 0.001 
    ax1.barh([ "대사 질환 위험도" ], [ bar_val ], color = color, height = 0.4, alpha = 0.8)
    ax1.axvline(threshold, color = "black", ls = "--", lw = 2, label = f"임계값 ({threshold})")
    ax1.set_xlim(0, 1)
    ax1.set_title(f"[1/3] {case_name} 진단 결과 : {diag}", fontsize = 16, fontweight = "bold", pad = 15)
    ax1.legend(loc = "center right", fontsize = 12)
    ax1.text(prob + 0.015, 0, f"{prob:.4f}", va = "center", ha = "left", fontsize = 14, fontweight = "bold", color = color)
    plt.tight_layout()
    fig1.savefig(os.path.join(save_dir, f"04_{case_name}_1_Diagnosis.png"), dpi = 300)
    plt.close(fig1)

    # 4-2. SHAP Waterfall (UX 가이드 포함)
    shap_vals : np.ndarray = explainer_ni.shap_values(sample)
    ko_features : list = [ feature_mapping.get(f, f) for f in reduced_features ]
    base_val : float = explainer_ni.expected_value[ 0 ] if isinstance(explainer_ni.expected_value, np.ndarray) else explainer_ni.expected_value
    
    fig2 = plt.figure(figsize = ( 12, 6 ))
    shap.plots._waterfall.waterfall_legacy(explainer_ni.expected_value, shap_vals[ 0 ], feature_names = ko_features, max_display = 10, show = False)
    plt.title(f"[2/3] {case_name} 인공지능 진단 근거 상세", fontsize = 16, fontweight = "bold", pad = 25)
    
    guide_text : str = f"※ 해석 가이드\n출발 기준점 : 전체 환자 평균 위험도 ({base_val:.3f})\n막대 수치 : 기준점 대비 해당 지표의 '위험도 증감폭'"
    plt.figtext(0.01, 0.85, guide_text, fontsize = 10, color = "#4B4B4B", bbox = dict(facecolor = "whitesmoke", edgecolor = "lightgrey", boxstyle = "round,pad=0.5"))

    for text_obj in plt.gcf().findobj(mpl.text.Text) :
        text_val : str = text_obj.get_text()
        if "\u2212" in text_val or "−" in text_val :
            text_obj.set_text(text_val.replace("\u2212", "-").replace("−", "-"))
            
    plt.tight_layout(rect = [ 0, 0, 1, 0.95 ])
    fig2.savefig(os.path.join(save_dir, f"04_{case_name}_2_SHAP.png"), dpi = 300)
    plt.close(fig2)

    # 4-3. Survival Curve (앙상블 보간 적용)
    fig3, ax3 = plt.subplots(figsize = ( 12, 6 ))
    times : np.ndarray = np.sort(time_test)
    base_surv : np.ndarray = np.array([ kmf_low.predict(t) for t in times ])
    high_surv : np.ndarray = np.array([ kmf_high.predict(t) for t in times ])
    
    patient_surv : np.ndarray = (1.0 - prob) * base_surv + prob * high_surv
    
    ax3.plot(times, base_surv, color = "#0047AB", ls = "--", alpha = 0.4, label = "저위험군 평균")
    ax3.plot(times, high_surv, color = "crimson", ls = "--", alpha = 0.4, label = "고위험군 평균")
    ax3.plot(times, patient_surv, color = "black", lw = 3.5, label = "해당 환자 예상 경로")
    
    final_time : float = times[ -1 ]
    p_surv_final : float = patient_surv[ -1 ]
    surv_h_final : float = high_surv[ -1 ]
    surv_l_final : float = base_surv[ -1 ]

    # 텍스트 겹침 방지 오프셋
    offset_p, offset_l, offset_h = 0.0, 0.0, 0.0
    if abs(p_surv_final - surv_l_final) < 0.02 :
        offset_p = 0.01 if p_surv_final >= surv_l_final else -0.01
        offset_l = -0.01 if p_surv_final >= surv_l_final else 0.01
    elif abs(p_surv_final - surv_h_final) < 0.02 :
        offset_p = 0.01 if p_surv_final >= surv_h_final else -0.01
        offset_h = -0.01 if p_surv_final >= surv_h_final else 0.01
            
    ax3.text(final_time, p_surv_final + offset_p, f"  {p_surv_final:.3f}", color = "black", fontsize = 12, fontweight = "bold", va = "center")
    ax3.text(final_time, surv_h_final + offset_h, f"  {surv_h_final:.3f}", color = "crimson", fontsize = 10, va = "center")
    ax3.text(final_time, surv_l_final + offset_l, f"  {surv_l_final:.3f}", color = "#0047AB", fontsize = 10, va = "center")

    ax3.set_title(f"[3/3] {case_name} 예상 무병 생존 경로", fontsize = 16, fontweight = "bold", pad = 15)
    ax3.set_xlabel("시간 (개월)", fontsize = 12)
    ax3.set_ylabel("발병 안 할 확률", fontsize = 12)
    
    y_lower : float = max(0.0, min(p_surv_final, surv_h_final, surv_l_final) - 0.10) 
    ax3.set_xlim(0, final_time * 1.08) 
    ax3.set_ylim(y_lower, 1.05)
    ax3.legend(loc = "lower left", fontsize = 12)
    ax3.grid(alpha = 0.3, ls = "--")
    
    plt.tight_layout()
    fig3.savefig(os.path.join(save_dir, f"04_{case_name}_3_Survival.png"), dpi = 300)
    plt.close(fig3)

# ---------------------------------------------------------
# 5. 핵심 3대 스펙트럼 환자 리포트 일괄 저장
# ---------------------------------------------------------
all_probs_test = clf_ni.predict_proba(X_test_red.astype(np.float32))[ :, 1 ]
target_thresh = 0.3268

idx_high : int = int(np.argmax(all_probs_test))
idx_mid : int = int(np.argmin(np.abs(all_probs_test - 0.50)))
idx_low : int = int(np.argmin(all_probs_test))

save_final_patient_report(idx_high, "HighRisk", target_thresh)
save_final_patient_report(idx_mid, "Borderline", target_thresh)
save_final_patient_report(idx_low, "LowRisk", target_thresh)

# ---------------------------------------------------------
# 6. 폴더 압축 (ZIP)
# ---------------------------------------------------------
zip_filename : str = "Metabolic_CDSS_Final_Assets"
shutil.make_archive(zip_filename, 'zip', save_dir)
print(f"✨ 모든 그래프가 저장 및 압축되었습니다 : {zip_filename}.zip")